In [329]:
import os
import json
import pandas as pd
import numpy as np
import eyed3
import pymupdf
import fitz  # PyMuPDF
import re
import matplotlib.pyplot as plt

In [330]:
# Importing my own excel tracking of tracks (with difficulty and enjoyment) and mixes
my_bodypump_tracks_df = pd.read_excel('BodyPump.xlsx',sheet_name='Bodypump_tracks',dtype={'track_id':str,'bodypump_release_number':str,'track_type_id':str})
my_bodypump_tracks_df['track_title'] = my_bodypump_tracks_df['track_title'].str.upper()
my_bodypump_tracks_df['difficulty'] = my_bodypump_tracks_df['difficulty'].fillna(0).astype(int)
my_bodypump_tracks_df['enjoyment'] = my_bodypump_tracks_df['enjoyment'].fillna(0).astype(int)

my_bodypump_mixes_df = pd.read_excel('BodyPump.xlsx',sheet_name='Mix', dtype={'date':str,'track_id':str,'mix_id':str,'track_type_id':str,'track_title':str})
my_bodypump_mixes_df['track_title'] = my_bodypump_mixes_df['track_title'].str.upper()

In [331]:
# Define all the possible tracks
bodypump_tracks = {'01':'WARMUP','02':'SQUATS','03':'CHEST','04':'BACK','05':'TRICEPS','06':'BICEPS','07':'LUNGES','08':'SHOULDERS','09':'CORE','10':'COOLDOWN'}
bodypump_track_numbers_list = list(bodypump_tracks.keys())
bodypump_muscle_groups_list = list(bodypump_tracks.values())

In [332]:
# 1. Function to read the file and extract the bodypump release number
def read_pdf_file(pdf_file_name):
    # Open the PDF file
    pdf_document = fitz.open(pdf_file_name)

    # Storing the Bodypump release number
    pump_number_pattern = r'BODYPUMP_(\d+)'
    match_pump_number = re.search(pump_number_pattern,pdf_file_name)
    bodypump_release_number = match_pump_number.group(1)

    print("***Processing Bodypump release: {}***".format(bodypump_release_number))
    return (pdf_document, bodypump_release_number)

In [333]:
# 2. Function to extract the song title, order and duration
def extract_song_titles_and_info(pdf_document,bodypump_release_number):
    # Find and isolate the track titles. They can be identified as being on the page together with TECHNIQUE AND COACHING
    # Specify the text string we search for
    text_for_song_title = "TECHNIQUE AND COACHING"
    #text_for_song_title_BP102_104 = "MUSIC"
    text_for_song_title_BP85_100 = "TRACK FOCUS"
    # Store the page numbers where the target text string appears
    pages_with_song_title = []

    for page_num in range(pdf_document.page_count):
        page = pdf_document[page_num]
        text_blocks = page.get_text("blocks")
        
        # Combine text blocks into a single text string
        page_text = " ".join([b[4] for b in text_blocks])

        # Check if the target text is present on the page
        if text_for_song_title in page_text:
            pages_with_song_title.append(page_num)
        elif text_for_song_title_BP85_100 in page_text:
            pages_with_song_title.append(page_num+1)


    # extracting the song titles only
    song_titles = []
    for page in pages_with_song_title:
        page_to_process =  pdf_document.load_page(page)
        for block in page_to_process.get_text('blocks'):
            if text_for_song_title in block[4]:
                song_titles.append(block[4].split('\n', 1)[0])
            elif 'mins' in block[4]:
                if 'mins' in block[4].split('\n', 1)[0]:
                    song_titles.append(block[4].split('\n', 1)[0])
                elif 'mins' in block[4].split('\n', 1)[1]:
                    song_titles.append(block[4].split('\n', 1)[1])

    # Transform the list to a dataframe for easier use of manipulation
    list_numbers = []
    list_titles = []
    list_durations = []

    if int(bodypump_release_number) < 101:
        # Create a list of tuples with the extracted information
        extracted_data = []
        for i, item in enumerate(song_titles, start=1):
            title, duration = item.split(' > ')
            duration = duration.replace(' mins', '').replace('\n','')
            extracted_data.append((f"{i:02}", title, duration))

        # Create the DataFrame
        df = pd.DataFrame(extracted_data, columns=['Number', 'Title', 'Duration'])
        df = df.drop_duplicates()
    
    else:
        # cleaning up to remove the text strings that don't have the song info (should contain 'mins')
        text_min = 'mins'
        song_titles = [song for song in song_titles if text_min in song] # now we have a list of song titles with info

        for song in song_titles:
        # Use regular expressions to extract the number, title, and duration
            match = re.match(r"(\d{2})\. (.*?) (\d+:\d+)mins", song)

            if match:
                number = match.group(1)
                title = match.group(2)
                duration = match.group(3)

                list_numbers.append(number)
                list_titles.append(title)
                list_durations.append(duration)

        # Create a DataFrame from the lists
        df = pd.DataFrame({'Number': list_numbers, 'Title': list_titles, 'Duration': list_durations})
        df = df.drop_duplicates()

    return df

In [334]:
alternative_tracks_titles_to_exclude = ['TRICEPS - CHAMOMILE BREATH','BICEPS - MANERA']

In [335]:
# 3. Function to extract the list of choregraphy notes pages to process
def extract_pages_with_choregraphy_notes(pdf_document,bodypump_release_number):
# Finding the pages which have the choregraphy details. they are those after the "Warm up" page, before the last of the song title pages, and those not in the song titles pages list
# Specify the text string you want to search for
    text_for_warmup_page = "WARM-UP"
    text_for_warmup_page_BP_85_100 = "WARMUP"
    page_of_warm_up = 0 # initialize
    text_for_cooldown_page = "COOLDOWN"
    page_of_cooldown = 0 # initialize
    text_for_non_alternative_track = 'TRACK FOCUS' # Alternative tracks (like 03. ALT for BP 116) don't have the TRACK FOCUS. We will exclude those
    text_for_alternative_track = 'ALT\n' # Alternative tracks
    text_for_alternative_track_old = 'Alternative\n' # Alternative tracks
    text_for_alternative_track_45min = '45-MIN FORMAT' # Alternative tracks
    text_for_bonus_track = 'BONUS\n' # Bonus tracks
    text_for_track_description = 'TECHNIQUE AND COACHING' # Description pages
    tracks_list_pages = []
    alternative_tracks_list_pages = []
    track_description_list_pages = []

    for page_num in range(pdf_document.page_count):
        page = pdf_document[page_num]
        text_blocks = page.get_text("blocks")

        # Combine text blocks into a single text string
        page_text = " ".join([b[4] for b in text_blocks])

        if text_for_warmup_page in page_text or (text_for_warmup_page_BP_85_100 in page_text and text_for_non_alternative_track in page_text):   # Check if it is the warm up page
            page_of_warm_up = page_num

        if text_for_cooldown_page in page_text: # Check if it is the cooldown page
            page_of_cooldown = page_num
    
        if text_for_alternative_track in page_text or text_for_alternative_track_old in page_text \
            or alternative_tracks_titles_to_exclude[0] in page_text or alternative_tracks_titles_to_exclude[1] in page_text \
            or text_for_alternative_track_45min in page_text or text_for_bonus_track in page_text: # Check if it is an alternative/bonus track page:
            alternative_tracks_list_pages.append(page_num)
    
        if text_for_track_description in page_text or (int(bodypump_release_number)< 101 and text_for_non_alternative_track not in page_text): # Check if it is a track description page:
            track_description_list_pages.append(page_num)

    #print("Cooldown page:",page_of_cooldown)
    #print("Alternative tracks pages:",alternative_tracks_list_pages)
    #print("Track description pages:",track_description_list_pages)
    # all pages with choregraphy notes (difference between all pages between warm up and last, and pages containing song titles):
    pages_with_choregraphy_notes = [x for x in range(page_of_warm_up,page_of_cooldown+1) if x not in alternative_tracks_list_pages and x not in track_description_list_pages]
    #print("Pages containing the choregraphy notes:", pages_with_choregraphy_notes)
    print(f"Warm up page: {page_of_warm_up} Cooldown page {page_of_cooldown} \n Pages containing the choregraphy notes:{pages_with_choregraphy_notes}")
    return pages_with_choregraphy_notes

In [336]:
# 4.Function to extract the move details (time index, lyrics, counts, move and reps) from the text
def extract_columns(row):
    # We will need to align vertically the columns, for now we will keep 5 column, 1st is time index, 2nd is lyrics guide, 3rd is the counts, 4th is everything related to the move, to be split after, last is the number of reps.
    pattern_time = r'(\d+):(\d+)'
    pattern_time_and_words = r'(\d+):(\d+)\s\w+'
    pattern_moves_counts = r'\b(?:\d+|½)x\d+\b'

    if len(re.findall(r'(\d)\n(\d):', row))>0:
        row = row[2:]
    # Split the row into lines
    row = re.sub(r'(\d+:\d+)\s', r'\1\n', row) # ensuring we have a /n after the time stamp
    lines = row.split('\n')
    
    # Extract the desired columns
    col1=lines[0]
    col2=lines[1] if len(lines) > 1 else ''

    match_moves_counts = re.search(pattern_moves_counts, ' '.join(lines[1:3])) if len(lines) >2 else ''
    col3 = ''
    if match_moves_counts:
        col3 = match_moves_counts.group(0)
    col4 = ' '.join(lines[3:-1]) if len(lines) > 3 else ''
    col5 = ''
    if lines[-2].strip():
        match = re.search(r'\d+x', lines[-2])
        if match:
            col5 = match.group()
    
    return [col1, col2, col3, col4,col5]

In [337]:
# 5. Function to run through the document and process the choregraphy notes to return a dataframe
def process_choregraphy_notes(pdf_document, bodypump_release_number, pages_with_choregraphy_notes_list):

    df_list = []
    pages_list = []
    # we first append all the relevant pages together
    for i,page_index in enumerate(pages_with_choregraphy_notes_list):
        page = pdf_document[pages_with_choregraphy_notes_list[i]]
        pages_list.append(page)

    track_nb_pattern = r'(\d+).\s+([A-Z]+(?:-[A-Z]+)*)'
    row_index_of_track_nb_on_page = 1

    # The rows describing moves contain the counts (in the music). These are rows containing 2x8, 4x8, 8x8 or even ½x8 as a pattern.
    pattern_moves_counts = r'(\b(?:\d+|½)x\d+\b)'

    for page in pages_list:
        # Extract elements at 5th  positions from each tuple using list comprehension
        extracted_data = [tup[4] for tup in page.get_text("blocks")]

        # extract track info (always on the 2nd row of the extracted data text, except when the instructions are spread over 2 pages)
        if int(bodypump_release_number) < 101:
            for  (number, track) in (bodypump_tracks.items()):
                if track in ''.join(extracted_data[0:3]):
                    muscle_group = track
                    track_number = number

        elif re.search(track_nb_pattern,extracted_data[row_index_of_track_nb_on_page]):
            match_track_nb = re.search(track_nb_pattern,extracted_data[row_index_of_track_nb_on_page])
            track_number = match_track_nb.group(1)
            muscle_group = match_track_nb.group(2)
        
        # The data is misaligned, with some rows containing the number of repetitions being alone, in the next line compared to the description of the move (eg. Combination 1 track 1, BP 116)
        # we first copy them back to the line above

        # Initialize an empty list to store the rows
        final_rows = []

        for i,row in enumerate(extracted_data):    
            if row.endswith('x\n') and len(row) <6:
                final_rows[i-1] += '\n' + row
            final_rows.append(row)
            i = i+1

        # Initializing a df to capture the block number first and then the text to be split
        df = pd.DataFrame(columns=['block', 'text'])

        current_block = 1
        previous_block = 0

        for line in final_rows:
            if line[:2] == str(current_block) + '\n':
                previous_block = current_block
                current_block += 1
            df = pd.concat([df,pd.DataFrame({'block': [previous_block], 'text': [line]})])

        #  Keeping only the rows with moves (with the pattern of moves counts)
        df = df[df['text'].str.extract(pattern_moves_counts, expand=False).notnull()]

        # Some rows have double \n, causing the data to be misaligned column wise this time. Let's replace the \n\n occurrences with \n
        df['text_clean'] = df['text'].str.replace('\n\n', '\n')

        # Create a second DataFrame to explode the text into the predetermined columns format
        df2 = pd.DataFrame([extract_columns(row) for row in df['text_clean']],columns=['time_index','lyrics','beat_counts','move_desc','reps'])
        df2['bodypump_release_number'] = bodypump_release_number
        df2['track_number'] = track_number
        df2['muscle_group'] = muscle_group

        # Re-combining the first df (with block number, raw and clean text) and the second df into one
        df = df.reset_index(drop=True)
        df2 = df2.reset_index(drop=True)
        final_df = pd.concat([df,df2],axis=1)

        df_list.append(final_df) # append the dataframe for that track to the list

    # Finally, concatenate all dataframes together into one dataframe for the release
    all_df= pd.concat(df_list,ignore_index=True, axis=0)
    return all_df

In [338]:
# 6. Function to extract info from the move description, such as move, tempo and reps, as well breakdown combinations
def extract_info(move_desc):  
    # Extract combos, moves, tempos and reps
    no_rep = ''
    combo = ''
    moves = []
    tempos = []
    reps = []

    
    tempo_pattern = r'\d+/\d+(?:/\d+)?'
    reps_pattern = r"(?<=\s)\d+x"             # r'\d+x'
    
    # find if it's a HOLD, SET position, TRANSITION etc. row
    for no_rep_word in no_rep_list:
        if move_desc.startswith(no_rep_word) or move_desc[4:].startswith(no_rep_word):
                no_rep = no_rep_word

    combo_pattern = r'COMBINATION\s*(\d+)?'
    combo_pattern2 = r'COMBO\s*(\d+)?'
    match_combo = re.search(combo_pattern,move_desc)
    match_combo2 = re.search(combo_pattern2,move_desc)
    if match_combo:
        if match_combo.group(1):
            combo = 'COMBINATION ' + str(match_combo.group(1))
        else:
            combo = 'COMBINATION'
    else:
        if match_combo2:
                if match_combo2.group(1):
                        combo = 'COMBO ' + str(match_combo2.group(1))
                else:
                        combo = 'COMBO'
        else:
                combo = ''   

    # Find the first move and tempo
    move1_match = re.search(r'\b(' + '|'.join(moves_list) + r')\b', move_desc, re.IGNORECASE)
    match_tempo = re.search(tempo_pattern, move_desc)
    match_reps = re.search(reps_pattern, move_desc)
    if move1_match:
            move1 = move1_match.group(0)
            moves.append(move1)
            move_desc = re.sub(move1, '',move_desc,count=1)
    if match_tempo:
            first_tempo = match_tempo.group()
            tempos.append(first_tempo)
            move_desc = re.sub(first_tempo, '', move_desc, count=1)
    if match_reps:
            first_reps = match_reps.group()
            reps.append(first_reps)
            move_desc = re.sub(first_reps, '', move_desc, count=1)
        
    # Find the second move and tempo
    move2_match = re.search(r'\b(' + '|'.join(moves_list) + r')\b', move_desc, re.IGNORECASE)
    match_tempo = re.search(tempo_pattern, move_desc)
    match_reps = re.search(reps_pattern, move_desc)
    if move2_match:
            move2 = move2_match.group(0)
            moves.append(move2)
            move_desc = re.sub(move2, '',move_desc,count=1)
    if match_tempo:
            second_tempo = match_tempo.group()
            tempos.append(second_tempo)
            move_desc = re.sub(second_tempo, '', move_desc, count=1)
    if match_reps:
            second_reps = match_reps.group()
            reps.append(second_reps)
            move_desc = re.sub(second_reps, '', move_desc, count=1)
        
    # Find the third move and tempo
    move3_match = re.search(r'\b(' + '|'.join(moves_list) + r')\b', move_desc, re.IGNORECASE)
    match_tempo = re.search(tempo_pattern, move_desc)
    match_reps = re.search(reps_pattern, move_desc)
    if move3_match:
            move3 = move3_match.group(0)
            moves.append(move3)
            move_desc = re.sub(move3, '',move_desc ,count=1)
    if match_tempo:
            third_tempo = match_tempo.group()
            tempos.append(third_tempo)
            move_desc = re.sub(third_tempo, '', move_desc, count=1)
    if match_reps:
            third_reps = match_reps.group()
            reps.append(third_reps)
            move_desc = re.sub(third_reps, '', move_desc, count=1)
        
    # Find the fourth move and tempo
    move4_match = re.search(r'\b(' + '|'.join(moves_list) + r')\b', move_desc, re.IGNORECASE)
    match_tempo = re.search(tempo_pattern, move_desc)
    match_reps = re.search(reps_pattern, move_desc)
    if move4_match:
            move4 = move4_match.group(0)
            moves.append(move4)
            move_desc = re.sub(move4, '',move_desc,count=1)
    if match_tempo:
            fourth_tempo = match_tempo.group()
            tempos.append(fourth_tempo)
            move_desc = re.sub(fourth_tempo, '', move_desc, count=1)
    if match_reps:
            fourth_reps = match_reps.group()
            reps.append(fourth_reps)
            move_desc = re.sub(fourth_reps, '', move_desc, count=1)
    
    # Populate the output columns
    move1 = moves[0] if moves else ''
    tempo1 = tempos[0] if tempos else ''
    reps1 = reps[0] if reps else ''
    move2 = moves[1] if len(moves) > 1 else ''
    tempo2 = tempos[1] if len(tempos) > 1 else ''
    reps2 = reps[1] if len(reps) > 1 else ''
    move3 = moves[2] if len(moves) > 2 else ''
    tempo3 = tempos[2] if len(tempos) > 2 else ''
    reps3 = reps[2] if len(reps) > 2 else ''
    move4 = moves[3] if len(moves) > 3 else ''
    tempo4 = tempos[3] if len(tempos) > 3 else ''
    reps4 = reps[3] if len(reps) > 3 else ''

    return pd.Series([no_rep,combo, move1,tempo1,reps1,move2,tempo2,reps2,move3,tempo3,reps3,move4,tempo4,reps4])

In [339]:
# 7. Function to apply the move_description info extraction to the dataframe, format, and capture the combination occurrence when there are multiple occurrences
def apply_move_description_extraction_and_format(raw_df):
    raw_df[['no_rep','combo', 'move1', 'tempo1','reps1', 'move2', 'tempo2','reps2','move3', 'tempo3','reps3','move4', 'tempo4','reps4']] = raw_df['move_desc'].apply(extract_info)

    # Cleaning up the rows that are missing a move when there should be one because it's not a 'no_rep' (this is due to some moves not being repeated in the text sometimes when there is a tempo change)
    raw_df['move1'] = raw_df['move1'].replace('', pd.NA).fillna(raw_df['move1'].shift(1)).where(raw_df['no_rep'] == '').replace(pd.NA,'')
    raw_df['move1'] = raw_df['move1'].replace('', pd.NA).fillna(raw_df['move1'].shift(1)).where(raw_df['no_rep'] == '').replace(pd.NA,'')
    raw_df['move1'] = raw_df['move1'].replace('', pd.NA).fillna(raw_df['move1'].shift(1)).where(raw_df['no_rep'] == '').replace(pd.NA,'')

    # Cleaning up the combos so that the value in the cell is unique at release + track number
    raw_df['combo_clean'] = raw_df['bodypump_release_number']+'-'+raw_df['track_number']+'-'+raw_df['combo']
    raw_df['combo_clean'] = np.where(raw_df['combo'] == '', '', raw_df['combo_clean'])

    # When a combination happens a Nth time in a song, it is not described again in the notes, so we don't have the moves in our dataframe. 
    # We will look for the 1st description of each combo, find the index and update the dataframe with the info from the 1st time the combo was described
    combos_described = []
    current_combination = None
    combo_indices = raw_df[raw_df['combo_clean'].str.contains('COMB')].index

    for i in combo_indices:
        current_combination = raw_df.loc[i,'combo_clean']
        if current_combination in combos_described:
            # update rows with info of first occurrence
            index_of_first_occ = raw_df[raw_df['combo_clean'].str.contains(current_combination)].index[0]
            raw_df.loc[i,['move1','tempo1','reps1','move2','tempo2','reps2','move3','tempo3','reps3','move4','tempo4','reps4']] = raw_df.loc[index_of_first_occ,['move1','tempo1','reps1','move2','tempo2','reps2','move3','tempo3','reps3','move4','tempo4','reps4']]
        else:
            combos_described.append(current_combination)

    return raw_df

In [340]:
# 8. Function to append the rows a block when a set is repeated and the instructions only mention to "repeat" the set. Takes the full final dataframe in and outputs its with the rows added and reindexed.
def append_repeated_set(processed_df):    # Defining a function to capture an instruction to repeat a set and then appending the rows of the set in question:
    
    # Defining a block_clean column that is unique: Bodypumprelease-track number-block
    processed_df['block_clean']=(processed_df['bodypump_release_number'])+'-'+(processed_df['track_number'])+'-'+processed_df['block'].astype(str) 
    
    # Initialize variables
    block_to_repeat = None
    block_clean_to_repeat = None
    current_bodypump_release_number = None
    current_track_number = None
    current_block = None
    current_block_clean = None
    rows_to_append = pd.DataFrame(columns = processed_df.columns)
    rows_to_append_final = pd.DataFrame(columns = processed_df.columns)
    output_df = pd.DataFrame(columns = processed_df.columns)

    # Looping on all the rows of the dataframe to detect when there is a "Repeat set in the instruction move_desc". 
    # Then getting the number of the block to repeat, appending the related rows, and updating block number to keep the current one
    for index, row in processed_df.iterrows():
        if "Repeat SET" in str(row['move_desc']) or "REPEAT SET" in str(row['move_desc']) or "Repeat BLOCK" in str(row['move_desc']) or "REPEAT BLOCK" in str(row['move_desc']) :
            match_int = re.search(r"(?:BLOCK |SET )(\d+)",row['move_desc']) # find the integer
            block_to_repeat = int(match_int.group(1))
            current_bodypump_release_number = row['bodypump_release_number']
            current_track_number = row['track_number']
            block_clean_to_repeat = str(current_bodypump_release_number)+'-'+str(current_track_number)+'-'+str(block_to_repeat)
            current_block = row['block']
            current_block_clean = row['block_clean']
            rows_to_append = processed_df[processed_df['block_clean']==block_clean_to_repeat]
            rows_to_append.loc[:, 'block'] = current_block
            rows_to_append.loc[:, 'block_clean'] = current_block_clean
            rows_to_append_final = pd.concat([rows_to_append_final, rows_to_append], ignore_index= True)
        
    output_df = pd.concat([processed_df, rows_to_append_final], ignore_index= True)
    
    return output_df

In [341]:
# 9. Function to finally join back the song info and details and clean up
def add_song_info(complete_df, song_info_df):
   clean_df = pd.merge(complete_df, song_info_df,how='left', left_on='track_number', right_on = 'Number')
   final_column_order = ['bodypump_release_number','track_number','muscle_group','Title','Duration','block','block_clean','time_index', 'lyrics', 'beat_counts', 'move_desc', 'reps','no_rep','combo_clean','move1', 'tempo1', 'reps1', 'move2',
       'tempo2', 'reps2', 'move3', 'tempo3', 'reps3', 'move4', 'tempo4','reps4']
   clean_df = clean_df[final_column_order]

   clean_df['move2'] = clean_df.apply(lambda row: '' if row['combo_clean'] == '' else row['move2'], axis =1)
   clean_df['tempo2'] = clean_df.apply(lambda row: '' if row['combo_clean'] == '' else row['tempo2'], axis =1)
   clean_df['reps2'] = clean_df.apply(lambda row: '' if row['combo_clean'] == '' else row['reps2'], axis =1)

   clean_df = clean_df.sort_values(by=['track_number','block_clean','time_index'], ascending=[True,True,True]).reset_index(drop=True)

   return clean_df

In [342]:
# Define all the possible moves in a strandardized way
no_rep_list = ['HOLD','Set Position','SET Position','SET POSITION','SET','Set up','TRANSITION','Transition','Repeat','REPEAT','RECOVERY','Shoulder Roll','SHOULDER ROLL','Plate down, shake out','Shake legs','Preview']

squats_02_moves_list = ['SQUAT MID STANCE','SQUAT WIDE STANCE','SQUAT WIDER STANCE','3x BOTTOM HALF PULSE SQUAT','7x BOTTOM HALF PULSE SQUAT','2x BOTTOM HALF SQUAT','2x BOTTOM-HALF SQUAT','BOTTOM HALF SQUAT','TRIPLE PULSE SQUAT with CALF RAISE','PAUSE SQUAT','STAGGERED SQUAT',\
                        '4x PULSE BOTTOM HALF SQUAT','2x PULSE SQUAT with CALF RAISE','PULSE SQUAT with CALF RAISE','7x PULSE SQUAT','4x PULSE SQUAT','2x PULSE SQUAT','TRIPLE PULSE SQUAT','PULSE SQUAT','TOP HALF SQUAT',\
                        '2x BOTTOM HALF FRONT SQUAT','4x BOTTOM HALF FRONT SQUAT','TRIPLE PULSE FRONT SQUAT','BARBELL FRONT SQUAT','FRONT SQUAT','SQUAT BOTTOM HALF PULSE','SQUAT PLATE PRESS','SQUAT']
chest_03_moves_list = ['BENCH CHEST PRESS','TRIPLE PULSE CHEST PRESS','4x BOTTOM HALF PULSE CHEST PRESS','3x PULSE CHEST PRESS','4x BOTTOM HALF CHEST PRESS','PULSE CHEST PRESS','STAGGERED CHEST PRESS','CHEST PRESS BOTTOM HALF PULSE' ,'CHEST PRESS','BOTTOM HALF PULSE',\
                    '4x PULSE CHEST PRESS','3x PULSE A-PRESS','4x PULSE A-PRESS','TRIPLE PULSE A-PRESS','PULSE A-PRESS','A-PRESS','PULSE NARROW PLATE PRESS','NARROW PLATE PRESS',\
                    '3x PULSE CHEST FLY','4x PULSE CHEST FLY','CHEST FLY',\
                    '3x PULSE PUSHUP','4x PULSE BENCH PUSHUP','4x PULSE PUSHUP','TRIPLE PULSE PUSHUP','PUSHUP PULSE','WIDE-GRIP PUSHUP','WIDE PUSHUP','MID-GRIP PUSHUP','TRIPLE WIDE-GRIP PUSHUP','STAGGERED BENCH PUSHUP','STAGGERED PUSHUP','PULSE BENCH PUSHUP','WALKING BENCH PUSHUP','BENCH PUSHUP','PUSHUP',\
                    'FAST PLANK FOOT TAP','SLOW PLANK FOOT TAP','PLANK FOOT TAP','ALT STRAIGHT LEG LIFT']
back_04_moves_list = ['WIDE DEADLIFT','PLATE SUMO DEADLIFT','DEADLIFT',\
                    'DOUBLE DEADROW','TRIPLE DEADROW','TRIPLE WIDE DEADROW','WIDE DEADROW','WIDE TRIPLE DEADROW','TRIPLE PULSE KNEELING SINGLE-ARM PLATE ROW','KNEELING SINGLE-ARM PLATE ROW','TRIPLE SINGLE-ARM PLATE DEADROW','SINGLE- ARM PLATE ROW','TRIPLE PLATE DEADROW','TRIPLE SPLIT PLATE ROW','TRIPLE ROW','PLATE DEADROW','DEADROW',\
                    'TRIPLE NARROW PLATE ROW', 'TRIPLE WIDE PLATE ROW','NARROW PLATE ROW', 'WIDE PLATE ROW',\
                    'CLEAN AND PRESS','CLEAN & PRESS','PLATE CLEAN & PRESS','PLATE CURL SQUAT PRESS','PLATE CURL SQUAT & PRESS','PLATE SQUAT CURL & PRESS','PLATE CLEAN & PRESS','PLATE CLEAN AND PRESS','PLATE TRIPLE SQUAT PRESS','PLATE SQUAT CURL POWER PRESS','PLATE SQUAT CURL PRESS','PLATE SQUAT PRESS','DROP SQUAT PLATE PRESS',\
                    'POWER PRESS',\
                    'WIDE HIGH PULL','LOW PULL','HIGH PULL']
triceps_05_moves_list = ['HOLD TRICEP PUSHUP','TOP HALF PULSE BENCH TRICEP PUSHUP','3x PULSE TRICEP BENCH PUSHUP', 'TRIPLE PULSE BENCH TRICEP PUSHUP','TRICEP PUSHUP BOTTOM HALF PULSE','BENCH TRICEP PUSHUP','TRICEP PUSHUP','TRICEP BENCH PUSHUP',\
                        'TRICEP BARBELL O/H EXTENSION','TRICEP BARBELL EXTENSION','TRICEP PRESS PULLOVER','TRICEP BARBELL PRESS','TRICEP PRESS PULL-OVER', 'TRICEP PRESS / PULLOVER',\
                        'TRIPLE KNEELING PLATE TRICEP KICKBACK ROW','TRIPLE KNEELING TRICEP KICKBACK','TRIPLE PULSE TRICEP KICKBACK','TRIPLE PULSE TRICEP BENCH KICKBACK','KNEELING TRICEP KICKBACK','TRICEP BENCH KICKBACK','TRICEP KICKBACK ROW','TRICEP KICKBACK','KICKBACK ROW', \
                        '3x PULSE SEATED O/H TRICEP PLATE EXTENSION','TRIPLE PULSE KNEELING O/H TRICEP EXTENSION','TRIPLE PULSE STANDING O/H TRICEP EXTENSION','TRIPLE PULSE SEATED O/H TRICEP PLATE EXTENSION','PULSE KNEELING O/H TRICEP EXTENSION','KNEELING O/H TRICEP PLATE EXTENSION','KNEELING O/H TRICEP EXTENSION','STANDING O/H TRICEP PLATE EXTENSION','STANDING O/H TRICEP EXTENSION','PULSE STANDING PLATE O/H TRICEP EXTENSION','SEATED O/H TRICEP PLATE EXTENSION','STANDING PLATE O/H TRICEP EXTENSION',\
                        'SEATED TRICEP OVERHEAD EXTENSION','STANDING TRICEP OVERHEAD EXTENSION', 'STANDING OVERHEAD TRICEP EXTENSION BOTTOM HALF PULSE','STANDING OVERHEAD TRICEP EXTENSION','STANDING O/H EXTENSION','SEATED O/H TRICEP EXTENSION','TRIPLE PULSE STANDING O/H TRICEP EXTENSION','TRIPLE PULSE TRICEPS OVERHEAD EXTENSION','PULSE TRICEP EXTENSION','TRICEP EXTENSION',\
                        'BOTTOM HALF PULSE TRICEP PRESS','8x TRICEP PRESS BOTTOM HALF PULSE','TRICEP PRESS BOTTOM HALF PULSE','TRIPLE PULSE TRICEP PRESS','3x PULSE TRICEP PRESS','PULSE TRICEP PRESS','TRICEP PRESS',\
                        'TOP HALF PULSE TRICEP NARROW PRESS','BOTTOM HALF PULSE TRICEP NARROW PRESS','TRICEP NARROW PRESS',\
                        'PULSE TRICEP DIP','TRICEP DIP']
biceps_06_moves_list = ['TRIPLE MID RANGE PULSE PLATE CURL','MID-RANGE TRIPLE PULSE','ALT SINGLE-ARM BICEP PLATE CURL','ALT SINGLE ARM BICEP PLATE CURL','FAST ALT SINGLE-ARM BICEP PLATE CURL','MID-RANGE PULSE BICEP PLATE CURL','ALT PLATE CURL','BICEP PLATE CURL','PLATE CURL',\
                        'BICEP CURL MIDRANGE PULSE','3x MID-RANGE PULSE','TRIPLE MID-RANGE PULSE','MID-RANGE ISOMETRIC HOLD','PULSE MID-RANGE CURL','MID-RANGE PULSE',\
                        'BARBELL BICEP CURL','BICEP BARBELL CURL','BICEP CURL',\
                        'TRIPLE PREACHER CURL','PREACHER CURL',\
                        'BICEP PLATE ROW','PLATE ROW','BICEP ROW']
lunges_07_moves_list = ['BOTTOM HALF PULSE LUNGE','BACK STEPPING TRIPLE PULSE LUNGE','3x BENCH BACKWARD-STEPPING PULSE LUNGE','3x PULSE BENCH BACKWARD-STEPPING LUNGE','3x PULSE ALT BENCH BACKWARD-STEPPING LUNGE','ALT BACK STEPPING LUNGE','ALT BACKWARD STEPPING LUNGE','ALT BACKWARD-STEPPING LUNGE','TRIPLE PULSE BACK STEPPING LUNGE','TRIPLE PULSE BACKWARD-STEPPING LUNGE','BACKWARD-STEPPING LUNGE','STEPPING LUNGE',\
                        '7x BOTTOM HALF PULSE PLATE SQUAT','3x BOTTOM HALF PULSE PLATE SQUAT','PLATE FRONT SQUAT','TRIPLE PULSE PLATE FRONT SQUAT','PULSE PLATE FRONT SQUAT','TRIPLE PULSE PLATE WIDE FRONT SQUAT','TRIPLE PULSE PLATE WIDER FRONT SQUAT','PULSE SQUAT',\
                        '2x PULSE PLATE LUNGE WITH STEP','PULSE PLATE LUNGE WITH STEP','PLATE LUNGE','TRIPLE PULSE LUNGE','16x PULSE LUNGE','7x BOTTOM HALF PULSE LUNGE','7x PULSE LUNGE','3x PULSE LUNGE','PULSE LUNGE','LUNGE']
shoulders_08_moves_list = ['WIDE UPRIGHT ROW','PLATE UPRIGHT ROW','KNEELING UPRIGHT ROW','PULSE UPRIGHT ROW','UPRIGHT ROW','TRIPLE WIDE ROW',\
                        'STANDING PLATE SIDE RAISE EXTENSION','STANDING SIDE ROTATOR RAISE','STANDING MAC RAISE','ALT MAC RAISE','KNEELING SIDE RAISE','STANDING SIDE RAISE','TRIPLE PULSE KNEELING REAR RAISE','KNEELING REAR RAISE', 'STANDING PLATE SIDE RAISE','STANDING PLATE FORWARD RAISE','KNEELING FORWARD RAISE','STANDING FORWARD RAISE','SPLIT REAR RAISE','ROTATOR RAISE', 'SIDE RAISE',\
                        'STANDING O/H SHOULDER PRESS','STANDING SHOULDER PRESS','STANDING O/H PRESS','O/H SHOULDER PRESS','STANDING PLATE O/H PRESS','PULSE STANDING PLATE O/H PRESS','SHOULDER PRESS',\
                        'I RAISE','Y RAISE','T RAISE','Y-RAISE','T-RAISE','A-RAISE',\
                        'STANDING ARNOLD PRESS','STANDING OVERHEAD PRESS','STANDING PUSH PRESS','PUSH PRESS',\
                        'STANDING REAR DELTOID FLY WITH STEP','STANDING REAR DELTOID FLY','STANDING TRIPLE REAR DELTOID FLY','KNEELING REAR DELTOID RAISE','STANDING REAR DELTOID RAISE','PULSE REAR DELTOID RAISE','REAR DELTOID RAISE','KNEELING REAR DELTOID FLY','TIPPING REAR DELTOID FLY WITH TWIST',\
                        'MID PUSHUP','MID-GRIP PUSHUP','WALKING UP / DOWN BENCH PLANK',\
                        'SLOW MOUNTAIN CLIMBER']
core_09_moves_list = ['TRIPLE PULSE C-CRUNCH','C CRUNCH HOLD with ALT LEG EXTENSION', 'C CRUNCH HOLD with DOUBLE LEG EXTENSION','C-CRUNCH / SINGLE LEG EXTENSION','BICYCLE CRUNCH', 'PULSE PLATE LIFT','15x PULSE PLATE CRUNCH','16x PULSE PLATE CRUNCH','16x PULSE PLATE LIFT','PULSE C-CRUNCH','3x PULSE PLATE CRUNCH','PLATE CRUNCH','C CRUNCH','C-CRUNCH','PULSE CRUNCH',\
                      'SINGLE-ARM PLATE LIFT','BENCH CRUNCH','BENCH REVERSE CRUNCH',\
                      'CROSS CRAWL','ALT CROSS CRAWL','SHOOT OUT','ALT SINGLE-LEG EXTENSION','DOUBLE-LEG EXTENSION','SINGLE LEG EXTENSION','ALT SINGLE-LEG HIP BRIDGE WITH LEG EXTENSION','ALT SINGLE-LEG HIP BRIDGE WITH PIKE LEG EXTENSION',\
                      'HOVER FORWARD / BACK','DIAGONAL LEG LIFT','HOVER ALT SINGLE-LEG LIFT', 'LEG LIFT','SIDE HOVER WITH ROTATION','ALT WALKING PLANK/HOVER','ROTATING PLATE SIDE HOVER','ROTATING SIDE HOVER','PULSING PLATE SIDE HOVER','SIDE HOVER PULSE','15x PULSE SIDE HOVER','WALKING HOVER','SLOW HOVER TAP','FAST HOVER TAP','ROTATING HOVER','HOVER',\
                      'HORSE STANCE','MOUNTAIN CLIMBER','SPRINTER',\
                      'ALT FAST SHOULDER TAP PLANK','ROTATING SIDE PLANK','SIDE PLANK ROTATION','SIDE PLANK','PLANK',\
                      'HIP BRIDGE with ALT KNEE LIFT','PULSE HIP BRIDGE','HIP BRIDGE']

moves_list = squats_02_moves_list + chest_03_moves_list + back_04_moves_list + triceps_05_moves_list + biceps_06_moves_list + lunges_07_moves_list + shoulders_08_moves_list + core_09_moves_list

In [343]:
# Initializing with Bodypump 116
file_name_116 = "BODYPUMP_116_(BODYPUMP116ChoreographyNotes_row_en_app_print.pdf).pdf"     
(document_file_116,bodypump_release_number_116) =  read_pdf_file(file_name_116)    # step 1
song_info_df_116 = extract_song_titles_and_info(document_file_116,bodypump_release_number_116)  # step 2
pages_with_choregraphy_notes_116 = extract_pages_with_choregraphy_notes(document_file_116,bodypump_release_number_116)  # step 3
raw_df_116 = process_choregraphy_notes(pdf_document=document_file_116, 
                          bodypump_release_number=bodypump_release_number_116, 
                          pages_with_choregraphy_notes_list=pages_with_choregraphy_notes_116) # step 4-5
processed_df_116 = apply_move_description_extraction_and_format(raw_df_116)     # step 6-7
complete_df_116 = append_repeated_set(processed_df=processed_df_116)    # step 8
final_df_116 = add_song_info(complete_df=complete_df_116, song_info_df = song_info_df_116)  # step 9

***Processing Bodypump release: 116***
Warm up page: 14 Cooldown page 37 
 Pages containing the choregraphy notes:[14, 16, 18, 19, 22, 25, 29, 31, 33, 35, 37]


In [344]:
file_name_list = ["BODYPUMP_105_(BODYPUMP105ChoreographyBooklet_row_en_app_print.pdf).pdf",\
                  "BODYPUMP_106_(BODYPUMP106ChoreographyNotes_row_en_app_print.pdf).pdf",\
                  "BODYPUMP_107_(BODYPUMP107ChoreographyNotes_row_en_app_print.pdf).pdf",\
                  "BODYPUMP_108_(BODYPUMP108ChoreographyNotes_row_en_app_print.pdf).pdf",\
                  "BODYPUMP_109_(BODYPUMP109ChoreographyNotes_row_en_app_print.pdf).pdf",\
                  "BODYPUMP_111_(BODYPUMP111ChoreographyNotes_row_en_app_print.pdf).pdf",\
                  "BODYPUMP_117_(BODYPUMP117ChoreographyNotes_row_en_app_print.pdf).pdf",\
                  "BODYPUMP_118_(BODYPUMP118ChoreographyNotes_row_en_app_print.pdf).pdf",\
                  "BODYPUMP_119_(BODYPUMP119ChoreographyNotes_row_en_app_print.pdf).pdf",\
                  "BODYPUMP_120_(BODYPUMP120ChoreographyNotes_row_en_app_print.pdf).pdf",\
                  "BODYPUMP_123_(BODYPUMP123ChoreographyNotes_row_en_app_print.pdf).pdf",\
                  "BODYPUMP_124_(BODYPUMP124ChoreographyNotes_row_en_app_print.pdf).pdf",\
                  "BODYPUMP_127_(BODYPUMP127ChoreographyNotes_row_en_app_print.pdf).pdf",\
                  "BODYPUMP_98 Choreography Booklet.pdf",\
                  "BODYPUMP_99 Choreography Booklet.pdf",\
                  "BODYPUMP_96 Choreography Booklet.pdf",\
                  "BODYPUMP_95 Choreography Booklet.pdf",\
                  "BODYPUMP_94 Choreography Booklet.pdf"]
combined_df = final_df_116

for file_name in file_name_list:
    (document_file,bodypump_release_number) =  read_pdf_file(file_name)   # step 1
    song_info_df = extract_song_titles_and_info(document_file,bodypump_release_number)  # step 2
    pages_with_choregraphy_notes = extract_pages_with_choregraphy_notes(document_file,bodypump_release_number)  # step 3
    raw_df = process_choregraphy_notes(pdf_document=document_file, 
                          bodypump_release_number=bodypump_release_number, 
                          pages_with_choregraphy_notes_list=pages_with_choregraphy_notes) # step 4-5
    processed_df = apply_move_description_extraction_and_format(raw_df)     # step 6-7
    complete_df = append_repeated_set(processed_df=processed_df)    # step 8
    final_df = add_song_info(complete_df=complete_df, song_info_df = song_info_df)  # step 9

    combined_df = pd.concat([combined_df, final_df], ignore_index= True)
    combined_df = combined_df.sort_values(by=['bodypump_release_number','track_number','block_clean','time_index'], ascending=[True,True,True,True]).reset_index(drop=True)
  # Close the documents
    document_file.close()


combined_df.drop_duplicates(inplace=True)
combined_df = combined_df.sort_values(by=['bodypump_release_number','track_number','block_clean','time_index'], ascending=[True,True,True,True]).reset_index(drop=True)

***Processing Bodypump release: 105***
Warm up page: 1 Cooldown page 19 
 Pages containing the choregraphy notes:[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 18, 19]
***Processing Bodypump release: 106***
Warm up page: 1 Cooldown page 24 
 Pages containing the choregraphy notes:[1, 3, 5, 6, 8, 9, 11, 12, 14, 16, 18, 19, 21, 22, 24]
***Processing Bodypump release: 107***
Warm up page: 1 Cooldown page 24 
 Pages containing the choregraphy notes:[1, 3, 5, 7, 8, 9, 11, 12, 14, 16, 18, 19, 21, 22, 24]
***Processing Bodypump release: 108***
Warm up page: 14 Cooldown page 44 
 Pages containing the choregraphy notes:[14, 16, 18, 19, 20, 22, 23, 24, 26, 27, 30, 31, 34, 35, 36, 38, 39, 41, 42, 44]
***Processing Bodypump release: 109***
Warm up page: 13 Cooldown page 34 
 Pages containing the choregraphy notes:[13, 15, 17, 19, 21, 22, 24, 26, 27, 29, 31, 32, 34]
***Processing Bodypump release: 111***
Warm up page: 15 Cooldown page 43 
 Pages containing the choregraphy notes:[15, 17, 19

In [345]:
# 10a. removing the bonus tracks that shouldn't be there

# check which tracks to remove: combined_df[combined_df['bodypump_release_number']=='99']['Title'].unique()
tracks_titles_to_remove_dict = {
    '94': ["Get 'Em Up"],
    '98': ['Two Minds'], #removed Two Minds because it was not picking up a lot of rows.
    '106': ['YOU & ME / LOST','ON THE RUN'],
    '107': ['PEOPLE SAY','GET IT RIGHT','CRAWL OUTTA LOVE'],
    '108': ['TRICEPS - CHAMOMILE BREATH','BICEPS - MANERA','BABABA','I WANNA KNOW','GREAT WIDE OPEN'], #removed GREAT WIDE OPEN because of duplication with barbell option
    '109': ['THE MIDDLE','GOLD (STUPID LOVE)'],
    '111': ['THIS FEELING','LOST MY MIND'],
    '116': ['CANT STOP','THEN WE CRY'],
    '117': ['MADE YOU LOOK','YOU’RE THE ONE'],
    '118': ['24/7','LONG WAY HOME','WE HAVE SPOKEN'],
    '119': ['ELEVATE','HOLD ON','MAJESTY'],
    '120': ['JUST FEELS TIGHT','MAKE YOU CRY','INDUSTRY BABY'],
    '123': ['WORK IT OUT','ESCAPE'],
    '124': ['WAY DOWN','MILLION DAYS'],
    '127': ['RATATA','ANGRY BEFORE']
}
from itertools import chain
tracks_titles_to_remove_list = list(chain(*tracks_titles_to_remove_dict.values()))

combined_df = combined_df[~combined_df['Title'].isin(tracks_titles_to_remove_list)]

In [346]:
combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='08')][['Title','block', 'block_clean', 'time_index', 'lyrics',
       'beat_counts', 'move_desc', 'no_rep','reps', 'combo_clean', 'move1','tempo1','reps1','move2','tempo2','reps2']]
# ,'move3','tempo3','reps3','move4','tempo4','reps4'

,Title,block,block_clean,time_index,lyrics,beat_counts,move_desc,no_rep,reps,combo_clean,move1,tempo1,reps1,move2,tempo2,reps2
3169,I Am Machine,1,95-08-1,0:05,Intro,½x8,,,,,SQUAT WIDE Stance,,,,,
3170,I Am Machine,1,95-08-1,0:08,Instr,,2x8 4/4 STANDING UPRIGHT ROW 2x,,2x,,UPRIGHT ROW,4/4,2x,,,
3171,I Am Machine,2,95-08-2,0:21,V1,,8x8 3/1 STANDING UPRIGHT ROW 8x 0:46 C I am ma...,,4x,95-08-COMBINATION,UPRIGHT ROW,3/1,8x,UPRIGHT ROW,1/1,8x
3172,I Am Machine,3,95-08-3,1:10,V2,,7x8 3/1 STANDING UPRIGHT ROW 7x 1:32 1x8 Tran...,,1x,,UPRIGHT ROW,3/1,7x,,,
3173,I Am Machine,3,95-08-3,1:35,C,,8x8 COMBINATION: 2x 1/1 OVERHEAD PRESS 1/1/2 O...,,4x,95-08-COMBINATION,UPRIGHT ROW,3/1,8x,UPRIGHT ROW,1/1,8x
3174,I Am Machine,3,95-08-3,2:01,Br,,4x8 1/1/2 PUSH PRESS 4x,,4x,,PUSH PRESS,1/1/2,4x,,,
3175,I Am Machine,3,95-08-3,2:11,1x8 Transition: Plates – use all 8 cts,1x8,,,1x,,PUSH PRESS,,,,,
3176,I Am Machine,3,95-08-3,2:15,Instr,8x8,4x,,4x,,PUSH PRESS,,,,,
3177,I Am Machine,3,95-08-3,2:41,C,,8x8 COMBINATION: 2x SINGLE SIDE RAISE 1x SIDE ...,,4x,95-08-COMBINATION,UPRIGHT ROW,3/1,8x,UPRIGHT ROW,1/1,8x
3178,I Am Machine,4,95-08-4,3:05,1x8 Transition: Bar – use all 8 cts,1x8,,,1x,,SIDE RAISE,,,,,


In [347]:
# manual changes/corrections

# 94
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='94')&(combined_df['track_number']=='01')&(combined_df['time_index']=='0:27')].index,'reps'] = '8x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='94')&(combined_df['track_number']=='01')&(combined_df['time_index']=='0:27')].index,'reps1'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='94')&(combined_df['track_number']=='01')&(combined_df['time_index']=='0:27')].index,'tempo1'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='94')&(combined_df['track_number']=='01')&(combined_df['time_index']=='0:27')].index,'reps2'] = '2x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='94')&(combined_df['track_number']=='01')&(combined_df['time_index']=='1:26')].index,'reps'] = '12x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='94')&(combined_df['track_number']=='01')&(combined_df['time_index']=='2:25')].index,'reps'] = '12x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='94')&(combined_df['track_number']=='01')&(combined_df['time_index']=='2:25')].index,'reps1'] = '12x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='94')&(combined_df['track_number']=='01')&(combined_df['time_index']=='2:55')].index,'reps1'] = '12x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='94')&(combined_df['track_number']=='01')&(combined_df['time_index']=='TRANSITION')&(combined_df['lyrics']=='3:24')].index,'time_index'] = '3:24'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='94')&(combined_df['track_number']=='01')&(combined_df['time_index']=='3:24')].index,'no_rep'] = 'TRANSITION'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='94')&(combined_df['track_number']=='01')&(combined_df['time_index']=='3:28')].index,'no_rep'] = 'TRANSITION'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='94')&(combined_df['track_number']=='01')&(combined_df['time_index']=='4:42')].index,'tempo1'] = '1/1'

combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='94')&(combined_df['track_number']=='02')&(combined_df['time_index']=='1:03')].index,'move2'] = 'SQUAT'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='94')&(combined_df['track_number']=='02')&(combined_df['time_index']=='1:39')].index,'tempo1'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='94')&(combined_df['track_number']=='02')&(combined_df['time_index']=='1:39')].index,'reps1'] = '4x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='94')&(combined_df['track_number']=='02')&(combined_df['time_index']=='1:39')].index,'move2'] = 'SQUAT'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='94')&(combined_df['track_number']=='02')&(combined_df['time_index']=='1:39')].index,'tempo2'] = '1/1/2'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='94')&(combined_df['track_number']=='02')&(combined_df['time_index']=='1:39')].index,'reps2'] = '2x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='94')&(combined_df['track_number']=='02')&(combined_df['time_index']=='RECOVERY')&(combined_df['lyrics']=='2:08')].index,'time_index'] = '2:08'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='94')&(combined_df['track_number']=='02')&(combined_df['time_index']=='2:08')].index,'no_rep'] = 'RECOVERY'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='94')&(combined_df['track_number']=='02')&(combined_df['time_index']=='2:38')].index,'tempo1'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='94')&(combined_df['track_number']=='02')&(combined_df['time_index']=='2:38')].index,'reps1'] = '4x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='94')&(combined_df['track_number']=='02')&(combined_df['time_index']=='2:38')].index,'move2'] = 'SQUAT'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='94')&(combined_df['track_number']=='02')&(combined_df['time_index']=='2:38')].index,'tempo2'] = '1/1/2'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='94')&(combined_df['track_number']=='02')&(combined_df['time_index']=='2:38')].index,'reps2'] = '2x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='94')&(combined_df['track_number']=='02')&(combined_df['time_index']=='3:13')].index,'tempo1'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='94')&(combined_df['track_number']=='02')&(combined_df['time_index']=='3:13')].index,'reps1'] = '4x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='94')&(combined_df['track_number']=='02')&(combined_df['time_index']=='3:13')].index,'move2'] = 'SQUAT'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='94')&(combined_df['track_number']=='02')&(combined_df['time_index']=='3:13')].index,'tempo2'] = '1/1/2'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='94')&(combined_df['track_number']=='02')&(combined_df['time_index']=='3:13')].index,'reps2'] = '2x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='94')&(combined_df['track_number']=='02')&(combined_df['time_index']=='RECOVERY')&(combined_df['lyrics']=='3:43')].index,'time_index'] = '3:43'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='94')&(combined_df['track_number']=='02')&(combined_df['time_index']=='3:43')].index,'no_rep'] = 'RECOVERY'

combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='94')&(combined_df['track_number']=='03')&(combined_df['time_index']=='0:37')].index,'move1'] = 'CHEST PRESS'

combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='94')&(combined_df['track_number']=='04')&(combined_df['time_index']=='RECOVERY')&(combined_df['lyrics']=='2:04')].index,'time_index'] = '2:04'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='94')&(combined_df['track_number']=='04')&(combined_df['time_index']=='2:04')].index,'no_rep'] = 'RECOVERY'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='94')&(combined_df['track_number']=='04')&(combined_df['time_index']=='3:01')].index,'move1'] = 'POWER PRESS'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='94')&(combined_df['track_number']=='04')&(combined_df['time_index']=='3:01')].index,'reps1'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='94')&(combined_df['track_number']=='04')&(combined_df['time_index']=='3:01')].index,'move2'] = 'TRIPLE ROW'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='94')&(combined_df['track_number']=='04')&(combined_df['time_index']=='3:01')].index,'reps2'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='94')&(combined_df['track_number']=='04')&(combined_df['time_index']=='RECOVERY')&(combined_df['lyrics']=='3:43')].index,'time_index'] = '3:43'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='94')&(combined_df['track_number']=='04')&(combined_df['time_index']=='3:43')].index,'no_rep'] = 'RECOVERY'

combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='94')&(combined_df['track_number']=='05')&(combined_df['time_index']=='0:33')].index,'reps'] = '20x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='94')&(combined_df['track_number']=='05')&(combined_df['time_index']=='0:33')].index,'move1'] = 'TRICEP DIP'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='94')&(combined_df['track_number']=='05')&(combined_df['time_index']=='0:33')].index,'tempo1'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='94')&(combined_df['track_number']=='05')&(combined_df['time_index']=='0:33')].index,'reps1'] = '20x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='94')&(combined_df['track_number']=='05')&(combined_df['time_index']=='2:30')].index,'reps'] = '20x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='94')&(combined_df['track_number']=='05')&(combined_df['time_index']=='2:30')].index,'tempo1'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='94')&(combined_df['track_number']=='05')&(combined_df['time_index']=='2:30')].index,'reps1'] = '20x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='94')&(combined_df['track_number']=='05')&(combined_df['time_index']=='3:04')].index,'reps'] = '15x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='94')&(combined_df['track_number']=='05')&(combined_df['time_index']=='3:04')].index,'tempo1'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='94')&(combined_df['track_number']=='05')&(combined_df['time_index']=='3:04')].index,'reps1'] = '15x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='94')&(combined_df['track_number']=='05')&(combined_df['time_index']=='4:06')].index,'reps'] = '20x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='94')&(combined_df['track_number']=='05')&(combined_df['time_index']=='4:06')].index,'tempo1'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='94')&(combined_df['track_number']=='05')&(combined_df['time_index']=='4:06')].index,'reps1'] = '20x'

combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='94')&(combined_df['track_number']=='07')].index,'Title'] = 'We Make It Bounce'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='94')&(combined_df['track_number']=='07')&(combined_df['time_index']=='1:18')].index,'move1'] = 'PLATE LUNGE'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='94')&(combined_df['track_number']=='07')&(combined_df['time_index']=='TRANSITION')&(combined_df['lyrics']=='2:01')].index,'time_index'] = '2:01'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='94')&(combined_df['track_number']=='07')&(combined_df['time_index']=='2:01')].index,'no_rep'] = 'TRANSITION'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='94')&(combined_df['track_number']=='07')&(combined_df['time_index']=='2:52')].index,'move1'] = 'PLATE LUNGE'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='94')&(combined_df['track_number']=='07')&(combined_df['time_index']=='TRANSITION')&(combined_df['lyrics']=='3:36')].index,'time_index'] = '3:36'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='94')&(combined_df['track_number']=='07')&(combined_df['time_index']=='3:36')].index,'no_rep'] = 'TRANSITION'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='94')&(combined_df['track_number']=='07')&(combined_df['time_index']=='3:53')].index,'reps1'] = '8x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='94')&(combined_df['track_number']=='07')&(combined_df['time_index']=='4:23')].index,'move1'] = 'PLATE SQUAT'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='94')&(combined_df['track_number']=='07')&(combined_df['time_index']=='4:37')].index,'reps1'] = '4x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='94')&(combined_df['track_number']=='07')&(combined_df['time_index']=='5:07')].index,'move1'] = 'PLATE SQUAT'

combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='94')&(combined_df['track_number']=='08')].index,'Title'] = 'This Is How It Goes'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='94')&(combined_df['track_number']=='08')&(combined_df['time_index']=='0:05')].index,'reps'] = '4x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='94')&(combined_df['track_number']=='08')&(combined_df['time_index']=='0:05')].index,'reps1'] = '2x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='94')&(combined_df['track_number']=='08')&(combined_df['time_index']=='0:05')].index,'move2'] = 'PUSHUP'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='94')&(combined_df['track_number']=='08')&(combined_df['time_index']=='0:05')].index,'reps2'] = '4x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='94')&(combined_df['track_number']=='08')&(combined_df['time_index']=='RECOVERY 1:04')].index,'time_index'] = '1:04'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='94')&(combined_df['track_number']=='08')&(combined_df['time_index']=='1:04')].index,'no_rep'] = 'RECOVERY'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='94')&(combined_df['track_number']=='08')&(combined_df['time_index']=='1:19')].index,'move1'] = 'UPRIGHT ROW'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='94')&(combined_df['track_number']=='08')&(combined_df['time_index']=='1:19')].index,'reps1'] = '4x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='94')&(combined_df['track_number']=='08')&(combined_df['time_index']=='1:19')].index,'tempo1'] = '2/2'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='94')&(combined_df['track_number']=='08')&(combined_df['time_index']=='1:33')].index,'move1'] = 'UPRIGHT ROW'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='94')&(combined_df['track_number']=='08')&(combined_df['time_index']=='1:33')].index,'reps1'] = '4x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='94')&(combined_df['track_number']=='08')&(combined_df['time_index']=='1:33')].index,'tempo1'] = '3/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='94')&(combined_df['track_number']=='08')&(combined_df['time_index']=='1:49')].index,'move1'] = 'STANDING OVERHEAD PRESS'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='94')&(combined_df['track_number']=='08')&(combined_df['time_index']=='1:49')].index,'reps1'] = '8x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='94')&(combined_df['track_number']=='08')&(combined_df['time_index']=='1:49')].index,'tempo1'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='94')&(combined_df['track_number']=='08')&(combined_df['time_index']=='2:04')].index,'move1'] = 'STANDING OVERHEAD PRESS'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='94')&(combined_df['track_number']=='08')&(combined_df['time_index']=='2:04')].index,'reps1'] = '4x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='94')&(combined_df['track_number']=='08')&(combined_df['time_index']=='2:04')].index,'tempo1'] = '2/2'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='94')&(combined_df['track_number']=='08')&(combined_df['time_index']=='2:19')].index,'reps'] = '13x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='94')&(combined_df['track_number']=='08')&(combined_df['time_index']=='2:19')].index,'move1'] = 'STANDING PUSH PRESS'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='94')&(combined_df['track_number']=='08')&(combined_df['time_index']=='2:19')].index,'reps1'] = '13x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='94')&(combined_df['track_number']=='08')&(combined_df['time_index']=='2:19')].index,'tempo1'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='94')&(combined_df['track_number']=='08')&(combined_df['time_index']=='COVERY 2:51')].index,'time_index'] = '2:51'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='94')&(combined_df['track_number']=='08')&(combined_df['time_index']=='2:51')].index,'reps'] = '4x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='94')&(combined_df['track_number']=='08')&(combined_df['time_index']=='2:51')].index,'reps1'] = '2x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='94')&(combined_df['track_number']=='08')&(combined_df['time_index']=='2:51')].index,'tempo1'] = '2/2'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='94')&(combined_df['track_number']=='08')&(combined_df['time_index']=='2:51')].index,'move2'] = 'PUSHUP'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='94')&(combined_df['track_number']=='08')&(combined_df['time_index']=='2:51')].index,'tempo2'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='94')&(combined_df['track_number']=='08')&(combined_df['time_index']=='2:51')].index,'reps2'] = '4x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='94')&(combined_df['track_number']=='08')&(combined_df['time_index']=='RECOVERY 3:50')].index,'time_index'] = '3:50'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='94')&(combined_df['track_number']=='08')&(combined_df['time_index']=='3:50')].index,'no_rep'] = 'RECOVERY'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='94')&(combined_df['track_number']=='08')&(combined_df['time_index']=='4:04')].index,'move1'] = 'UPRIGHT ROW'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='94')&(combined_df['track_number']=='08')&(combined_df['time_index']=='4:04')].index,'reps1'] = '8x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='94')&(combined_df['track_number']=='08')&(combined_df['time_index']=='4:04')].index,'tempo1'] = '2/2'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='94')&(combined_df['track_number']=='08')&(combined_df['time_index']=='4:35')].index,'move1'] = 'STANDING OVERHEAD PRESS'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='94')&(combined_df['track_number']=='08')&(combined_df['time_index']=='4:35')].index,'reps1'] = '8x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='94')&(combined_df['track_number']=='08')&(combined_df['time_index']=='4:35')].index,'tempo1'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='94')&(combined_df['track_number']=='08')&(combined_df['time_index']=='4:50')].index,'move1'] = 'STANDING OVERHEAD PRESS'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='94')&(combined_df['track_number']=='08')&(combined_df['time_index']=='4:50')].index,'reps1'] = '4x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='94')&(combined_df['track_number']=='08')&(combined_df['time_index']=='4:50')].index,'tempo1'] = '2/2'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='94')&(combined_df['track_number']=='08')&(combined_df['time_index']=='5:04')].index,'reps'] = '13x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='94')&(combined_df['track_number']=='08')&(combined_df['time_index']=='5:04')].index,'move1'] = 'STANDING PUSH PRESS'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='94')&(combined_df['track_number']=='08')&(combined_df['time_index']=='5:04')].index,'reps1'] = '13x'

combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='94')&(combined_df['track_number']=='09')&(combined_df['time_index']=='TRANSITION')&(combined_df['lyrics']=='3:11')].index,'time_index'] = '3:11'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='94')&(combined_df['track_number']=='09')&(combined_df['time_index']=='3:11')].index,'no_rep'] = 'TRANSITION'

# 95
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='01')&(combined_df['time_index']=='0:42')].index,'reps1'] = '8x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='01')&(combined_df['time_index']=='1:55')].index,'reps1'] = '8x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='01')&(combined_df['time_index']=='TRANSITION')&(combined_df['lyrics']=='2:39')].index,'time_index'] = '2:39'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='01')&(combined_df['time_index']=='2:39')].index,'no_rep'] = 'TRANSITION'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='01')&(combined_df['time_index']=='TRANSITION')&(combined_df['lyrics']=='3:41')].index,'time_index'] = '3:41'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='01')&(combined_df['time_index']=='3:41')].index,'no_rep'] = 'TRANSITION'

combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='02')&(combined_df['time_index']=='2:42')].index,'move2'] = 'BOTTOM HALF SQUAT'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='02')&(combined_df['time_index']=='3:25')].index,'move1'] = 'SQUAT WIDE Stance'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='02')&(combined_df['time_index']=='3:39')].index,'move1'] = 'SQUAT WIDE Stance'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='02')&(combined_df['time_index']=='4:08')].index,'move2'] = 'BOTTOM HALF SQUAT'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='02')&(combined_df['time_index']=='4:51')].index,'move1'] = 'SQUAT WIDE Stance'

combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='03')&(combined_df['time_index']=='0:21')].index,'reps'] = '18x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='03')&(combined_df['time_index']=='0:21')].index,'reps1'] = '18x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='03')&(combined_df['time_index']=='2:22')].index,'move1'] = 'CHEST PRESS BOTTOM HALF PULSE'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='03')&(combined_df['time_index']=='2:47')].index,'no_rep'] = 'TRANSITION'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='03')&(combined_df['time_index']=='2:47')].index,'reps1'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='03')&(combined_df['time_index']=='2:47')].index,'move1'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='03')&(combined_df['time_index']=='2:55')].index,'move1'] = 'A-PRESS'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='03')&(combined_df['time_index']=='3:11')].index,'move1'] = 'A-PRESS'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='03')&(combined_df['time_index']=='3:27')].index,'move1'] = '4x PULSE A-PRESS'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='03')&(combined_df['time_index']=='3:27')].index,'reps1'] = '5x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='03')&(combined_df['time_index']=='2:47')].index,'reps1'] = '8x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='03')&(combined_df['time_index']=='2:47')].index,'tempo1'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='03')&(combined_df['time_index']=='4:26')].index,'no_rep'] = 'TRANSITION'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='03')&(combined_df['time_index']=='4:26')].index,'move1'] = ''
to_append_95_03 = combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='03')&(combined_df['block_clean']=='95-05-3')]
to_append_95_03.loc[:,'block']='4'
to_append_95_03.loc[:,'time_index']='4:34'
to_append_95_03.loc[:,'block_clean']='96-05-2'
combined_df = pd.concat([combined_df, to_append_95_03], ignore_index= True)
combined_df = combined_df.sort_values(by=['bodypump_release_number','track_number','block_clean','time_index'], ascending=[True,True,True,True]).reset_index(drop=True)

combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='04')&(combined_df['time_index']=='0:44')].index,'reps1'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='04')&(combined_df['time_index']=='0:44')].index,'reps2'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='04')&(combined_df['time_index']=='1:13')].index,'reps2'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='04')&(combined_df['time_index']=='2:18')].index,'reps1'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='04')&(combined_df['time_index']=='2:18')].index,'reps2'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='04')&(combined_df['time_index']=='2:32')].index,'reps2'] = '1x'

combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='05')&(combined_df['time_index']=='0:27')].index,'reps'] = '12x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='05')&(combined_df['time_index']=='0:27')].index,'reps1'] = '12x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='05')&(combined_df['time_index']=='TRANSITION')&(combined_df['lyrics']=='0:51')].index,'time_index'] = '0:51'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='05')&(combined_df['time_index']=='0:51')].index,'no_rep'] = 'TRANSITION'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='05')&(combined_df['time_index']=='TRANSITION')&(combined_df['lyrics']=='1:18')].index,'time_index'] = '1:18'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='05')&(combined_df['time_index']=='1:18')].index,'no_rep'] = 'TRANSITION'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='05')&(combined_df['time_index']=='1:18')].index,'move1'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='05')&(combined_df['time_index']=='1:18')].index,'reps1'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='05')&(combined_df['time_index']=='1:21')].index,'reps1'] = '16x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='05')&(combined_df['time_index']=='1:21')].index,'reps'] = '16x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='05')&(combined_df['time_index']=='TRANSITION')&(combined_df['lyrics']=='1:57')].index,'time_index'] = '1:57'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='05')&(combined_df['time_index']=='1:57')].index,'no_rep'] = 'TRANSITION'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='05')&(combined_df['time_index']=='2:00')].index,'reps'] = '16x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='05')&(combined_df['time_index']=='2:00')].index,'reps1'] = '16x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='05')&(combined_df['time_index']=='TRANSITION')&(combined_df['lyrics']=='2.23')].index,'time_index'] = '2:23'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='05')&(combined_df['time_index']=='2:23')].index,'no_rep'] = 'TRANSITION'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='05')&(combined_df['time_index']=='2:53')].index,'reps'] = '16x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='05')&(combined_df['time_index']=='2:53')].index,'reps1'] = '16x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='05')&(combined_df['time_index']=='TRANSITION')&(combined_df['lyrics']=='3:16')].index,'time_index'] = '3:16'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='05')&(combined_df['time_index']=='3:16')].index,'no_rep'] = 'TRANSITION'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='05')&(combined_df['time_index']=='3:47')].index,'reps'] = '16x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='05')&(combined_df['time_index']=='3:47')].index,'reps1'] = '16x'

combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='06')&(combined_df['time_index']=='0:19')].index,'reps1'] = '4x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='06')&(combined_df['time_index']=='1:10')].index,'reps1'] = '4x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='06')&(combined_df['time_index']=='2:01')].index,'reps1'] = '8x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='06')&(combined_df['time_index']=='2:40')].index,'reps1'] = '8x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='06')&(combined_df['time_index']=='3:18')].index,'reps1'] = '8x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='06')&(combined_df['time_index']=='3:57')].index,'reps1'] = '8x'

combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='07')&(combined_df['time_index']=='TRANSITION')&(combined_df['lyrics']=='1:35')].index,'time_index'] = '1:35'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='07')&(combined_df['time_index']=='1:35')].index,'no_rep'] = 'TRANSITION'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='07')&(combined_df['time_index']=='TRANSITION')&(combined_df['lyrics']=='3:39')].index,'time_index'] = '3:39'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='07')&(combined_df['time_index']=='3:39')].index,'no_rep'] = 'TRANSITION'

combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='08')&(combined_df['time_index']=='0:21')].index,'reps'] = '8x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='08')&(combined_df['time_index']=='0:21')].index,'reps1'] = '2x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='08')&(combined_df['time_index']=='0:21')].index,'tempo1'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='08')&(combined_df['time_index']=='0:21')].index,'reps2'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='08')&(combined_df['time_index']=='0:21')].index,'tempo2'] = '1/1/2'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='08')&(combined_df['time_index']=='0:21')].index,'reps3'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='08')&(combined_df['time_index']=='0:21')].index,'move3'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='08')&(combined_df['time_index']=='0:21')].index,'tempo3'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='08')&(combined_df['time_index']=='1:35')].index,'move1'] = 'STANDING OVERHEAD PRESS'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='08')&(combined_df['time_index']=='1:35')].index,'reps1'] = '2x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='08')&(combined_df['time_index']=='1:35')].index,'tempo1'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='08')&(combined_df['time_index']=='1:35')].index,'move2'] = 'STANDING OVERHEAD PRESS'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='08')&(combined_df['time_index']=='1:35')].index,'reps2'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='08')&(combined_df['time_index']=='1:35')].index,'tempo2'] = '1/1/2'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='08')&(combined_df['time_index']=='1:35')].index,'reps3'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='08')&(combined_df['time_index']=='1:35')].index,'move3'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='08')&(combined_df['time_index']=='1:35')].index,'tempo3'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='08')&(combined_df['time_index']=='2:11')].index,'no_rep'] = 'TRANSITION'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='08')&(combined_df['time_index']=='2:11')].index,'move1'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='08')&(combined_df['time_index']=='2:15')].index,'move1'] = 'SIDE RAISE'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='08')&(combined_df['time_index']=='2:15')].index,'tempo1'] = '4/4'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='08')&(combined_df['time_index']=='2:15')].index,'reps1'] = '4x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='08')&(combined_df['time_index']=='2:41')].index,'move1'] = 'SIDE RAISE'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='08')&(combined_df['time_index']=='2:41')].index,'reps1'] = '2x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='08')&(combined_df['time_index']=='2:41')].index,'tempo1'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='08')&(combined_df['time_index']=='2:41')].index,'move2'] = 'ROTATOR RAISE'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='08')&(combined_df['time_index']=='2:41')].index,'reps2'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='08')&(combined_df['time_index']=='2:41')].index,'tempo2'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='08')&(combined_df['time_index']=='2:41')].index,'reps3'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='08')&(combined_df['time_index']=='2:41')].index,'move3'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='08')&(combined_df['time_index']=='2:41')].index,'tempo3'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='08')&(combined_df['time_index']=='3:29')].index,'no_rep'] = 'TRANSITION'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='08')&(combined_df['time_index']=='3:29')].index,'move1'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='08')&(combined_df['time_index']=='3:33')].index,'move1'] = 'STANDING OVERHEAD PRESS'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='08')&(combined_df['time_index']=='3:33')].index,'reps1'] = '2x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='08')&(combined_df['time_index']=='3:33')].index,'tempo1'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='08')&(combined_df['time_index']=='3:33')].index,'move2'] = 'STANDING OVERHEAD PRESS'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='08')&(combined_df['time_index']=='3:33')].index,'reps2'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='08')&(combined_df['time_index']=='3:33')].index,'tempo2'] = '1/1/2'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='08')&(combined_df['time_index']=='3:33')].index,'reps3'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='08')&(combined_df['time_index']=='3:33')].index,'move3'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='08')&(combined_df['time_index']=='3:33')].index,'tempo3'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='08')&(combined_df['time_index']=='4:10')].index,'no_rep'] = 'TRANSITION'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='08')&(combined_df['time_index']=='4:10')].index,'move1'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='08')&(combined_df['time_index']=='4:13')].index,'reps'] = '8x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='08')&(combined_df['time_index']=='4:13')].index,'move1'] = 'SIDE RAISE'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='08')&(combined_df['time_index']=='4:13')].index,'reps1'] = '2x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='08')&(combined_df['time_index']=='4:13')].index,'tempo1'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='08')&(combined_df['time_index']=='4:13')].index,'move2'] = 'ROTATOR RAISE'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='08')&(combined_df['time_index']=='4:13')].index,'reps2'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='08')&(combined_df['time_index']=='4:13')].index,'tempo2'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='08')&(combined_df['time_index']=='4:13')].index,'reps3'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='08')&(combined_df['time_index']=='4:13')].index,'move3'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='08')&(combined_df['time_index']=='4:13')].index,'tempo3'] = ''

combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='09')&(combined_df['time_index']=='TRANSITION')&(combined_df['lyrics']=='2:20')].index,'time_index'] = '2:20'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='95')&(combined_df['track_number']=='09')&(combined_df['time_index']=='2:20')].index,'no_rep'] = 'TRANSITION'

# 96
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='01')&(combined_df['time_index']=='0:33')].index,'reps1'] = '12x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='01')&(combined_df['time_index']=='1:46')].index,'move1'] = 'CLEAN & PRESS'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='01')&(combined_df['time_index']=='1:46')].index,'tempo1'] = '2/2'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='01')&(combined_df['time_index']=='2:15')].index,'move1'] = 'CLEAN & PRESS'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='01')&(combined_df['time_index']=='TRANSITION')&(combined_df['lyrics']=='2:44')].index,'time_index'] = '2:44'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='01')&(combined_df['time_index']=='2:44')].index,'no_rep'] = 'TRANSITION'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='01')&(combined_df['time_index']=='2:44')].index,'move1'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='01')&(combined_df['time_index']=='2:44')].index,'reps1'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='01')&(combined_df['time_index']=='TRANSITION')&(combined_df['lyrics']=='3:28')].index,'time_index'] = '3:28'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='01')&(combined_df['time_index']=='3:28')].index,'no_rep'] = 'TRANSITION'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='01')&(combined_df['time_index']=='TRANSITION')&(combined_df['lyrics']=='4:42')].index,'time_index'] = '4:42'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='01')&(combined_df['time_index']=='4:42')].index,'no_rep'] = 'TRANSITION'

combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='02')&(combined_df['time_index']=='1:08')].index,'move2'] = 'BOTTOM HALF SQUAT'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='02')&(combined_df['time_index']=='1:08')].index,'reps2'] = '4x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='02')&(combined_df['time_index']=='TRANSITION')&(combined_df['lyrics']=='2:07')].index,'time_index'] = '2:07'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='02')&(combined_df['time_index']=='2:07')].index,'no_rep'] = 'TRANSITION'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='02')&(combined_df['time_index']=='3:23')].index,'tempo1'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='02')&(combined_df['time_index']=='3:23')].index,'reps1'] = '4x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='02')&(combined_df['time_index']=='3:23')].index,'move2'] = 'BOTTOM HALF SQUAT'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='02')&(combined_df['time_index']=='3:23')].index,'reps2'] = '4x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='02')&(combined_df['time_index']=='TRANSITION')&(combined_df['lyrics']=='4:07')].index,'time_index'] = '4:07'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='02')&(combined_df['time_index']=='4:07')].index,'no_rep'] = 'TRANSITION'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='02')&(combined_df['time_index']=='5:23')].index,'tempo1'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='02')&(combined_df['time_index']=='5:23')].index,'reps1'] = '4x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='02')&(combined_df['time_index']=='5:23')].index,'move2'] = 'BOTTOM HALF SQUAT'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='02')&(combined_df['time_index']=='5:23')].index,'reps2'] = '4x'

combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='03')&(combined_df['time_index']=='0:44')].index,'move1'] = 'PULSE A-PRESS'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='03')&(combined_df['time_index']=='0:44')].index,'tempo1'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='03')&(combined_df['time_index']=='0:44')].index,'reps1'] = '4x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='03')&(combined_df['time_index']=='0:44')].index,'move2'] = 'A-PRESS'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='03')&(combined_df['time_index']=='0:44')].index,'tempo2'] = '2/2'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='03')&(combined_df['time_index']=='0:44')].index,'reps2'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='03')&(combined_df['time_index']=='1:57')].index,'move1'] = 'PULSE A-PRESS'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='03')&(combined_df['time_index']=='1:57')].index,'tempo1'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='03')&(combined_df['time_index']=='1:57')].index,'reps1'] = '4x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='03')&(combined_df['time_index']=='1:57')].index,'move2'] = 'A-PRESS'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='03')&(combined_df['time_index']=='1:57')].index,'tempo2'] = '2/2'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='03')&(combined_df['time_index']=='1:57')].index,'reps2'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='03')&(combined_df['time_index']=='TRANSITION')&(combined_df['lyrics']=='2:23')].index,'time_index'] = '2:23'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='03')&(combined_df['time_index']=='2:23')].index,'no_rep'] = 'TRANSITION'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='03')&(combined_df['time_index']=='2:36')].index,'move1'] = '4x PULSE PUSHUP'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='03')&(combined_df['time_index']=='2:36')].index,'tempo1'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='03')&(combined_df['time_index']=='2:36')].index,'reps1'] = '4x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='03')&(combined_df['time_index']=='2:36')].index,'move2'] = 'PUSHUP'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='03')&(combined_df['time_index']=='2:36')].index,'tempo2'] = '2/2'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='03')&(combined_df['time_index']=='2:36')].index,'reps2'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='03')&(combined_df['time_index']=='TRANSITION')&(combined_df['lyrics']=='3:02')].index,'time_index'] = '3:02'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='03')&(combined_df['time_index']=='3:02')].index,'no_rep'] = 'TRANSITION'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='03')&(combined_df['time_index']=='3:45')].index,'move1'] = 'PULSE A-PRESS'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='03')&(combined_df['time_index']=='3:45')].index,'tempo1'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='03')&(combined_df['time_index']=='3:45')].index,'reps1'] = '4x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='03')&(combined_df['time_index']=='3:45')].index,'move2'] = 'A-PRESS'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='03')&(combined_df['time_index']=='3:45')].index,'tempo2'] = '2/2'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='03')&(combined_df['time_index']=='3:45')].index,'reps2'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='03')&(combined_df['time_index']=='4:58')].index,'move1'] = 'PULSE A-PRESS'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='03')&(combined_df['time_index']=='4:58')].index,'tempo1'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='03')&(combined_df['time_index']=='4:58')].index,'reps1'] = '4x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='03')&(combined_df['time_index']=='4:58')].index,'move2'] = 'A-PRESS'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='03')&(combined_df['time_index']=='4:58')].index,'tempo2'] = '2/2'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='03')&(combined_df['time_index']=='4:58')].index,'reps2'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='03')&(combined_df['time_index']=='TRANSITION')&(combined_df['lyrics']=='5:24')].index,'time_index'] = '5:24'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='03')&(combined_df['time_index']=='5:24')].index,'no_rep'] = 'TRANSITION'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='03')&(combined_df['time_index']=='5:37')].index,'move1'] = '4x PULSE PUSHUP'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='03')&(combined_df['time_index']=='5:37')].index,'tempo1'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='03')&(combined_df['time_index']=='5:37')].index,'reps1'] = '4x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='03')&(combined_df['time_index']=='5:37')].index,'move2'] = 'PUSHUP'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='03')&(combined_df['time_index']=='5:37')].index,'tempo2'] = '2/2'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='03')&(combined_df['time_index']=='5:37')].index,'reps2'] = '1x'

combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='04')&(combined_df['time_index']=='0:58')].index,'tempo1'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='04')&(combined_df['time_index']=='0:58')].index,'tempo2'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='04')&(combined_df['time_index']=='0:58')].index,'reps2'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='04')&(combined_df['time_index']=='2:01')].index,'move1'] = 'CLEAN & PRESS'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='04')&(combined_df['time_index']=='2:01')].index,'tempo1'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='04')&(combined_df['time_index']=='2:01')].index,'reps1'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='04')&(combined_df['time_index']=='2:01')].index,'move2'] = 'DEADROW'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='04')&(combined_df['time_index']=='2:01')].index,'tempo2'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='04')&(combined_df['time_index']=='2:01')].index,'reps2'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='04')&(combined_df['time_index']=='TRANSITION')&(combined_df['lyrics']=='2:52')].index,'time_index'] = '2:52'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='04')&(combined_df['time_index']=='2:52')].index,'no_rep'] = 'TRANSITION'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='04')&(combined_df['time_index']=='TRANSITION')&(combined_df['lyrics']=='3:32')].index,'time_index'] = '3:32'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='04')&(combined_df['time_index']=='3:32')].index,'no_rep'] = 'TRANSITION'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='04')&(combined_df['time_index']=='3:56')].index,'move1'] = 'CLEAN & PRESS'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='04')&(combined_df['time_index']=='3:56')].index,'tempo1'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='04')&(combined_df['time_index']=='3:56')].index,'reps1'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='04')&(combined_df['time_index']=='3:56')].index,'move2'] = 'DEADROW'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='04')&(combined_df['time_index']=='3:56')].index,'tempo2'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='04')&(combined_df['time_index']=='3:56')].index,'reps2'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='04')&(combined_df['time_index']=='TRANSITION')&(combined_df['lyrics']=='4:35')].index,'time_index'] = '4:35'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='04')&(combined_df['time_index']=='4:35')].index,'no_rep'] = 'TRANSITION'

combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='05')&(combined_df['time_index']=='0:07')].index,'tempo1'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='05')&(combined_df['time_index']=='0:07')].index,'reps1'] = '2x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='05')&(combined_df['time_index']=='0:07')].index,'move2'] = 'SEATED TRICEP OVERHEAD EXTENSION'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='05')&(combined_df['time_index']=='0:07')].index,'reps2'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='05')&(combined_df['time_index']=='0:07')].index,'tempo2'] = '1/1/2'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='05')&(combined_df['time_index']=='1:11')].index,'tempo1'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='05')&(combined_df['time_index']=='1:11')].index,'reps1'] = '2x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='05')&(combined_df['time_index']=='1:11')].index,'move2'] = 'SEATED TRICEP OVERHEAD EXTENSION'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='05')&(combined_df['time_index']=='1:11')].index,'reps2'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='05')&(combined_df['time_index']=='1:11')].index,'tempo2'] = '1/1/2'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='05')&(combined_df['time_index']=='2:16')].index,'move1'] = 'TRICEP PUSHUP'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='05')&(combined_df['time_index']=='2:16')].index,'tempo1'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='05')&(combined_df['time_index']=='2:16')].index,'reps1'] = '2x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='05')&(combined_df['time_index']=='2:16')].index,'move2'] = 'TRICEP PUSHUP'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='05')&(combined_df['time_index']=='2:16')].index,'reps2'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='05')&(combined_df['time_index']=='2:16')].index,'tempo2'] = '1/1/2'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='05')&(combined_df['time_index']=='TRANSITION')&(combined_df['lyrics']=='3:24')].index,'time_index'] = '3:24'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='05')&(combined_df['time_index']=='3:26')].index,'move1'] = 'TRICEP PUSHUP'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='05')&(combined_df['time_index']=='3:26')].index,'tempo1'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='05')&(combined_df['time_index']=='3:26')].index,'reps1'] = '2x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='05')&(combined_df['time_index']=='3:26')].index,'move2'] = 'TRICEP PUSHUP'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='05')&(combined_df['time_index']=='3:26')].index,'reps2'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='05')&(combined_df['time_index']=='3:26')].index,'tempo2'] = '1/1/2'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='05')&(combined_df['time_index']=='3:26')].index,'move1'] = 'STANDING TRICEP OVERHEAD EXTENSION'

combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='06')&(combined_df['time_index']=='0:04')].index,'no_rep'] = 'SET Position'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='06')&(combined_df['time_index']=='0:31')].index,'reps1'] = '4x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='06')&(combined_df['time_index']=='1:15')].index,'reps2'] = '3x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='06')&(combined_df['time_index']=='1:37')].index,'reps1'] = '4x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='06')&(combined_df['time_index']=='2:22')].index,'tempo1'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='06')&(combined_df['time_index']=='2:22')].index,'reps1'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='06')&(combined_df['time_index']=='2:22')].index,'move2'] = 'MID-RANGE PULSE'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='06')&(combined_df['time_index']=='2:22')].index,'reps1'] = '3x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='06')&(combined_df['time_index']=='RECOVERY')&(combined_df['lyrics']=='3:07')].index,'time_index'] = '3:07'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='06')&(combined_df['time_index']=='3:07')].index,'no_rep'] = 'RECOVERY'

combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='07')&(combined_df['time_index']=='0:37')].index,'move1'] = '3x PULSE LUNGE'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='07')&(combined_df['time_index']=='0:37')].index,'reps1'] = '4x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='07')&(combined_df['time_index']=='0:52')].index,'move1'] = 'LUNGE'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='07')&(combined_df['time_index']=='0:37')].index,'move1'] = '7x PULSE LUNGE'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='07')&(combined_df['time_index']=='0:37')].index,'reps1'] = '4x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='07')&(combined_df['time_index']=='TRANSITION')&(combined_df['lyrics']=='2:05')].index,'time_index'] = '2:05'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='07')&(combined_df['time_index']=='2:05')].index,'no_rep'] = 'TRANSITION'
to_append_96_07 = combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='07')&(combined_df['time_index']=='2:10')]
to_append_96_07.loc[:,'block']='2'
to_append_96_07.loc[:,'time_index']='2:10'
to_append_96_07.loc[:,'block_clean']='96-0-2'
combined_df = pd.concat([combined_df, to_append_96_07], ignore_index= True)
combined_df = combined_df.sort_values(by=['bodypump_release_number','track_number','block_clean','time_index'], ascending=[True,True,True,True]).reset_index(drop=True)

combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='08')&(combined_df['time_index']=='0:05')].index,'no_rep'] = 'SET Position'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='08')&(combined_df['time_index']=='0:50')].index,'tempo1'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='08')&(combined_df['time_index']=='0:50')].index,'reps1'] = '4x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='08')&(combined_df['time_index']=='0:50')].index,'move2'] = 'ROTATOR RAISE'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='08')&(combined_df['time_index']=='0:50')].index,'tempo2'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='08')&(combined_df['time_index']=='0:50')].index,'reps2'] = '2x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='08')&(combined_df['time_index']=='TRANSITION 2:32')].index,'time_index'] = '2:32'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='08')&(combined_df['time_index']=='2:32')].index,'no_rep'] = 'TRANSITION'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='08')&(combined_df['time_index']=='TRANSITION 3:34')].index,'time_index'] = '3:34'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='08')&(combined_df['time_index']=='3:34')].index,'no_rep'] = 'TRANSITION'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='08')&(combined_df['time_index']=='3:48')].index,'tempo1'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='08')&(combined_df['time_index']=='3:48')].index,'reps1'] = '4x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='08')&(combined_df['time_index']=='3:48')].index,'move2'] = 'ROTATOR RAISE'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='08')&(combined_df['time_index']=='3:48')].index,'tempo2'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='08')&(combined_df['time_index']=='3:48')].index,'reps2'] = '2x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='08')&(combined_df['time_index']=='TRANSITION 5:01')].index,'time_index'] = '5:01'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='08')&(combined_df['time_index']=='5:01')].index,'no_rep'] = 'TRANSITION'

combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='09')&(combined_df['time_index']=='RECOVERY')&(combined_df['lyrics']=='2:21')].index,'time_index'] = '2:21'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='96')&(combined_df['track_number']=='09')&(combined_df['time_index']=='2:21')].index,'no_rep'] = 'RECOVERY'

# 98
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='98')&(combined_df['track_number']=='02')&(combined_df['time_index']=='2:45')].index,'combo_clean'] = '98-02-COMBINATION'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='98')&(combined_df['track_number']=='02')&(combined_df['time_index']=='2:45')].index,'move1'] = 'SQUAT'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='98')&(combined_df['track_number']=='02')&(combined_df['time_index']=='2:45')].index,'tempo1'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='98')&(combined_df['track_number']=='02')&(combined_df['time_index']=='2:45')].index,'reps1'] = '12x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='98')&(combined_df['track_number']=='02')&(combined_df['time_index']=='2:45')].index,'move1'] = 'BOTTOM HALF SQUAT'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='98')&(combined_df['track_number']=='02')&(combined_df['time_index']=='2:45')].index,'reps1'] = '4x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='98')&(combined_df['track_number']=='02')&(combined_df['time_index']=='4:27')].index,'combo_clean'] = '98-02-COMBINATION'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='98')&(combined_df['track_number']=='02')&(combined_df['time_index']=='4:27')].index,'move1'] = 'SQUAT'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='98')&(combined_df['track_number']=='02')&(combined_df['time_index']=='4:27')].index,'tempo1'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='98')&(combined_df['track_number']=='02')&(combined_df['time_index']=='4:27')].index,'reps1'] = '12x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='98')&(combined_df['track_number']=='02')&(combined_df['time_index']=='4:27')].index,'move2'] = 'BOTTOM HALF SQUAT'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='98')&(combined_df['track_number']=='02')&(combined_df['time_index']=='4:27')].index,'reps2'] = '4x'

combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='98')&(combined_df['track_number']=='03')&(combined_df['time_index']=='0:52')].index,'reps1'] = '8x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='98')&(combined_df['track_number']=='03')&(combined_df['time_index']=='1:48')].index,'reps1'] = '8x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='98')&(combined_df['track_number']=='03')&(combined_df['time_index']=='2:09')].index,'move1'] = 'CHEST PRESS'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='98')&(combined_df['track_number']=='03')&(combined_df['time_index']=='2:27')].index,'move1'] = 'CHEST PRESS'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='98')&(combined_df['track_number']=='03')&(combined_df['time_index']=='2:48')].index,'reps1'] = '16x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='98')&(combined_df['track_number']=='03')&(combined_df['time_index']=='4:05')].index,'reps1'] = '8x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='98')&(combined_df['track_number']=='03')&(combined_df['time_index']=='4:26')].index,'move1'] = 'CHEST PRESS'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='98')&(combined_df['track_number']=='03')&(combined_df['time_index']=='4:45')].index,'move1'] = 'CHEST PRESS'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='98')&(combined_df['track_number']=='03')&(combined_df['time_index']=='5:06')].index,'reps1'] = '16x'

combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='98')&(combined_df['track_number']=='04')&(combined_df['time_index']=='1:31')].index,'reps1'] = '8x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='98')&(combined_df['track_number']=='04')&(combined_df['time_index']=='3:13')].index,'reps1'] = '8x'

combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='98')&(combined_df['track_number']=='06')&(combined_df['time_index']=='0:16')].index,'reps1'] = '2x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='98')&(combined_df['track_number']=='06')&(combined_df['time_index']=='1:15')].index,'reps1'] = '8x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='98')&(combined_df['track_number']=='06')&(combined_df['time_index']=='2:40')].index,'reps1'] = '8x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='98')&(combined_df['track_number']=='06')&(combined_df['time_index']=='4:16')].index,'reps1'] = '8x'

combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='98')&(combined_df['track_number']=='07')&(combined_df['time_index']=='3:15')].index,'move1'] = 'LUNGE'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='98')&(combined_df['track_number']=='07')&(combined_df['time_index']=='5:11')].index,'move1'] = 'LUNGE'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='98')&(combined_df['track_number']=='07')&(combined_df['time_index']=='4:27')].index,'combo_clean'] = '98-07-COMBINATION 4'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='98')&(combined_df['track_number']=='07')&(combined_df['time_index']=='4:27')].index,'move1'] = 'BOTTOM HALF PULSE'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='98')&(combined_df['track_number']=='07')&(combined_df['time_index']=='4:27')].index,'reps1'] = '4x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='98')&(combined_df['track_number']=='07')&(combined_df['time_index']=='4:27')].index,'move2'] = 'LUNGE'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='98')&(combined_df['track_number']=='07')&(combined_df['time_index']=='4:27')].index,'reps2'] = '2x'

combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='98')&(combined_df['track_number']=='08')&(combined_df['time_index']=='0:18')].index,'move1'] = 'STANDING SIDE RAISE'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='98')&(combined_df['track_number']=='08')&(combined_df['time_index']=='0:18')].index,'tempo1'] = '2/2'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='98')&(combined_df['track_number']=='08')&(combined_df['time_index']=='0:18')].index,'reps1'] = '4x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='98')&(combined_df['track_number']=='08')&(combined_df['time_index']=='0:31')].index,'reps1'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='98')&(combined_df['track_number']=='08')&(combined_df['time_index']=='1:44')].index,'tempo1'] = '2/2'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='98')&(combined_df['track_number']=='08')&(combined_df['time_index']=='1:44')].index,'reps1'] = '4x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='98')&(combined_df['track_number']=='08')&(combined_df['time_index']=='1:53')].index,'tempo1'] = '1/3'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='98')&(combined_df['track_number']=='08')&(combined_df['time_index']=='1:53')].index,'reps1'] = '5x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='98')&(combined_df['track_number']=='08')&(combined_df['time_index']=='2:13')].index,'combo_clean'] = '98-08-COMBINATION 1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='98')&(combined_df['track_number']=='08')&(combined_df['time_index']=='2:13')].index,'move1'] = 'STANDING SIDE RAISE'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='98')&(combined_df['track_number']=='08')&(combined_df['time_index']=='2:13')].index,'reps1'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='98')&(combined_df['track_number']=='08')&(combined_df['time_index']=='2:13')].index,'tempo1'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='98')&(combined_df['track_number']=='08')&(combined_df['time_index']=='2:13')].index,'move2'] = 'STANDING MAC RAISE'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='98')&(combined_df['track_number']=='08')&(combined_df['time_index']=='2:13')].index,'reps2'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='98')&(combined_df['track_number']=='08')&(combined_df['time_index']=='2:13')].index,'tempo2'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='98')&(combined_df['track_number']=='08')&(combined_df['time_index']=='2:13')].index,'move3'] = 'STANDING SIDE RAISE'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='98')&(combined_df['track_number']=='08')&(combined_df['time_index']=='2:13')].index,'reps3'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='98')&(combined_df['track_number']=='08')&(combined_df['time_index']=='2:13')].index,'tempo3'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='98')&(combined_df['track_number']=='08')&(combined_df['time_index']=='2:13')].index,'move4'] = 'STANDING MAC RAISE'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='98')&(combined_df['track_number']=='08')&(combined_df['time_index']=='2:13')].index,'reps4'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='98')&(combined_df['track_number']=='08')&(combined_df['time_index']=='2:13')].index,'tempo4'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='98')&(combined_df['track_number']=='08')&(combined_df['time_index']=='TRANSITION 3:05')].index,'time_index'] = '3:05'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='98')&(combined_df['track_number']=='08')&(combined_df['time_index']=='3:11')].index,'move1'] = 'STANDING UPRIGHT ROW'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='98')&(combined_df['track_number']=='08')&(combined_df['time_index']=='3:11')].index,'tempo1'] = '2/2'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='98')&(combined_df['track_number']=='08')&(combined_df['time_index']=='3:11')].index,'reps1'] = '9x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='98')&(combined_df['track_number']=='08')&(combined_df['time_index']=='3:24')].index,'reps1'] = '10x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='98')&(combined_df['track_number']=='08')&(combined_df['time_index']=='3:24')].index,'reps2'] = '7x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='98')&(combined_df['track_number']=='08')&(combined_df['time_index']=='3:24')].index,'move3'] = 'PUSHUP'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='98')&(combined_df['track_number']=='08')&(combined_df['time_index']=='3:24')].index,'reps3'] = '15x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='98')&(combined_df['track_number']=='08')&(combined_df['time_index']=='3:24')].index,'move4'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='98')&(combined_df['track_number']=='08')&(combined_df['time_index']=='3:24')].index,'reps4'] = ''

combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='98')&(combined_df['track_number']=='09')&(combined_df['time_index']=='0:45')].index,'reps1'] = '8x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='98')&(combined_df['track_number']=='09')&(combined_df['time_index']=='1:44')].index,'reps1'] = '8x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='98')&(combined_df['track_number']=='09')&(combined_df['time_index']=='2:33')].index,'reps1'] = '1x'

combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='99')&(combined_df['track_number']=='08')&(combined_df['time_index']=='1:10')].index,'move1'] = 'KNEELING REAR DELTOID RAISE'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='99')&(combined_df['track_number']=='08')&(combined_df['time_index']=='2:21')].index,'move1'] = 'KNEELING UPRIGHT ROW'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='99')&(combined_df['track_number']=='08')&(combined_df['time_index']=='3:59')].index,'move1'] = 'STANDING UPRIGHT ROW'

# 99
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='99')&(combined_df['track_number']=='01')&(combined_df['time_index']=='2:09')].index,'move1'] = 'UPRIGHT ROW'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='99')&(combined_df['track_number']=='01')&(combined_df['time_index']=='3:36')].index,'no_rep'] = 'SHOULDER ROLL'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='99')&(combined_df['track_number']=='01')&(combined_df['time_index']=='3:58')].index,'move1'] = 'UPRIGHT ROW'

combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='99')&(combined_df['track_number']=='02')&(combined_df['time_index']=='3:14')].index,'combo_clean'] = '99-02-COMBINATION'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='99')&(combined_df['track_number']=='02')&(combined_df['time_index']=='3:14')].index,'move1'] = 'SQUAT'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='99')&(combined_df['track_number']=='02')&(combined_df['time_index']=='3:14')].index,'tempo1'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='99')&(combined_df['track_number']=='02')&(combined_df['time_index']=='3:14')].index,'reps1'] = '4x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='99')&(combined_df['track_number']=='02')&(combined_df['time_index']=='3:14')].index,'move2'] = 'BOTTOM HALF SQUAT'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='99')&(combined_df['track_number']=='02')&(combined_df['time_index']=='3:14')].index,'reps1'] = '4x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='99')&(combined_df['track_number']=='02')&(combined_df['time_index']=='4:56')].index,'combo_clean'] = '99-02-COMBINATION'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='99')&(combined_df['track_number']=='02')&(combined_df['time_index']=='4:56')].index,'move1'] = 'SQUAT'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='99')&(combined_df['track_number']=='02')&(combined_df['time_index']=='4:56')].index,'tempo1'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='99')&(combined_df['track_number']=='02')&(combined_df['time_index']=='4:56')].index,'reps1'] = '4x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='99')&(combined_df['track_number']=='02')&(combined_df['time_index']=='4:56')].index,'move2'] = 'BOTTOM HALF SQUAT'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='99')&(combined_df['track_number']=='02')&(combined_df['time_index']=='4:56')].index,'reps1'] = '4x'

combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='99')&(combined_df['track_number']=='04')&(combined_df['time_index']=='0:05')].index,'no_rep'] = 'SET Position'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='99')&(combined_df['track_number']=='04')&(combined_df['time_index']=='0:10')].index,'move1'] = 'DEADROW'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='99')&(combined_df['track_number']=='04')&(combined_df['time_index']=='0:10')].index,'tempo1'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='99')&(combined_df['track_number']=='04')&(combined_df['time_index']=='0:10')].index,'reps1'] = '8x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='99')&(combined_df['track_number']=='04')&(combined_df['time_index']=='RECOVERY')].index,'no_rep'] = 'RECOVERY'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='99')&(combined_df['track_number']=='04')&(combined_df['time_index']=='0:57')].index,'move1'] = 'DEADLIFT'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='99')&(combined_df['track_number']=='04')&(combined_df['time_index']=='0:57')].index,'tempo1'] = '3/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='99')&(combined_df['track_number']=='04')&(combined_df['time_index']=='0:57')].index,'reps1'] = '8x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='99')&(combined_df['track_number']=='04')&(combined_df['time_index']=='1:57')].index,'move1'] = 'DEADLIFT'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='99')&(combined_df['track_number']=='04')&(combined_df['time_index']=='1:57')].index,'tempo1'] = '2/2'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='99')&(combined_df['track_number']=='04')&(combined_df['time_index']=='1:57')].index,'reps1'] = '4x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='99')&(combined_df['track_number']=='04')&(combined_df['time_index']=='2:12')].index,'move1'] = 'CLEAN & PRESS'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='99')&(combined_df['track_number']=='04')&(combined_df['time_index']=='2:12')].index,'tempo1'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='99')&(combined_df['track_number']=='04')&(combined_df['time_index']=='2:12')].index,'reps1'] = '4x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='99')&(combined_df['track_number']=='04')&(combined_df['time_index']=='2:54')].index,'reps1'] = '8x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='99')&(combined_df['track_number']=='04')&(combined_df['time_index']=='3:41')].index,'move1'] = 'DEADLIFT'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='99')&(combined_df['track_number']=='04')&(combined_df['time_index']=='3:41')].index,'tempo1'] = '3/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='99')&(combined_df['track_number']=='04')&(combined_df['time_index']=='3:41')].index,'reps1'] = '8x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='99')&(combined_df['track_number']=='04')&(combined_df['time_index']=='4:42')].index,'move1'] = 'DEADLIFT'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='99')&(combined_df['track_number']=='04')&(combined_df['time_index']=='4:42')].index,'tempo1'] = '2/2'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='99')&(combined_df['track_number']=='04')&(combined_df['time_index']=='4:42')].index,'reps1'] = '4x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='99')&(combined_df['track_number']=='04')&(combined_df['time_index']=='4:57')].index,'move1'] = 'CLEAN & PRESS'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='99')&(combined_df['track_number']=='04')&(combined_df['time_index']=='4:57')].index,'tempo1'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='99')&(combined_df['track_number']=='04')&(combined_df['time_index']=='4:57')].index,'reps1'] = '4x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='99')&(combined_df['track_number']=='04')&(combined_df['time_index']=='5:39')].index,'move1'] = 'CLEAN & PRESS'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='99')&(combined_df['track_number']=='04')&(combined_df['time_index']=='5:39')].index,'reps1'] = '8x'

combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='99')&(combined_df['track_number']=='05')&(combined_df['time_index']=='RECOVERY')].index,'no_rep'] = 'RECOVERY'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='99')&(combined_df['track_number']=='05')&(combined_df['time_index']=='1:00')].index,'reps1'] = '2x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='99')&(combined_df['track_number']=='05')&(combined_df['time_index']=='2:13')].index,'reps1'] = '2x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='99')&(combined_df['track_number']=='05')&(combined_df['time_index']=='3:04')].index,'tempo2'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='99')&(combined_df['track_number']=='05')&(combined_df['time_index']=='3:04')].index,'reps2'] = '4x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='99')&(combined_df['track_number']=='05')&(combined_df['time_index']=='4:12')].index,'move1'] = 'STANDING OVERHEAD TRICEP EXTENSION BOTTOM HALF PULSE'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='99')&(combined_df['track_number']=='05')&(combined_df['time_index']=='4:12')].index,'move2'] = 'STANDING OVERHEAD TRICEP EXTENSION'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='99')&(combined_df['track_number']=='05')&(combined_df['time_index']=='4:12')].index,'tempo2'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='99')&(combined_df['track_number']=='05')&(combined_df['time_index']=='4:12')].index,'reps2'] = '4x'

combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='99')&(combined_df['track_number']=='06')&(combined_df['time_index']=='0:39')].index,'move2'] = 'BICEP CURL MID-RANGE PULSE'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='99')&(combined_df['track_number']=='06')&(combined_df['time_index']=='1:27')].index,'move1'] = 'BICEP CURL'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='99')&(combined_df['track_number']=='06')&(combined_df['time_index']=='1:27')].index,'tempo1'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='99')&(combined_df['track_number']=='06')&(combined_df['time_index']=='1:27')].index,'reps1'] = '2x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='99')&(combined_df['track_number']=='06')&(combined_df['time_index']=='1:27')].index,'move2'] = 'BICEP CURL MID-RANGE PULSE'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='99')&(combined_df['track_number']=='06')&(combined_df['time_index']=='1:27')].index,'reps2'] = '4x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='99')&(combined_df['track_number']=='06')&(combined_df['time_index']=='1:58')].index,'move1'] = 'BICEP CURL'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='99')&(combined_df['track_number']=='06')&(combined_df['time_index']=='1:58')].index,'tempo1'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='99')&(combined_df['track_number']=='06')&(combined_df['time_index']=='1:58')].index,'reps1'] = '2x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='99')&(combined_df['track_number']=='06')&(combined_df['time_index']=='1:58')].index,'move2'] = 'BICEP CURL MID-RANGE PULSE'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='99')&(combined_df['track_number']=='06')&(combined_df['time_index']=='1:58')].index,'reps2'] = '4x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='99')&(combined_df['track_number']=='06')&(combined_df['time_index']=='3:04')].index,'move1'] = 'BICEP CURL'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='99')&(combined_df['track_number']=='06')&(combined_df['time_index']=='3:04')].index,'tempo1'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='99')&(combined_df['track_number']=='06')&(combined_df['time_index']=='3:04')].index,'reps1'] = '2x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='99')&(combined_df['track_number']=='06')&(combined_df['time_index']=='3:04')].index,'move2'] = 'BICEP CURL MID-RANGE PULSE'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='99')&(combined_df['track_number']=='06')&(combined_df['time_index']=='3:04')].index,'reps2'] = '4x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='99')&(combined_df['track_number']=='06')&(combined_df['time_index']=='3:35')].index,'move1'] = 'BICEP CURL'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='99')&(combined_df['track_number']=='06')&(combined_df['time_index']=='3:35')].index,'tempo1'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='99')&(combined_df['track_number']=='06')&(combined_df['time_index']=='3:35')].index,'reps1'] = '2x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='99')&(combined_df['track_number']=='06')&(combined_df['time_index']=='3:35')].index,'move2'] = 'BICEP CURL MID-RANGE PULSE'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='99')&(combined_df['track_number']=='06')&(combined_df['time_index']=='3:35')].index,'reps2'] = '4x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='99')&(combined_df['track_number']=='06')&(combined_df['time_index']=='4:05')].index,'move1'] = 'BICEP CURL'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='99')&(combined_df['track_number']=='06')&(combined_df['time_index']=='4:05')].index,'tempo1'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='99')&(combined_df['track_number']=='06')&(combined_df['time_index']=='4:05')].index,'reps1'] = '4x'

combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='99')&(combined_df['track_number']=='07')&(combined_df['time_index']=='1:02')].index,'tempo2'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='99')&(combined_df['track_number']=='07')&(combined_df['time_index']=='2:29')].index,'tempo2'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='99')&(combined_df['track_number']=='07')&(combined_df['time_index']=='4:26')].index,'tempo2'] = '1/1'

combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='99')&(combined_df['track_number']=='08')&(combined_df['time_index']=='RECOVERY 1:24')].index,'time_index'] = '1:24'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='99')&(combined_df['track_number']=='08')&(combined_df['time_index']=='RECOVERY 3:17')].index,'time_index'] = '3:17'

# 105
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='105')&(combined_df['track_number']=='01')].index,'Title'] = 'ALL STARS'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='105')&(combined_df['track_number']=='01')].index,'Duration'] = '5:21'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='105')&(combined_df['track_number']=='01')&(combined_df['time_index']=='5:05')].index,'no_rep'] = 'HOLD'

combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='105')&(combined_df['track_number']=='02')].index,'Title'] = 'DO OR DIE'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='105')&(combined_df['track_number']=='02')].index,'Duration'] = '5:40'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='105')&(combined_df['track_number']=='02')&(combined_df['time_index']=='1:16')].index,'tempo1'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='105')&(combined_df['track_number']=='02')&(combined_df['time_index']=='1:16')].index,'reps1'] = '2x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='105')&(combined_df['track_number']=='02')&(combined_df['time_index']=='1:16')].index,'move2'] = '2x BOTTOM HALF SQUAT'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='105')&(combined_df['track_number']=='02')&(combined_df['time_index']=='1:16')].index,'reps2'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='105')&(combined_df['track_number']=='02')&(combined_df['time_index']=='3:05')].index,'tempo1'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='105')&(combined_df['track_number']=='02')&(combined_df['time_index']=='3:05')].index,'reps1'] = '2x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='105')&(combined_df['track_number']=='02')&(combined_df['time_index']=='3:05')].index,'move2'] = '2x BOTTOM HALF SQUAT'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='105')&(combined_df['track_number']=='02')&(combined_df['time_index']=='3:05')].index,'reps2'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='105')&(combined_df['track_number']=='02')&(combined_df['time_index']=='4:53')].index,'tempo1'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='105')&(combined_df['track_number']=='02')&(combined_df['time_index']=='4:53')].index,'reps1'] = '2x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='105')&(combined_df['track_number']=='02')&(combined_df['time_index']=='4:53')].index,'move2'] = '2x BOTTOM HALF SQUAT'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='105')&(combined_df['track_number']=='02')&(combined_df['time_index']=='4:53')].index,'reps2'] = '1x'

combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='105')&(combined_df['track_number']=='03')].index,'Title'] = 'THE MAN'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='105')&(combined_df['track_number']=='03')].index,'Duration'] = '5:25'

combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='105')&(combined_df['track_number']=='04')].index,'Title'] = 'STAY FOR IT'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='105')&(combined_df['track_number']=='04')].index,'Duration'] = '6:12'

combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='105')&(combined_df['track_number']=='05')].index,'Title'] = 'WHAT ABOUT US'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='105')&(combined_df['track_number']=='05')].index,'Duration'] = '5:34'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='105')&(combined_df['track_number']=='05')&(combined_df['time_index']=='3:37')].index,'move1'] = 'PULSE TRICEP DIP'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='105')&(combined_df['track_number']=='05')&(combined_df['time_index']=='3:37')].index,'move2'] = 'TRICEP DIP'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='105')&(combined_df['track_number']=='05')&(combined_df['time_index']=='4:42')].index,'move1'] = 'PULSE TRICEP EXTENSION'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='105')&(combined_df['track_number']=='05')&(combined_df['time_index']=='4:42')].index,'move2'] = 'STANDING O/H EXTENSION'

combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='105')&(combined_df['track_number']=='06')].index,'Title'] = 'CHAMPION'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='105')&(combined_df['track_number']=='06')].index,'Duration'] = '4:55'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='105')&(combined_df['track_number']=='06')&(combined_df['time_index']=='0:55')].index,'move3'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='105')&(combined_df['track_number']=='06')&(combined_df['time_index']=='0:55')].index,'move4'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='105')&(combined_df['track_number']=='06')&(combined_df['time_index']=='1:45')].index,'move3'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='105')&(combined_df['track_number']=='06')&(combined_df['time_index']=='1:45')].index,'move4'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='105')&(combined_df['track_number']=='06')&(combined_df['time_index']=='2:24')].index,'move3'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='105')&(combined_df['track_number']=='06')&(combined_df['time_index']=='2:24')].index,'move4'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='105')&(combined_df['track_number']=='06')&(combined_df['time_index']=='3:09')].index,'move1'] = 'PLATE CURL'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='105')&(combined_df['track_number']=='06')&(combined_df['time_index']=='3:29')].index,'move3'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='105')&(combined_df['track_number']=='06')&(combined_df['time_index']=='3:29')].index,'move4'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='105')&(combined_df['track_number']=='06')&(combined_df['time_index']=='4:09')].index,'move3'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='105')&(combined_df['track_number']=='06')&(combined_df['time_index']=='4:09')].index,'move4'] = ''

combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='105')&(combined_df['track_number']=='07')].index,'Title'] = 'HOW DO YOU FEEL RIGHT NOW'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='105')&(combined_df['track_number']=='07')].index,'Duration'] = '5:47'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='105')&(combined_df['track_number']=='07')&(combined_df['time_index']=='1:39')].index,'move1'] = 'PULSE SQUAT'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='105')&(combined_df['track_number']=='07')&(combined_df['time_index']=='1:39')].index,'reps1'] = '2x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='105')&(combined_df['track_number']=='07')&(combined_df['time_index']=='1:39')].index,'move2'] = 'PULSE LUNGE'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='105')&(combined_df['track_number']=='07')&(combined_df['time_index']=='1:39')].index,'reps2'] = '2x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='105')&(combined_df['track_number']=='07')&(combined_df['time_index']=='2:08')].index,'move1'] = 'PULSE SQUAT'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='105')&(combined_df['track_number']=='07')&(combined_df['time_index']=='2:08')].index,'reps1'] = '4x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='105')&(combined_df['track_number']=='07')&(combined_df['time_index']=='2:08')].index,'move2'] = 'PULSE LUNGE'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='105')&(combined_df['track_number']=='07')&(combined_df['time_index']=='2:08')].index,'reps2'] = '4x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='105')&(combined_df['track_number']=='07')&(combined_df['time_index']=='4:27')].index,'move1'] = 'PULSE SQUAT'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='105')&(combined_df['track_number']=='07')&(combined_df['time_index']=='4:27')].index,'reps1'] = '2x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='105')&(combined_df['track_number']=='07')&(combined_df['time_index']=='4:27')].index,'move2'] = 'PULSE LUNGE'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='105')&(combined_df['track_number']=='07')&(combined_df['time_index']=='4:27')].index,'reps2'] = '2x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='105')&(combined_df['track_number']=='07')&(combined_df['time_index']=='4:56')].index,'move1'] = 'PULSE SQUAT'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='105')&(combined_df['track_number']=='07')&(combined_df['time_index']=='4:56')].index,'reps1'] = '4x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='105')&(combined_df['track_number']=='07')&(combined_df['time_index']=='4:56')].index,'move2'] = 'PULSE LUNGE'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='105')&(combined_df['track_number']=='07')&(combined_df['time_index']=='4:56')].index,'reps2'] = '4x'

combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='105')&(combined_df['track_number']=='08')].index,'Title'] = 'LEAN BACK'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='105')&(combined_df['track_number']=='08')].index,'Duration'] = '4:49'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='105')&(combined_df['track_number']=='08')&(combined_df['time_index']=='0:32')].index,'move2'] = 'PULSE PUSHUP'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='105')&(combined_df['track_number']=='08')&(combined_df['time_index']=='0:59')].index,'move1'] = 'REAR DELTOID RAISE'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='105')&(combined_df['track_number']=='08')&(combined_df['time_index']=='0:59')].index,'move2'] = 'PULSE REAR DELTOID RAISE'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='105')&(combined_df['track_number']=='08')&(combined_df['time_index']=='2:03')].index,'move2'] = 'PULSE PUSHUP'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='105')&(combined_df['track_number']=='08')&(combined_df['time_index']=='2:31')].index,'move1'] = 'STANDING MAC RAISE'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='105')&(combined_df['track_number']=='08')&(combined_df['time_index']=='2:31')].index,'move1'] = 'PULSE UPRIGHT ROW'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='105')&(combined_df['track_number']=='08')&(combined_df['time_index']=='3:37')].index,'move2'] = 'PULSE PUSHUP'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='105')&(combined_df['track_number']=='08')&(combined_df['time_index']=='4:05')].index,'move1'] = 'STANDING PLATE O/H PRESS'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='105')&(combined_df['track_number']=='08')&(combined_df['time_index']=='4:05')].index,'move1'] = 'PULSE STANDING PLATE O/H PRESS'

combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='105')&(combined_df['track_number']=='09')].index,'Title'] = 'SWEAT'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='105')&(combined_df['track_number']=='09')].index,'Duration'] = '3:09'

combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='105')&(combined_df['track_number']=='10')].index,'Title'] = 'BREAK MY HABITS'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='105')&(combined_df['track_number']=='10')].index,'Duration'] = '2:57'

# 106
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='106')&(combined_df['track_number']=='01')&(combined_df['time_index']=='2:30')].index,'move1'] = 'CLEAN & PRESS'

combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='106')&(combined_df['track_number']=='03')&(combined_df['time_index']=='0:47')].index,'move1'] = '4x BOTTOM HALF CHEST PRESS'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='106')&(combined_df['track_number']=='03')&(combined_df['time_index']=='1:34')].index,'move1'] = '4x BOTTOM HALF CHEST PRESS'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='106')&(combined_df['track_number']=='03')&(combined_df['time_index']=='2:20')].index,'move1'] = '4x BOTTOM HALF CHEST FLY'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='106')&(combined_df['track_number']=='03')&(combined_df['time_index']=='3:46')].index,'move1'] = '4x BOTTOM HALF CHEST PRESS'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='106')&(combined_df['track_number']=='03')&(combined_df['time_index']=='4:32')].index,'move1'] = '4x BOTTOM HALF CHEST FLY'
to_append_106_03 = combined_df[(combined_df['bodypump_release_number']=='106')&(combined_df['track_number']=='03')&(combined_df['time_index']=='4:56')]
to_append_106_03.loc[:,'block']='3'
to_append_106_03.loc[:,'time_index']='2:20'
to_append_106_03.loc[:,'block_clean']='106-03-3'
combined_df = pd.concat([combined_df, to_append_106_03], ignore_index= True)
combined_df = combined_df.sort_values(by=['bodypump_release_number','track_number','block_clean','time_index'], ascending=[True,True,True,True]).reset_index(drop=True)

combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='106')&(combined_df['track_number']=='04')&(combined_df['time_index']=='2:52')].index,'move1'] = 'TRIPLE PULSE KNEELING SINGLE-ARM PLATE ROW'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='106')&(combined_df['track_number']=='04')&(combined_df['time_index']=='4:28')].index,'move1'] = 'TRIPLE PULSE KNEELING SINGLE-ARM PLATE ROW'

combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='106')&(combined_df['track_number']=='05')&(combined_df['time_index']=='1:06')].index,'move1'] = 'TRIPLE PULSE STANDING O/H TRICEP EXTENSION'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='106')&(combined_df['track_number']=='05')&(combined_df['time_index']=='2:53')].index,'move1'] = 'TRIPLE PULSE SEATED O/H TRICEP EXTENSION'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='106')&(combined_df['track_number']=='05')&(combined_df['time_index']=='4:42')].index,'move1'] = 'TRIPLE PULSE STANDING O/H TRICEP EXTENSION'

combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='106')&(combined_df['track_number']=='06')&(combined_df['time_index']=='1:55')].index,'move1'] = 'BICEP PLATE ROW'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='106')&(combined_df['track_number']=='06')&(combined_df['time_index']=='1:55')].index,'reps1'] = '4x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='106')&(combined_df['track_number']=='06')&(combined_df['time_index']=='1:55')].index,'tempo1'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='106')&(combined_df['track_number']=='06')&(combined_df['time_index']=='2:08')].index,'move1'] = 'BICEP PLATE ROW'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='106')&(combined_df['track_number']=='06')&(combined_df['time_index']=='3:59')].index,'move1'] = 'BICEP PLATE ROW'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='106')&(combined_df['track_number']=='06')&(combined_df['time_index']=='3:59')].index,'reps1'] = '4x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='106')&(combined_df['track_number']=='06')&(combined_df['time_index']=='3:59')].index,'tempo1'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='106')&(combined_df['track_number']=='06')&(combined_df['time_index']=='4:12')].index,'move1'] = 'BICEP PLATE ROW'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='106')&(combined_df['track_number']=='06')&(combined_df['time_index']=='0:45')].index,'reps1'] = '2x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='106')&(combined_df['track_number']=='06')&(combined_df['time_index']=='1:26')].index,'reps1'] = '2x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='106')&(combined_df['track_number']=='06')&(combined_df['time_index']=='2:21')].index,'reps1'] = '2x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='106')&(combined_df['track_number']=='06')&(combined_df['time_index']=='3:31')].index,'reps1'] = '2x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='106')&(combined_df['track_number']=='06')&(combined_df['time_index']=='4:26')].index,'reps1'] = '2x'
combined_df.drop(index= combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='106')&(combined_df['track_number']=='06')&(combined_df['time_index']=='Combo with option of the bar:')].index].index, axis='index', inplace = True)

combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='106')&(combined_df['track_number']=='07')&(combined_df['time_index']=='1:07')].index,'move1'] = 'SQUAT'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='106')&(combined_df['track_number']=='07')&(combined_df['time_index']=='1:07')].index,'tempo1'] = '4/4'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='106')&(combined_df['track_number']=='07')&(combined_df['time_index']=='1:07')].index,'reps1'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='106')&(combined_df['track_number']=='07')&(combined_df['time_index']=='1:07')].index,'move2'] = 'LUNGE'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='106')&(combined_df['track_number']=='07')&(combined_df['time_index']=='1:07')].index,'tempo2'] = '4/4'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='106')&(combined_df['track_number']=='07')&(combined_df['time_index']=='1:07')].index,'reps2'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='106')&(combined_df['track_number']=='07')&(combined_df['time_index']=='1:07')].index,'reps3'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='106')&(combined_df['track_number']=='07')&(combined_df['time_index']=='1:07')].index,'move3'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='106')&(combined_df['track_number']=='07')&(combined_df['time_index']=='1:07')].index,'reps4'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='106')&(combined_df['track_number']=='07')&(combined_df['time_index']=='1:07')].index,'move4'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='106')&(combined_df['track_number']=='07')&(combined_df['time_index']=='1:36')].index,'move1'] = 'SQUAT'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='106')&(combined_df['track_number']=='07')&(combined_df['time_index']=='1:36')].index,'tempo1'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='106')&(combined_df['track_number']=='07')&(combined_df['time_index']=='1:36')].index,'reps1'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='106')&(combined_df['track_number']=='07')&(combined_df['time_index']=='1:36')].index,'move2'] = 'LUNGE'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='106')&(combined_df['track_number']=='07')&(combined_df['time_index']=='1:36')].index,'tempo2'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='106')&(combined_df['track_number']=='07')&(combined_df['time_index']=='1:36')].index,'reps2'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='106')&(combined_df['track_number']=='07')&(combined_df['time_index']=='1:36')].index,'reps3'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='106')&(combined_df['track_number']=='07')&(combined_df['time_index']=='1:36')].index,'move3'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='106')&(combined_df['track_number']=='07')&(combined_df['time_index']=='1:36')].index,'reps4'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='106')&(combined_df['track_number']=='07')&(combined_df['time_index']=='1:36')].index,'move4'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='106')&(combined_df['track_number']=='07')&(combined_df['time_index']=='2:05')].index,'reps1'] = '2x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='106')&(combined_df['track_number']=='07')&(combined_df['time_index']=='2:05')].index,'reps2'] = '2x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='106')&(combined_df['track_number']=='07')&(combined_df['time_index']=='2:05')].index,'reps3'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='106')&(combined_df['track_number']=='07')&(combined_df['time_index']=='2:05')].index,'move3'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='106')&(combined_df['track_number']=='07')&(combined_df['time_index']=='2:05')].index,'reps4'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='106')&(combined_df['track_number']=='07')&(combined_df['time_index']=='2:05')].index,'move4'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='106')&(combined_df['track_number']=='07')&(combined_df['time_index']=='3:36')].index,'move1'] = 'SQUAT'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='106')&(combined_df['track_number']=='07')&(combined_df['time_index']=='3:36')].index,'tempo1'] = '4/4'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='106')&(combined_df['track_number']=='07')&(combined_df['time_index']=='3:36')].index,'reps1'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='106')&(combined_df['track_number']=='07')&(combined_df['time_index']=='3:36')].index,'move2'] = 'LUNGE'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='106')&(combined_df['track_number']=='07')&(combined_df['time_index']=='3:36')].index,'tempo2'] = '4/4'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='106')&(combined_df['track_number']=='07')&(combined_df['time_index']=='3:36')].index,'reps2'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='106')&(combined_df['track_number']=='07')&(combined_df['time_index']=='3:36')].index,'reps3'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='106')&(combined_df['track_number']=='07')&(combined_df['time_index']=='3:36')].index,'move3'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='106')&(combined_df['track_number']=='07')&(combined_df['time_index']=='3:36')].index,'reps4'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='106')&(combined_df['track_number']=='07')&(combined_df['time_index']=='3:36')].index,'move4'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='106')&(combined_df['track_number']=='07')&(combined_df['time_index']=='4:05')].index,'move1'] = 'SQUAT'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='106')&(combined_df['track_number']=='07')&(combined_df['time_index']=='4:05')].index,'tempo1'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='106')&(combined_df['track_number']=='07')&(combined_df['time_index']=='4:05')].index,'reps1'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='106')&(combined_df['track_number']=='07')&(combined_df['time_index']=='4:05')].index,'move2'] = 'LUNGE'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='106')&(combined_df['track_number']=='07')&(combined_df['time_index']=='4:05')].index,'tempo2'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='106')&(combined_df['track_number']=='07')&(combined_df['time_index']=='4:05')].index,'reps2'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='106')&(combined_df['track_number']=='07')&(combined_df['time_index']=='4:05')].index,'reps3'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='106')&(combined_df['track_number']=='07')&(combined_df['time_index']=='4:05')].index,'move3'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='106')&(combined_df['track_number']=='07')&(combined_df['time_index']=='4:05')].index,'reps4'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='106')&(combined_df['track_number']=='07')&(combined_df['time_index']=='4:05')].index,'move4'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='106')&(combined_df['track_number']=='07')&(combined_df['time_index']=='4:34')].index,'reps1'] = '2x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='106')&(combined_df['track_number']=='07')&(combined_df['time_index']=='4:34')].index,'reps2'] = '2x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='106')&(combined_df['track_number']=='07')&(combined_df['time_index']=='4:34')].index,'reps3'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='106')&(combined_df['track_number']=='07')&(combined_df['time_index']=='4:34')].index,'move3'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='106')&(combined_df['track_number']=='07')&(combined_df['time_index']=='4:34')].index,'reps4'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='106')&(combined_df['track_number']=='07')&(combined_df['time_index']=='4:34')].index,'move4'] = ''

combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='106')&(combined_df['track_number']=='08')&(combined_df['time_index']=='0:48')].index,'move1'] = 'TRIPLE PULSE KNEELING REAR DELTOID RAISE'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='106')&(combined_df['track_number']=='08')&(combined_df['time_index']=='3:00')].index,'move1'] = 'TRIPLE PULSE UPRIGHT ROW'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='106')&(combined_df['track_number']=='08')&(combined_df['time_index']=='4:29')].index,'move1'] = 'TRIPLE PULSE UPRIGHT ROW'

# 107
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='107')&(combined_df['track_number']=='01')&(combined_df['time_index']=='2:01')].index,'move1'] = 'CLEAN & PRESS'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='107')&(combined_df['track_number']=='01')&(combined_df['time_index']=='2:32')].index,'move1'] = 'SQUAT'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='107')&(combined_df['track_number']=='01')&(combined_df['time_index']=='3:51')].index,'move1'] = 'PLATE CURL SQUAT & PRESS'

combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='107')&(combined_df['track_number']=='02')&(combined_df['time_index']=='2:47')].index,'move1'] = 'SQUAT WIDE Stance'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='107')&(combined_df['track_number']=='02')&(combined_df['time_index']=='5:23')].index,'move1'] = 'SQUAT WIDER Stance'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='107')&(combined_df['track_number']=='02')&(combined_df['time_index']=='5:52')].index,'move1'] = 'SQUAT WIDER Stance'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='107')&(combined_df['track_number']=='02')&(combined_df['time_index']=='5:52')].index,'tempo1'] = '4/4'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='107')&(combined_df['track_number']=='02')&(combined_df['time_index']=='5:52')].index,'reps1'] = '2x'

combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='107')&(combined_df['track_number']=='03')&(combined_df['time_index']=='2:32')].index,'move2'] = 'TRIPLE PULSE A-PRESS'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='107')&(combined_df['track_number']=='03')&(combined_df['time_index']=='2:32')].index,'reps2'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='107')&(combined_df['track_number']=='03')&(combined_df['time_index']=='2:32')].index,'move3'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='107')&(combined_df['track_number']=='03')&(combined_df['time_index']=='2:32')].index,'reps3'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='107')&(combined_df['track_number']=='03')&(combined_df['time_index']=='3:46')].index,'move2'] = '4x PULSE PUSHUP'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='107')&(combined_df['track_number']=='03')&(combined_df['time_index']=='4:43')].index,'move2'] = 'TRIPLE PULSE A-PRESS'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='107')&(combined_df['track_number']=='03')&(combined_df['time_index']=='4:43')].index,'reps2'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='107')&(combined_df['track_number']=='03')&(combined_df['time_index']=='4:43')].index,'move3'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='107')&(combined_df['track_number']=='03')&(combined_df['time_index']=='4:43')].index,'reps3'] = ''

combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='107')&(combined_df['track_number']=='05')&(combined_df['time_index']=='0:29')].index,'reps'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='107')&(combined_df['track_number']=='05')&(combined_df['time_index']=='0:29')].index,'reps1'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='107')&(combined_df['track_number']=='05')&(combined_df['time_index']=='1:09')].index,'reps'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='107')&(combined_df['track_number']=='05')&(combined_df['time_index']=='1:09')].index,'reps1'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='107')&(combined_df['track_number']=='05')&(combined_df['time_index']=='1:33')].index,'move1'] = '16x PULSE STANDING O/H TRICEP EXTENSION'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='107')&(combined_df['track_number']=='05')&(combined_df['time_index']=='1:33')].index,'reps'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='107')&(combined_df['track_number']=='05')&(combined_df['time_index']=='1:33')].index,'reps1'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='107')&(combined_df['track_number']=='05')&(combined_df['time_index']=='2:14')].index,'move1'] = '16x PULSE STANDING O/H TRICEP EXTENSION'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='107')&(combined_df['track_number']=='05')&(combined_df['time_index']=='2:14')].index,'reps'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='107')&(combined_df['track_number']=='05')&(combined_df['time_index']=='2:14')].index,'reps1'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='107')&(combined_df['track_number']=='05')&(combined_df['time_index']=='2:54')].index,'reps'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='107')&(combined_df['track_number']=='05')&(combined_df['time_index']=='2:54')].index,'reps1'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='107')&(combined_df['track_number']=='05')&(combined_df['time_index']=='4:00')].index,'move1'] = '16x PULSE TRICEP BENCH KICKBACK'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='107')&(combined_df['track_number']=='05')&(combined_df['time_index']=='4:00')].index,'reps'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='107')&(combined_df['track_number']=='05')&(combined_df['time_index']=='4:00')].index,'reps1'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='107')&(combined_df['track_number']=='05')&(combined_df['time_index']=='4:40')].index,'move1'] = '16x PULSE STANDING O/H TRICEP EXTENSION'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='107')&(combined_df['track_number']=='05')&(combined_df['time_index']=='4:40')].index,'reps'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='107')&(combined_df['track_number']=='05')&(combined_df['time_index']=='4:40')].index,'reps1'] = '1x'

combined_df['time_index'] = combined_df['time_index'].str.replace('.', ':')
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='107')&(combined_df['track_number']=='06')&(combined_df['time_index']=='1:09')].index,'move1'] = '3x MID-RANGE PULSE'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='107')&(combined_df['track_number']=='06')&(combined_df['time_index']=='2:11')].index,'move1'] = '3x MID-RANGE PULSE'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='107')&(combined_df['track_number']=='06')&(combined_df['time_index']=='2:55')].index,'move1'] = '3x MID-RANGE PULSE'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='107')&(combined_df['track_number']=='06')&(combined_df['time_index']=='3:45')].index,'move1'] = '3x MID-RANGE PULSE'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='107')&(combined_df['track_number']=='06')&(combined_df['time_index']=='4:27')].index,'move1'] = '3x MID-RANGE PULSE'

combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='107')&(combined_df['track_number']=='07')&(combined_df['time_index']=='0:58')].index,'move2'] = 'TRIPLE PULSE LUNGE'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='107')&(combined_df['track_number']=='07')&(combined_df['time_index']=='0:58')].index,'move3'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='107')&(combined_df['track_number']=='07')&(combined_df['time_index']=='0:58')].index,'reps3'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='107')&(combined_df['track_number']=='07')&(combined_df['time_index']=='0:58')].index,'move4'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='107')&(combined_df['track_number']=='07')&(combined_df['time_index']=='0:58')].index,'reps4'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='107')&(combined_df['track_number']=='07')&(combined_df['time_index']=='2:04')].index,'move2'] = 'TRIPLE PULSE LUNGE'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='107')&(combined_df['track_number']=='07')&(combined_df['time_index']=='2:04')].index,'move3'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='107')&(combined_df['track_number']=='07')&(combined_df['time_index']=='2:04')].index,'reps3'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='107')&(combined_df['track_number']=='07')&(combined_df['time_index']=='2:04')].index,'move4'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='107')&(combined_df['track_number']=='07')&(combined_df['time_index']=='2:04')].index,'reps4'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='107')&(combined_df['track_number']=='07')&(combined_df['time_index']=='2:38')].index,'move2'] = 'TRIPLE PULSE FRONT SQUAT'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='107')&(combined_df['track_number']=='07')&(combined_df['time_index']=='2:38')].index,'move3'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='107')&(combined_df['track_number']=='07')&(combined_df['time_index']=='2:38')].index,'reps3'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='107')&(combined_df['track_number']=='07')&(combined_df['time_index']=='2:38')].index,'move4'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='107')&(combined_df['track_number']=='07')&(combined_df['time_index']=='2:38')].index,'reps4'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='107')&(combined_df['track_number']=='07')&(combined_df['time_index']=='3:23')].index,'move1'] = 'PLATE FRONT SQUAT'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='107')&(combined_df['track_number']=='07')&(combined_df['time_index']=='3:23')].index,'reps1'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='107')&(combined_df['track_number']=='07')&(combined_df['time_index']=='3:23')].index,'tempo1'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='107')&(combined_df['track_number']=='07')&(combined_df['time_index']=='3:23')].index,'move2'] = 'BACKWARD-STEPPING LUNGE'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='107')&(combined_df['track_number']=='07')&(combined_df['time_index']=='3:23')].index,'reps2'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='107')&(combined_df['track_number']=='07')&(combined_df['time_index']=='3:23')].index,'tempo2'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='107')&(combined_df['track_number']=='07')&(combined_df['time_index']=='3:23')].index,'move3'] = 'PLATE FRONT SQUAT'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='107')&(combined_df['track_number']=='07')&(combined_df['time_index']=='3:23')].index,'reps3'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='107')&(combined_df['track_number']=='07')&(combined_df['time_index']=='3:23')].index,'tempo3'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='107')&(combined_df['track_number']=='07')&(combined_df['time_index']=='3:23')].index,'move4'] = 'BACKWARD-STEPPING LUNGE'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='107')&(combined_df['track_number']=='07')&(combined_df['time_index']=='3:23')].index,'reps4'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='107')&(combined_df['track_number']=='07')&(combined_df['time_index']=='3:23')].index,'tempo4'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='107')&(combined_df['track_number']=='07')&(combined_df['time_index']=='4:10')].index,'move2'] = 'TRIPLE PULSE FRONT SQUAT'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='107')&(combined_df['track_number']=='07')&(combined_df['time_index']=='4:10')].index,'move3'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='107')&(combined_df['track_number']=='07')&(combined_df['time_index']=='4:10')].index,'reps3'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='107')&(combined_df['track_number']=='07')&(combined_df['time_index']=='4:10')].index,'move4'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='107')&(combined_df['track_number']=='07')&(combined_df['time_index']=='4:10')].index,'reps4'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='107')&(combined_df['track_number']=='07')&(combined_df['time_index']=='4:56')].index,'move1'] = 'PLATE FRONT SQUAT'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='107')&(combined_df['track_number']=='07')&(combined_df['time_index']=='4:56')].index,'reps1'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='107')&(combined_df['track_number']=='07')&(combined_df['time_index']=='4:56')].index,'tempo1'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='107')&(combined_df['track_number']=='07')&(combined_df['time_index']=='4:56')].index,'move2'] = 'BACKWARD-STEPPING LUNGE'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='107')&(combined_df['track_number']=='07')&(combined_df['time_index']=='4:56')].index,'reps2'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='107')&(combined_df['track_number']=='07')&(combined_df['time_index']=='4:56')].index,'tempo2'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='107')&(combined_df['track_number']=='07')&(combined_df['time_index']=='4:56')].index,'move3'] = 'PLATE FRONT SQUAT'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='107')&(combined_df['track_number']=='07')&(combined_df['time_index']=='4:56')].index,'reps3'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='107')&(combined_df['track_number']=='07')&(combined_df['time_index']=='4:56')].index,'tempo3'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='107')&(combined_df['track_number']=='07')&(combined_df['time_index']=='4:56')].index,'move4'] = 'BACKWARD-STEPPING LUNGE'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='107')&(combined_df['track_number']=='07')&(combined_df['time_index']=='4:56')].index,'reps4'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='107')&(combined_df['track_number']=='07')&(combined_df['time_index']=='4:56')].index,'tempo4'] = '1/1'

combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='107')&(combined_df['track_number']=='08')&(combined_df['time_index']=='1:03')].index,'move1'] = 'STANDING TRIPLE REAR DELTOID FLY'
to_append_107_08 = combined_df[(combined_df['bodypump_release_number']=='107')&(combined_df['track_number']=='08')&(combined_df['time_index']=='5:00')]
to_append_107_08.loc[:,'block']='2'
to_append_107_08.loc[:,'time_index']='3:09'
to_append_107_08.loc[:,'block_clean']='107-08-2'
combined_df = pd.concat([combined_df, to_append_107_08], ignore_index= True)
combined_df = combined_df.sort_values(by=['bodypump_release_number','track_number','block_clean','time_index'], ascending=[True,True,True,True]).reset_index(drop=True)
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='107')&(combined_df['track_number']=='08')&(combined_df['time_index']=='5:14')].index,'move1'] = '16x PULSE PUSHUP'

combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='107')&(combined_df['track_number']=='09')&(combined_df['time_index']=='0:50')].index,'move1'] = '16x PULSE CRUNCH'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='107')&(combined_df['track_number']=='09')&(combined_df['time_index']=='0:50')].index,'reps'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='107')&(combined_df['track_number']=='09')&(combined_df['time_index']=='0:50')].index,'reps1'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='107')&(combined_df['track_number']=='09')&(combined_df['time_index']=='1:26')].index,'move1'] = '12x PULSE HIP BRIDGE'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='107')&(combined_df['track_number']=='09')&(combined_df['time_index']=='1:26')].index,'reps'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='107')&(combined_df['track_number']=='09')&(combined_df['time_index']=='1:26')].index,'reps1'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='107')&(combined_df['track_number']=='09')&(combined_df['time_index']=='2:11')].index,'move1'] = '16x PULSE CRUNCH'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='107')&(combined_df['track_number']=='09')&(combined_df['time_index']=='2:11')].index,'reps'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='107')&(combined_df['track_number']=='09')&(combined_df['time_index']=='2:11')].index,'reps1'] = '1x'

# 108
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='108')&(combined_df['track_number']=='01')&(combined_df['time_index']=='2:29')].index,'move1'] = 'CLEAN & PRESS'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='108')&(combined_df['track_number']=='01')&(combined_df['time_index']=='4:30')].index,'move1'] = 'CLEAN & PRESS'

combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='108')&(combined_df['track_number']=='02')&(combined_df['time_index']=='3:43')].index,'move1'] = 'SQUAT'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='108')&(combined_df['track_number']=='02')&(combined_df['time_index']=='5:11')].index,'move1'] = 'SQUAT'

combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='108')&(combined_df['track_number']=='04')&(combined_df['time_index']=='0:33')].index,'move1'] = 'WIDE DEADROW'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='108')&(combined_df['track_number']=='04')&(combined_df['time_index']=='0:33')].index,'tempo1'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='108')&(combined_df['track_number']=='04')&(combined_df['time_index']=='0:33')].index,'reps1'] = '10x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='108')&(combined_df['track_number']=='04')&(combined_df['time_index']=='3:03')].index,'combo_clean'] = '108-04-COMBO 2'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='108')&(combined_df['track_number']=='04')&(combined_df['time_index']=='3:03')].index,'move2'] = 'TRIPLE DEADROW'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='108')&(combined_df['track_number']=='04')&(combined_df['time_index']=='3:03')].index,'reps2'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='108')&(combined_df['track_number']=='04')&(combined_df['time_index']=='5:14')].index,'reps2'] = '3x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='108')&(combined_df['track_number']=='04')&(combined_df['time_index']=='4:01')].index,'reps2'] = '3x'

combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='108')&(combined_df['track_number']=='05')&(combined_df['time_index']=='0:42')].index,'no_rep'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='108')&(combined_df['track_number']=='05')&(combined_df['time_index']=='0:42')].index,'move1'] = 'STANDING O/H TRICEP PLATE EXTENSION'

combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='108')&(combined_df['track_number']=='07')&(combined_df['time_index']=='2:05')].index,'tempo1'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='108')&(combined_df['track_number']=='07')&(combined_df['time_index']=='2:05')].index,'tempo2'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='108')&(combined_df['track_number']=='07')&(combined_df['time_index']=='2:05')].index,'move3'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='108')&(combined_df['track_number']=='07')&(combined_df['time_index']=='2:05')].index,'tempo3'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='108')&(combined_df['track_number']=='07')&(combined_df['time_index']=='2:05')].index,'reps3'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='108')&(combined_df['track_number']=='07')&(combined_df['time_index']=='2:05')].index,'move4'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='108')&(combined_df['track_number']=='07')&(combined_df['time_index']=='2:05')].index,'tempo4'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='108')&(combined_df['track_number']=='07')&(combined_df['time_index']=='2:05')].index,'reps4'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='108')&(combined_df['track_number']=='07')&(combined_df['time_index']=='3:00')].index,'tempo1'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='108')&(combined_df['track_number']=='07')&(combined_df['time_index']=='3:00')].index,'tempo2'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='108')&(combined_df['track_number']=='07')&(combined_df['time_index']=='3:00')].index,'move3'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='108')&(combined_df['track_number']=='07')&(combined_df['time_index']=='3:00')].index,'tempo3'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='108')&(combined_df['track_number']=='07')&(combined_df['time_index']=='3:00')].index,'reps3'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='108')&(combined_df['track_number']=='07')&(combined_df['time_index']=='3:00')].index,'move4'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='108')&(combined_df['track_number']=='07')&(combined_df['time_index']=='3:00')].index,'tempo4'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='108')&(combined_df['track_number']=='07')&(combined_df['time_index']=='3:00')].index,'reps4'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='108')&(combined_df['track_number']=='07')&(combined_df['time_index']=='3:44')].index,'tempo1'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='108')&(combined_df['track_number']=='07')&(combined_df['time_index']=='3:44')].index,'tempo2'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='108')&(combined_df['track_number']=='07')&(combined_df['time_index']=='3:44')].index,'move3'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='108')&(combined_df['track_number']=='07')&(combined_df['time_index']=='3:44')].index,'tempo3'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='108')&(combined_df['track_number']=='07')&(combined_df['time_index']=='3:44')].index,'reps3'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='108')&(combined_df['track_number']=='07')&(combined_df['time_index']=='3:44')].index,'move4'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='108')&(combined_df['track_number']=='07')&(combined_df['time_index']=='3:44')].index,'tempo4'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='108')&(combined_df['track_number']=='07')&(combined_df['time_index']=='3:44')].index,'reps4'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='108')&(combined_df['track_number']=='07')&(combined_df['time_index']=='4:39')].index,'tempo1'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='108')&(combined_df['track_number']=='07')&(combined_df['time_index']=='4:39')].index,'tempo2'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='108')&(combined_df['track_number']=='07')&(combined_df['time_index']=='4:39')].index,'move3'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='108')&(combined_df['track_number']=='07')&(combined_df['time_index']=='4:39')].index,'tempo3'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='108')&(combined_df['track_number']=='07')&(combined_df['time_index']=='4:39')].index,'reps3'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='108')&(combined_df['track_number']=='07')&(combined_df['time_index']=='4:39')].index,'move4'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='108')&(combined_df['track_number']=='07')&(combined_df['time_index']=='4:39')].index,'tempo4'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='108')&(combined_df['track_number']=='07')&(combined_df['time_index']=='4:39')].index,'reps4'] = ''

combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='108')&(combined_df['track_number']=='08')&(combined_df['time_index']=='4:20')].index,'move1'] = 'UPRIGHT ROW'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='108')&(combined_df['track_number']=='08')&(combined_df['time_index']=='4:20')].index,'move1'] = 'UPRIGHT ROW'

combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='108')&(combined_df['track_number']=='09')&(combined_df['time_index']=='0:41')].index,'move1'] = 'PLATE CRUNCH'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='108')&(combined_df['track_number']=='09')&(combined_df['time_index']=='0:41')].index,'reps1'] = '2x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='108')&(combined_df['track_number']=='08')&(combined_df['time_index']=='0:41')].index,'move2'] = 'SHOOT-OUT'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='108')&(combined_df['track_number']=='08')&(combined_df['time_index']=='0:41')].index,'reps2'] = '2x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='108')&(combined_df['track_number']=='09')&(combined_df['time_index']=='1:36')].index,'move1'] = 'PLATE CRUNCH'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='108')&(combined_df['track_number']=='09')&(combined_df['time_index']=='1:36')].index,'reps1'] = '2x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='108')&(combined_df['track_number']=='08')&(combined_df['time_index']=='1:36')].index,'move2'] = 'SHOOT-OUT'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='108')&(combined_df['track_number']=='08')&(combined_df['time_index']=='1:36')].index,'reps2'] = '2x'

# 109
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='109')&(combined_df['track_number']=='01')&(combined_df['time_index']=='2:19')].index,'move1'] = 'CLEAN & PRESS'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='109')&(combined_df['track_number']=='01')&(combined_df['time_index']=='2:48')].index,'move1'] = 'SQUAT MID Stance'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='109')&(combined_df['track_number']=='01')&(combined_df['time_index']=='4:17')].index,'move1'] = 'CLEAN & PRESS'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='109')&(combined_df['track_number']=='01')&(combined_df['time_index']=='4:47')].index,'move1'] = 'SQUAT MID Stance'

to_append_109_02 = combined_df[(combined_df['bodypump_release_number']=='109')&(combined_df['track_number']=='02')&(combined_df['time_index']=='2:09')]
to_append_109_02.loc[:,'block']='2'
to_append_109_02.loc[:,'time_index']='2:50'
to_append_109_02.loc[:,'block_clean']='111-06-2'
combined_df = pd.concat([combined_df, to_append_109_02], ignore_index= True)
to_append_109_02_bis = combined_df[(combined_df['bodypump_release_number']=='109')&(combined_df['track_number']=='02')&(combined_df['time_index']=='2:09')]
to_append_109_02_bis.loc[:,'block']='4'
to_append_109_02_bis.loc[:,'time_index']='5:56'
to_append_109_02_bis.loc[:,'block_clean']='111-06-4'
combined_df = pd.concat([combined_df, to_append_109_02_bis], ignore_index= True)
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='109')&(combined_df['track_number']=='02')&(combined_df['time_index']=='2:50')].index,'reps'] = '9x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='109')&(combined_df['track_number']=='02')&(combined_df['time_index']=='2:50')].index,'reps1'] = '9x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='109')&(combined_df['track_number']=='02')&(combined_df['time_index']=='2:50')].index,'tempo1'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='109')&(combined_df['track_number']=='02')&(combined_df['time_index']=='5:56')].index,'reps'] = '9x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='109')&(combined_df['track_number']=='02')&(combined_df['time_index']=='5:56')].index,'reps1'] = '9x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='109')&(combined_df['track_number']=='02')&(combined_df['time_index']=='5:56')].index,'tempo1'] = '1/1'
combined_df = combined_df.sort_values(by=['bodypump_release_number','track_number','block_clean','time_index'], ascending=[True,True,True,True]).reset_index(drop=True)

combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='109')&(combined_df['track_number']=='03')&(combined_df['time_index']=='0:07')].index,'move1'] = 'WALKING BENCH PUSHUP'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='109')&(combined_df['track_number']=='03')&(combined_df['time_index']=='0:07')].index,'tempo1'] = '2/2'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='109')&(combined_df['track_number']=='03')&(combined_df['time_index']=='0:07')].index,'reps1'] = '8x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='109')&(combined_df['track_number']=='03')&(combined_df['time_index']=='0:39')].index,'move1'] = 'WALKING BENCH PUSHUP'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='109')&(combined_df['track_number']=='03')&(combined_df['time_index']=='0:39')].index,'reps1'] = '4x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='109')&(combined_df['track_number']=='03')&(combined_df['time_index']=='0:55')].index,'move1'] = 'WALKING BENCH PUSHUP'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='109')&(combined_df['track_number']=='03')&(combined_df['time_index']=='0:55')].index,'reps1'] = '8x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='109')&(combined_df['track_number']=='03')&(combined_df['time_index']=='2:07')].index,'combo_clean'] = '109-03-1 COMBINATION'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='109')&(combined_df['track_number']=='03')&(combined_df['time_index']=='2:07')].index,'tempo1'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='109')&(combined_df['track_number']=='03')&(combined_df['time_index']=='2:07')].index,'reps1'] = '2x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='109')&(combined_df['track_number']=='03')&(combined_df['time_index']=='2:07')].index,'move2'] = '3x PULSE CHEST PRESS'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='109')&(combined_df['track_number']=='03')&(combined_df['time_index']=='2:07')].index,'reps2'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='109')&(combined_df['track_number']=='03')&(combined_df['time_index']=='2:43')].index,'tempo1'] = '2/2'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='109')&(combined_df['track_number']=='03')&(combined_df['time_index']=='2:43')].index,'reps1'] = '8x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='109')&(combined_df['track_number']=='03')&(combined_df['time_index']=='3:15')].index,'move1'] = 'WALKING BENCH PUSHUP'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='109')&(combined_df['track_number']=='03')&(combined_df['time_index']=='3:15')].index,'reps1'] = '4x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='109')&(combined_df['track_number']=='03')&(combined_df['time_index']=='3:31')].index,'move1'] = 'WALKING BENCH PUSHUP'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='109')&(combined_df['track_number']=='03')&(combined_df['time_index']=='3:31')].index,'reps1'] = '8x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='109')&(combined_df['track_number']=='03')&(combined_df['time_index']=='4:43')].index,'combo_clean'] = '109-03-1 COMBINATION'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='109')&(combined_df['track_number']=='03')&(combined_df['time_index']=='4:43')].index,'tempo1'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='109')&(combined_df['track_number']=='03')&(combined_df['time_index']=='4:43')].index,'reps1'] = '2x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='109')&(combined_df['track_number']=='03')&(combined_df['time_index']=='4:43')].index,'move2'] = '3x PULSE CHEST PRESS'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='109')&(combined_df['track_number']=='03')&(combined_df['time_index']=='4:43')].index,'reps2'] = '1x'

combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='109')&(combined_df['track_number']=='04')&(combined_df['time_index']=='0:56')].index,'combo_clean'] = '109-04-1 COMBINATION'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='109')&(combined_df['track_number']=='04')&(combined_df['time_index']=='0:56')].index,'tempo1'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='109')&(combined_df['track_number']=='04')&(combined_df['time_index']=='0:56')].index,'move2'] = 'WIDE DEADROW'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='109')&(combined_df['track_number']=='04')&(combined_df['time_index']=='0:56')].index,'reps2'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='109')&(combined_df['track_number']=='04')&(combined_df['time_index']=='0:56')].index,'tempo2'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='109')&(combined_df['track_number']=='04')&(combined_df['time_index']=='2:13')].index,'move1'] = 'WIDE HIGH PULL'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='109')&(combined_df['track_number']=='04')&(combined_df['time_index']=='2:13')].index,'tempo1'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='109')&(combined_df['track_number']=='04')&(combined_df['time_index']=='2:13')].index,'reps1'] = '2x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='109')&(combined_df['track_number']=='04')&(combined_df['time_index']=='2:13')].index,'move2'] = 'WIDE DEADROW'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='109')&(combined_df['track_number']=='04')&(combined_df['time_index']=='2:13')].index,'tempo2'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='109')&(combined_df['track_number']=='04')&(combined_df['time_index']=='2:13')].index,'reps2'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='109')&(combined_df['track_number']=='04')&(combined_df['time_index']=='4:21')].index,'move1'] = 'WIDE HIGH PULL'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='109')&(combined_df['track_number']=='04')&(combined_df['time_index']=='4:21')].index,'tempo1'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='109')&(combined_df['track_number']=='04')&(combined_df['time_index']=='4:21')].index,'reps1'] = '2x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='109')&(combined_df['track_number']=='04')&(combined_df['time_index']=='4:21')].index,'move2'] = 'WIDE DEADROW'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='109')&(combined_df['track_number']=='04')&(combined_df['time_index']=='4:21')].index,'tempo2'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='109')&(combined_df['track_number']=='04')&(combined_df['time_index']=='4:21')].index,'reps2'] = '1x'

combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='109')&(combined_df['track_number']=='05')&(combined_df['time_index']=='0:56')].index,'move1'] = '3x PULSE SEATED O/H TRICEP PLATE EXTENSION'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='109')&(combined_df['track_number']=='05')&(combined_df['time_index']=='2:00')].index,'move1'] = '3x PULSE SEATED O/H TRICEP PLATE EXTENSION'

combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='109')&(combined_df['track_number']=='06')&(combined_df['time_index']=='0:46')].index,'combo_clean'] = '109-06-1 COMBINATION'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='109')&(combined_df['track_number']=='06')&(combined_df['time_index']=='0:46')].index,'move2'] = 'BICEP CURL'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='109')&(combined_df['track_number']=='06')&(combined_df['time_index']=='0:46')].index,'tempo2'] = '1/1/2'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='109')&(combined_df['track_number']=='06')&(combined_df['time_index']=='0:46')].index,'reps2'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='109')&(combined_df['track_number']=='06')&(combined_df['time_index']=='1:42')].index,'combo_clean'] = '109-06-1 COMBINATION'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='109')&(combined_df['track_number']=='06')&(combined_df['time_index']=='1:42')].index,'move1'] = 'BICEP CURL'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='109')&(combined_df['track_number']=='06')&(combined_df['time_index']=='1:42')].index,'tempo1'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='109')&(combined_df['track_number']=='06')&(combined_df['time_index']=='1:42')].index,'reps1'] = '2x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='109')&(combined_df['track_number']=='06')&(combined_df['time_index']=='1:42')].index,'move2'] = 'BICEP CURL'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='109')&(combined_df['track_number']=='06')&(combined_df['time_index']=='1:42')].index,'tempo2'] = '1/1/2'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='109')&(combined_df['track_number']=='06')&(combined_df['time_index']=='1:42')].index,'reps2'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='109')&(combined_df['track_number']=='06')&(combined_df['time_index']=='2:32')].index,'combo_clean'] = '109-06-1 COMBINATION'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='109')&(combined_df['track_number']=='06')&(combined_df['time_index']=='2:32')].index,'move1'] = 'BICEP CURL'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='109')&(combined_df['track_number']=='06')&(combined_df['time_index']=='2:32')].index,'tempo1'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='109')&(combined_df['track_number']=='06')&(combined_df['time_index']=='2:32')].index,'reps1'] = '2x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='109')&(combined_df['track_number']=='06')&(combined_df['time_index']=='2:32')].index,'move2'] = 'BICEP CURL'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='109')&(combined_df['track_number']=='06')&(combined_df['time_index']=='2:32')].index,'tempo2'] = '1/1/2'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='109')&(combined_df['track_number']=='06')&(combined_df['time_index']=='2:32')].index,'reps2'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='109')&(combined_df['track_number']=='06')&(combined_df['time_index']=='3:27')].index,'combo_clean'] = '109-06-1 COMBINATION'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='109')&(combined_df['track_number']=='06')&(combined_df['time_index']=='3:27')].index,'move1'] = 'BICEP CURL'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='109')&(combined_df['track_number']=='06')&(combined_df['time_index']=='3:27')].index,'tempo1'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='109')&(combined_df['track_number']=='06')&(combined_df['time_index']=='3:27')].index,'reps1'] = '2x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='109')&(combined_df['track_number']=='06')&(combined_df['time_index']=='3:27')].index,'move2'] = 'BICEP CURL'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='109')&(combined_df['track_number']=='06')&(combined_df['time_index']=='3:27')].index,'tempo2'] = '1/1/2'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='109')&(combined_df['track_number']=='06')&(combined_df['time_index']=='3:27')].index,'reps2'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='109')&(combined_df['track_number']=='06')&(combined_df['time_index']=='4:17')].index,'combo_clean'] = '109-06-1 COMBINATION'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='109')&(combined_df['track_number']=='06')&(combined_df['time_index']=='4:17')].index,'move1'] = 'BICEP CURL'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='109')&(combined_df['track_number']=='06')&(combined_df['time_index']=='4:17')].index,'tempo1'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='109')&(combined_df['track_number']=='06')&(combined_df['time_index']=='4:17')].index,'reps1'] = '2x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='109')&(combined_df['track_number']=='06')&(combined_df['time_index']=='4:17')].index,'move2'] = 'BICEP CURL'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='109')&(combined_df['track_number']=='06')&(combined_df['time_index']=='4:17')].index,'tempo2'] = '1/1/2'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='109')&(combined_df['track_number']=='06')&(combined_df['time_index']=='4:17')].index,'reps2'] = '1x'

combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='109')&(combined_df['track_number']=='07')&(combined_df['time_index']=='1:08')].index,'move1'] = '2x BOTTOM HALF FRONT SQUAT'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='109')&(combined_df['track_number']=='07')&(combined_df['time_index']=='2:08')].index,'move1'] = 'PLATE FRONT SQUAT'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='109')&(combined_df['track_number']=='07')&(combined_df['time_index']=='2:56')].index,'move1'] = 'PLATE FRONT SQUAT'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='109')&(combined_df['track_number']=='07')&(combined_df['time_index']=='3:08')].index,'move1'] = '3x PULSE BENCH BACKWARD-STEPPING LUNGE'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='109')&(combined_df['track_number']=='07')&(combined_df['time_index']=='3:20')].index,'move1'] = '3x PULSE BENCH BACKWARD-STEPPING LUNGE'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='109')&(combined_df['track_number']=='07')&(combined_df['time_index']=='4:08')].index,'move1'] = 'PLATE FRONT SQUAT'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='109')&(combined_df['track_number']=='07')&(combined_df['time_index']=='4:20')].index,'move1'] = '3x PULSE BENCH BACKWARD-STEPPING LUNGE'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='109')&(combined_df['track_number']=='07')&(combined_df['time_index']=='4:56')].index,'move1'] = 'PLATE FRONT SQUAT'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='109')&(combined_df['track_number']=='07')&(combined_df['time_index']=='5:08')].index,'move1'] = '3x PULSE BENCH BACKWARD-STEPPING LUNGE'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='109')&(combined_df['track_number']=='07')&(combined_df['time_index']=='5:20')].index,'move1'] = '3x PULSE BENCH BACKWARD-STEPPING LUNGE'

combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='109')&(combined_df['track_number']=='08')&(combined_df['time_index']=='1:54')].index,'move1'] = 'KNEELING FORWARD RAISE'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='109')&(combined_df['track_number']=='08')&(combined_df['time_index']=='1:54')].index,'tempo1'] = '2/2'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='109')&(combined_df['track_number']=='08')&(combined_df['time_index']=='1:54')].index,'reps1'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='109')&(combined_df['track_number']=='08')&(combined_df['time_index']=='1:54')].index,'move2'] = 'KNEELING SIDE RAISE'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='109')&(combined_df['track_number']=='08')&(combined_df['time_index']=='1:54')].index,'tempo2'] = '2/2'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='109')&(combined_df['track_number']=='08')&(combined_df['time_index']=='1:54')].index,'reps2'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='109')&(combined_df['track_number']=='08')&(combined_df['time_index']=='3:42')].index,'move1'] = 'KNEELING FORWARD RAISE'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='109')&(combined_df['track_number']=='08')&(combined_df['time_index']=='3:42')].index,'tempo1'] = '2/2'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='109')&(combined_df['track_number']=='08')&(combined_df['time_index']=='3:42')].index,'reps1'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='109')&(combined_df['track_number']=='08')&(combined_df['time_index']=='3:42')].index,'move2'] = 'KNEELING SIDE RAISE'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='109')&(combined_df['track_number']=='08')&(combined_df['time_index']=='3:42')].index,'tempo2'] = '2/2'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='109')&(combined_df['track_number']=='08')&(combined_df['time_index']=='3:42')].index,'reps2'] = '1x'

combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='109')&(combined_df['track_number']=='09')&(combined_df['time_index']=='0:44')].index,'move1'] = 'SINGLE-ARM PLATE LIFT'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='109')&(combined_df['track_number']=='09')&(combined_df['time_index']=='2:20')].index,'move1'] = 'SINGLE-ARM PLATE LIFT'

# 111
combined_df.drop(index= combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='111')&(combined_df['track_number']=='02')&(combined_df['time_index']=='1x8')].index].index, axis='index', inplace = True)
combined_df.drop(index= combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='111')&(combined_df['track_number']=='02')&(combined_df['time_index']=='8x8')].index].index, axis='index', inplace = True)
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='111')&(combined_df['track_number']=='02')&(combined_df['time_index']=='3:29')].index,'move1'] = 'SQUAT WIDER Stance'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='111')&(combined_df['track_number']=='02')&(combined_df['time_index']=='3:43')].index,'move1'] = 'SQUAT'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='111')&(combined_df['track_number']=='02')&(combined_df['time_index']=='3:43')].index,'tempo1'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='111')&(combined_df['track_number']=='02')&(combined_df['time_index']=='3:43')].index,'reps1'] = '2x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='111')&(combined_df['track_number']=='02')&(combined_df['time_index']=='3:43')].index,'move2'] = '3x PULSE SQUAT'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='111')&(combined_df['track_number']=='02')&(combined_df['time_index']=='3:43')].index,'reps1'] = '1x'
to_append_111_02 = combined_df[(combined_df['bodypump_release_number']=='111')&(combined_df['track_number']=='05')&(combined_df['time_index']=='3:43')]
to_append_111_02.loc[:,'block']='2'
to_append_111_02.loc[:,'time_index']='1:47'
to_append_111_02.loc[:,'block_clean']='111-02-2'
combined_df = pd.concat([combined_df, to_append_111_02], ignore_index= True)
combined_df = combined_df.sort_values(by=['bodypump_release_number','track_number','block_clean','time_index'], ascending=[True,True,True,True]).reset_index(drop=True)

combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='111')&(combined_df['track_number']=='03')&(combined_df['time_index']=='3:26')].index,'no_rep'] = 'HOLD'

combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='111')&(combined_df['track_number']=='04')&(combined_df['time_index']=='0:17')].index,'move1'] = 'DEADROW'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='111')&(combined_df['track_number']=='04')&(combined_df['time_index']=='0:17')].index,'tempo1'] = '2/2'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='111')&(combined_df['track_number']=='04')&(combined_df['time_index']=='0:17')].index,'reps1'] = '6x'

combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='111')&(combined_df['track_number']=='04')&(combined_df['time_index']=='2:29')].index,'combo_clean'] = '111-04-COMBINATION'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='111')&(combined_df['track_number']=='04')&(combined_df['time_index']=='2:29')].index,'move1'] = 'LOW PULL'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='111')&(combined_df['track_number']=='04')&(combined_df['time_index']=='2:29')].index,'tempo1'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='111')&(combined_df['track_number']=='04')&(combined_df['time_index']=='2:29')].index,'reps1'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='111')&(combined_df['track_number']=='04')&(combined_df['time_index']=='2:29')].index,'move2'] = 'HIGH PULL'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='111')&(combined_df['track_number']=='04')&(combined_df['time_index']=='2:29')].index,'tempo2'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='111')&(combined_df['track_number']=='04')&(combined_df['time_index']=='2:29')].index,'reps2'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='111')&(combined_df['track_number']=='04')&(combined_df['time_index']=='2:29')].index,'move3'] = 'CLEAN & PRESS'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='111')&(combined_df['track_number']=='04')&(combined_df['time_index']=='2:29')].index,'tempo3'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='111')&(combined_df['track_number']=='04')&(combined_df['time_index']=='2:29')].index,'reps3'] = '1x'

combined_df.drop(index= combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='111')&(combined_df['track_number']=='05')&(combined_df['time_index']=='3x8')].index].index, axis='index', inplace = True)
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='111')&(combined_df['track_number']=='05')&(combined_df['time_index']=='1:15')].index,'combo_clean'] = '111-05-COMBINATION 1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='111')&(combined_df['track_number']=='05')&(combined_df['time_index']=='1:15')].index,'move1'] = 'TRICEP DIP'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='111')&(combined_df['track_number']=='05')&(combined_df['time_index']=='1:15')].index,'tempo1'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='111')&(combined_df['track_number']=='05')&(combined_df['time_index']=='1:15')].index,'reps1'] = '2x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='111')&(combined_df['track_number']=='05')&(combined_df['time_index']=='1:15')].index,'move2'] = 'PULSE TRICEP DIP'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='111')&(combined_df['track_number']=='05')&(combined_df['time_index']=='1:15')].index,'reps2'] = '3x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='111')&(combined_df['track_number']=='05')&(combined_df['time_index']=='2:49')].index,'move2'] = 'PULSE KNEELING O/H TRICEP EXTENSION'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='111')&(combined_df['track_number']=='05')&(combined_df['time_index']=='3:49')].index,'tempo1'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='111')&(combined_df['track_number']=='05')&(combined_df['time_index']=='3:49')].index,'reps1'] = '2x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='111')&(combined_df['track_number']=='05')&(combined_df['time_index']=='3:49')].index,'move2'] = 'PULSE TRICEP DIP'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='111')&(combined_df['track_number']=='05')&(combined_df['time_index']=='3:49')].index,'reps2'] = '3x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='111')&(combined_df['track_number']=='05')&(combined_df['time_index']=='4:58')].index,'move1'] = 'TRICEP KICKBACK'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='111')&(combined_df['track_number']=='05')&(combined_df['time_index']=='2:49')].index,'move2'] = 'PULSE STANDING O/H TRICEP EXTENSION'

combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='111')&(combined_df['track_number']=='06')&(combined_df['time_index']=='0:50')].index,'no_rep'] = 'SET POSITION'
to_append_111_06 = combined_df[(combined_df['bodypump_release_number']=='111')&(combined_df['track_number']=='06')&(combined_df['time_index']=='0:50')]
to_append_111_06.loc[:,'block']='3'
to_append_111_06.loc[:,'time_index']='2:27'
to_append_111_06.loc[:,'block_clean']='111-06-3'
combined_df = pd.concat([combined_df, to_append_111_06], ignore_index= True)
to_append_111_06_bis = combined_df[(combined_df['bodypump_release_number']=='111')&(combined_df['track_number']=='06')&(combined_df['time_index']=='0:50')]
to_append_111_06_bis.loc[:,'block']='5'
to_append_111_06_bis.loc[:,'time_index']='4:07'
to_append_111_06_bis.loc[:,'block_clean']='111-06-5'
combined_df = pd.concat([combined_df, to_append_111_06_bis], ignore_index= True)
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='111')&(combined_df['track_number']=='06')&(combined_df['time_index']=='2:27')].index,'reps1'] = '18x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='111')&(combined_df['track_number']=='06')&(combined_df['time_index']=='4:07')].index,'reps1'] = '18x'
combined_df = combined_df.sort_values(by=['bodypump_release_number','track_number','block_clean','time_index'], ascending=[True,True,True,True]).reset_index(drop=True)

combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='111')&(combined_df['track_number']=='07')&(combined_df['time_index']=='4x8')].index,'time_index'] = '2:17'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='111')&(combined_df['track_number']=='07')&(combined_df['time_index']=='2:17')].index,'move1'] = '3x PULSE BENCH BACKWARD-STEPPING LUNGE'

combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='111')&(combined_df['track_number']=='08')&(combined_df['time_index']=='0:33')].index,'move1'] = 'STANDING SIDE RAISE'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='111')&(combined_df['track_number']=='08')&(combined_df['time_index']=='0:33')].index,'tempo1'] = '2/2'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='111')&(combined_df['track_number']=='08')&(combined_df['time_index']=='0:33')].index,'reps1'] = '4x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='111')&(combined_df['track_number']=='08')&(combined_df['time_index']=='0:59')].index,'reps1'] = '8x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='111')&(combined_df['track_number']=='08')&(combined_df['time_index']=='1:45')].index,'move1'] = 'UPRIGHT ROW'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='111')&(combined_df['track_number']=='08')&(combined_df['time_index']=='2:12')].index,'no_rep'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='111')&(combined_df['track_number']=='08')&(combined_df['time_index']=='2:12')].index,'move1'] = 'STANDING O/H SHOULDER PRESS'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='111')&(combined_df['track_number']=='08')&(combined_df['time_index']=='3:51')].index,'move1'] = 'UPRIGHT ROW'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='111')&(combined_df['track_number']=='08')&(combined_df['time_index']=='4:18')].index,'no_rep'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='111')&(combined_df['track_number']=='08')&(combined_df['time_index']=='4:18')].index,'move1'] = 'STANDING O/H SHOULDER PRESS'

combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='111')&(combined_df['track_number']=='08')&(combined_df['time_index']=='0:59')].index,'move1'] = 'HIP BRIDGE'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='111')&(combined_df['track_number']=='08')&(combined_df['time_index']=='0:59')].index,'tempo1'] = '3/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='111')&(combined_df['track_number']=='08')&(combined_df['time_index']=='0:59')].index,'reps1'] = '4x'

# 116
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='116')&(combined_df['track_number']=='01')&(combined_df['time_index']=='2:12')].index,'move1'] = 'CLEAN & PRESS'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='116')&(combined_df['track_number']=='01')&(combined_df['time_index']=='3:58')].index,'move1'] = 'PLATE CLEAN & PRESS'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='116')&(combined_df['track_number']=='04')&(combined_df['time_index']=='2:53')].index,'move1'] ='TRIPLE PULSE KNEELING O/H TRICEP EXTENSION'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='116')&(combined_df['track_number']=='04')&(combined_df['time_index']=='4:49')].index,'move1'] ='TRIPLE PULSE KNEELING O/H TRICEP EXTENSION'

# 117
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='117')&(combined_df['track_number']=='03')&(combined_df['time_index']=='0:22')].index,'move1'] = 'BENCH CHEST PRESS'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='117')&(combined_df['track_number']=='03')&(combined_df['time_index']=='0:33')].index,'move1'] = 'BENCH CHEST PRESS'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='117')&(combined_df['track_number']=='03')&(combined_df['time_index']=='0:47')].index,'move1'] = 'BENCH CHEST PRESS'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='117')&(combined_df['track_number']=='03')&(combined_df['time_index']=='1:07')].index,'move1'] = 'BENCH CHEST PRESS'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='117')&(combined_df['track_number']=='03')&(combined_df['time_index']=='2:22')].index,'move1'] = 'BENCH CHEST PRESS'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='117')&(combined_df['track_number']=='03')&(combined_df['time_index']=='3:24')].index,'move1'] = 'BENCH MID-GRIP PUSHUP'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='117')&(combined_df['track_number']=='03')&(combined_df['time_index']=='3:39')].index,'move1'] = 'BENCH MID-GRIP PUSHUP'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='117')&(combined_df['track_number']=='03')&(combined_df['time_index']=='4:18')].index,'move1'] = 'BENCH WIDE-GRIP PUSHUP'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='117')&(combined_df['track_number']=='03')&(combined_df['time_index']=='4:33')].index,'move1'] = 'BENCH WIDE-GRIP PUSHUP'

combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='117')&(combined_df['track_number']=='04')&(combined_df['time_index']=='1:19')].index,'reps1'] = '2x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='117')&(combined_df['track_number']=='04')&(combined_df['time_index']=='1:19')].index,'reps2'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='117')&(combined_df['track_number']=='04')&(combined_df['time_index']=='2:59')].index,'reps1'] = '2x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='117')&(combined_df['track_number']=='04')&(combined_df['time_index']=='2:59')].index,'reps2'] = '1x'

combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='117')&(combined_df['track_number']=='05')&(combined_df['time_index']=='2:23')].index,'move1'] = 'TRICEP BENCH PUSHUP'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='117')&(combined_df['track_number']=='05')&(combined_df['time_index']=='2:23')].index,'tempo1'] = '4/4'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='117')&(combined_df['track_number']=='05')&(combined_df['time_index']=='4:08')].index,'move1'] = 'TRICEP BENCH PUSHUP'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='117')&(combined_df['track_number']=='05')&(combined_df['time_index']=='4:08')].index,'tempo1'] = '4/4'

combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='117')&(combined_df['track_number']=='06')&(combined_df['time_index']=='0:32')].index,'move1'] = 'TRIPLE MID-RANGE PULSE'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='117')&(combined_df['track_number']=='06')&(combined_df['time_index']=='0:32')].index,'reps1'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='117')&(combined_df['track_number']=='06')&(combined_df['time_index']=='0:32')].index,'move2'] = 'BICEP PLATE CURL'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='117')&(combined_df['track_number']=='06')&(combined_df['time_index']=='0:32')].index,'tempo2'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='117')&(combined_df['track_number']=='06')&(combined_df['time_index']=='0:32')].index,'reps2'] = '2x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='117')&(combined_df['track_number']=='06')&(combined_df['time_index']=='1:34')].index,'move1'] = 'TRIPLE MID-RANGE PULSE'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='117')&(combined_df['track_number']=='06')&(combined_df['time_index']=='1:34')].index,'reps1'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='117')&(combined_df['track_number']=='06')&(combined_df['time_index']=='1:34')].index,'move2'] = 'BICEP PLATE CURL'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='117')&(combined_df['track_number']=='06')&(combined_df['time_index']=='1:34')].index,'tempo2'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='117')&(combined_df['track_number']=='06')&(combined_df['time_index']=='1:34')].index,'reps2'] = '2x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='117')&(combined_df['track_number']=='06')&(combined_df['time_index']=='2:28')].index,'move1'] = 'TRIPLE MID-RANGE PULSE'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='117')&(combined_df['track_number']=='06')&(combined_df['time_index']=='2:28')].index,'reps1'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='117')&(combined_df['track_number']=='06')&(combined_df['time_index']=='2:28')].index,'move2'] = 'BICEP PLATE ROW'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='117')&(combined_df['track_number']=='06')&(combined_df['time_index']=='2:28')].index,'tempo2'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='117')&(combined_df['track_number']=='06')&(combined_df['time_index']=='2:28')].index,'reps2'] = '1x'

# 118
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='118')&(combined_df['track_number']=='02')&(combined_df['time_index']=='1:12')].index,'reps1'] = '8x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='118')&(combined_df['track_number']=='02')&(combined_df['time_index']=='1:34')].index,'reps1'] = '8x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='118')&(combined_df['track_number']=='02')&(combined_df['time_index']=='2:41')].index,'reps1'] = '8x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='118')&(combined_df['track_number']=='02')&(combined_df['time_index']=='4:13')].index,'reps1'] = '8x'


combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='118')&(combined_df['track_number']=='04')&(combined_df['time_index']=='2:54')].index,'combo_clean'] = '118-04-COMBINATION 1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='118')&(combined_df['track_number']=='04')&(combined_df['time_index']=='2:54')].index,'move1'] = 'CLEAN & PRESS'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='118')&(combined_df['track_number']=='04')&(combined_df['time_index']=='2:54')].index,'tempo1'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='118')&(combined_df['track_number']=='04')&(combined_df['time_index']=='2:54')].index,'reps1'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='118')&(combined_df['track_number']=='04')&(combined_df['time_index']=='2:54')].index,'move2'] = 'DEADROW'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='118')&(combined_df['track_number']=='04')&(combined_df['time_index']=='2:54')].index,'tempo2'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='118')&(combined_df['track_number']=='04')&(combined_df['time_index']=='2:54')].index,'reps2'] = '1x'

combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='118')&(combined_df['track_number']=='05')&(combined_df['time_index']=='0:50')].index,'reps1'] = '2x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='118')&(combined_df['track_number']=='05')&(combined_df['time_index']=='0:50')].index,'reps2'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='118')&(combined_df['track_number']=='05')&(combined_df['time_index']=='1:49')].index,'reps1'] = '2x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='118')&(combined_df['track_number']=='05')&(combined_df['time_index']=='1:49')].index,'reps2'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='118')&(combined_df['track_number']=='05')&(combined_df['time_index']=='2:31')].index,'reps1'] = '2x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='118')&(combined_df['track_number']=='05')&(combined_df['time_index']=='2:31')].index,'reps2'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='118')&(combined_df['track_number']=='05')&(combined_df['time_index']=='3:53')].index,'reps1'] = '2x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='118')&(combined_df['track_number']=='05')&(combined_df['time_index']=='3:53')].index,'reps2'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='118')&(combined_df['track_number']=='05')&(combined_df['time_index']=='4:35')].index,'reps1'] = '2x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='118')&(combined_df['track_number']=='05')&(combined_df['time_index']=='4:35')].index,'reps2'] = '1x'

combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='118')&(combined_df['track_number']=='06')&(combined_df['time_index']=='0:29')].index,'move1'] = 'ALT SINGLE ARM BICEP PLATE CURL'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='118')&(combined_df['track_number']=='06')&(combined_df['time_index']=='0:29')].index,'reps1'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='118')&(combined_df['track_number']=='06')&(combined_df['time_index']=='0:29')].index,'move2'] = 'BICEP PLATE CURL'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='118')&(combined_df['track_number']=='06')&(combined_df['time_index']=='0:29')].index,'tempo2'] = '2/2'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='118')&(combined_df['track_number']=='06')&(combined_df['time_index']=='0:29')].index,'reps2'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='118')&(combined_df['track_number']=='06')&(combined_df['time_index']=='0:52')].index,'move1'] = 'ALT SINGLE ARM BICEP PLATE CURL'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='118')&(combined_df['track_number']=='06')&(combined_df['time_index']=='0:52')].index,'reps1'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='118')&(combined_df['track_number']=='06')&(combined_df['time_index']=='0:52')].index,'move2'] = 'BICEP PLATE ROW'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='118')&(combined_df['track_number']=='06')&(combined_df['time_index']=='0:52')].index,'tempo2'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='118')&(combined_df['track_number']=='06')&(combined_df['time_index']=='0:52')].index,'reps2'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='118')&(combined_df['track_number']=='06')&(combined_df['time_index']=='1:27')].index,'move1'] = 'BARBELL BICEP CURL'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='118')&(combined_df['track_number']=='06')&(combined_df['time_index']=='1:27')].index,'tempo1'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='118')&(combined_df['track_number']=='06')&(combined_df['time_index']=='1:27')].index,'reps1'] = '2x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='118')&(combined_df['track_number']=='06')&(combined_df['time_index']=='1:27')].index,'move2'] = 'BARBELL BICEP CURL'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='118')&(combined_df['track_number']=='06')&(combined_df['time_index']=='1:27')].index,'tempo2'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='118')&(combined_df['track_number']=='06')&(combined_df['time_index']=='1:27')].index,'reps2'] = '2x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='118')&(combined_df['track_number']=='06')&(combined_df['time_index']=='1:51')].index,'move1'] = 'BARBELL BICEP CURL'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='118')&(combined_df['track_number']=='06')&(combined_df['time_index']=='1:51')].index,'tempo1'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='118')&(combined_df['track_number']=='06')&(combined_df['time_index']=='1:51')].index,'reps1'] = '2x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='118')&(combined_df['track_number']=='06')&(combined_df['time_index']=='1:51')].index,'move2'] = 'TRIPLE MID-RANGE PULSE'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='118')&(combined_df['track_number']=='06')&(combined_df['time_index']=='1:51')].index,'reps2'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='118')&(combined_df['track_number']=='06')&(combined_df['time_index']=='2:39')].index,'move1'] = 'ALT SINGLE ARM BICEP PLATE CURL'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='118')&(combined_df['track_number']=='06')&(combined_df['time_index']=='2:39')].index,'reps1'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='118')&(combined_df['track_number']=='06')&(combined_df['time_index']=='2:39')].index,'move2'] = 'BICEP PLATE CURL'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='118')&(combined_df['track_number']=='06')&(combined_df['time_index']=='2:39')].index,'tempo2'] = '2/2'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='118')&(combined_df['track_number']=='06')&(combined_df['time_index']=='2:39')].index,'reps2'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='118')&(combined_df['track_number']=='06')&(combined_df['time_index']=='3:02')].index,'move1'] = 'ALT SINGLE ARM BICEP PLATE CURL'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='118')&(combined_df['track_number']=='06')&(combined_df['time_index']=='3:02')].index,'reps1'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='118')&(combined_df['track_number']=='06')&(combined_df['time_index']=='3:02')].index,'move2'] = 'BICEP PLATE ROW'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='118')&(combined_df['track_number']=='06')&(combined_df['time_index']=='3:02')].index,'tempo2'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='118')&(combined_df['track_number']=='06')&(combined_df['time_index']=='3:02')].index,'reps2'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='118')&(combined_df['track_number']=='06')&(combined_df['time_index']=='3:37')].index,'move1'] = 'BARBELL BICEP CURL'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='118')&(combined_df['track_number']=='06')&(combined_df['time_index']=='3:37')].index,'tempo1'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='118')&(combined_df['track_number']=='06')&(combined_df['time_index']=='3:37')].index,'reps1'] = '2x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='118')&(combined_df['track_number']=='06')&(combined_df['time_index']=='3:37')].index,'move2'] = 'BARBELL BICEP CURL'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='118')&(combined_df['track_number']=='06')&(combined_df['time_index']=='3:37')].index,'tempo2'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='118')&(combined_df['track_number']=='06')&(combined_df['time_index']=='3:37')].index,'reps2'] = '2x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='118')&(combined_df['track_number']=='06')&(combined_df['time_index']=='4:00')].index,'move1'] = 'BARBELL BICEP CURL'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='118')&(combined_df['track_number']=='06')&(combined_df['time_index']=='4:00')].index,'tempo1'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='118')&(combined_df['track_number']=='06')&(combined_df['time_index']=='4:00')].index,'reps1'] = '2x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='118')&(combined_df['track_number']=='06')&(combined_df['time_index']=='4:00')].index,'move2'] = 'TRIPLE MID-RANGE PULSE'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='118')&(combined_df['track_number']=='06')&(combined_df['time_index']=='4:00')].index,'reps2'] = '1x'

combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='118')&(combined_df['track_number']=='07')&(combined_df['time_index']=='1:06')].index,'tempo1'] = '1/3'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='118')&(combined_df['track_number']=='07')&(combined_df['time_index']=='2:15')].index,'tempo1'] = '1/3'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='118')&(combined_df['track_number']=='07')&(combined_df['time_index']=='3:36')].index,'tempo1'] = '1/3'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='118')&(combined_df['track_number']=='07')&(combined_df['time_index']=='4:44')].index,'tempo1'] = '1/3'

# 119
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='119')&(combined_df['track_number']=='03')&(combined_df['time_index']=='2:11')].index,'move1'] = 'CHEST PRESS'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='119')&(combined_df['track_number']=='03')&(combined_df['time_index']=='2:30')].index,'move1'] = 'CHEST PRESS'

combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='119')&(combined_df['track_number']=='05')&(combined_df['time_index']=='1:03')].index,'reps1'] = '2x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='119')&(combined_df['track_number']=='05')&(combined_df['time_index']=='1:03')].index,'reps2'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='119')&(combined_df['track_number']=='05')&(combined_df['time_index']=='1:03')].index,'tempo2'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='119')&(combined_df['track_number']=='05')&(combined_df['time_index']=='1:03')].index,'move3'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='119')&(combined_df['track_number']=='05')&(combined_df['time_index']=='1:03')].index,'reps3'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='119')&(combined_df['track_number']=='05')&(combined_df['time_index']=='1:03')].index,'move4'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='119')&(combined_df['track_number']=='05')&(combined_df['time_index']=='1:03')].index,'reps4'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='119')&(combined_df['track_number']=='05')&(combined_df['time_index']=='2:42')].index,'tempo2'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='119')&(combined_df['track_number']=='05')&(combined_df['time_index']=='2:42')].index,'move3'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='119')&(combined_df['track_number']=='05')&(combined_df['time_index']=='2:42')].index,'reps3'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='119')&(combined_df['track_number']=='05')&(combined_df['time_index']=='2:42')].index,'move4'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='119')&(combined_df['track_number']=='05')&(combined_df['time_index']=='2:42')].index,'reps4'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='119')&(combined_df['track_number']=='05')&(combined_df['time_index']=='4:32')].index,'tempo2'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='119')&(combined_df['track_number']=='05')&(combined_df['time_index']=='4:32')].index,'move3'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='119')&(combined_df['track_number']=='05')&(combined_df['time_index']=='4:32')].index,'reps3'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='119')&(combined_df['track_number']=='05')&(combined_df['time_index']=='4:32')].index,'move4'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='119')&(combined_df['track_number']=='05')&(combined_df['time_index']=='4:32')].index,'reps4'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='119')&(combined_df['track_number']=='05')&(combined_df['time_index']=='4:32')].index,'move2'] = 'BOTTOM HALF PULSE TRICEP PRESS'

combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='119')&(combined_df['track_number']=='06')&(combined_df['time_index']=='0:52')].index,'reps'] = '4x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='119')&(combined_df['track_number']=='06')&(combined_df['time_index']=='0:52')].index,'move1'] = 'FAST ALT SINGLE-ARM BICEP PLATE CURL'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='119')&(combined_df['track_number']=='06')&(combined_df['time_index']=='0:52')].index,'reps1'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='119')&(combined_df['track_number']=='06')&(combined_df['time_index']=='0:52')].index,'move2'] = 'MID-RANGE PULSE BICEP PLATE CURL'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='119')&(combined_df['track_number']=='06')&(combined_df['time_index']=='0:52')].index,'reps2'] = '2x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='119')&(combined_df['track_number']=='06')&(combined_df['time_index']=='1:48')].index,'move1'] = 'FAST ALT SINGLE-ARM BICEP PLATE CURL'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='119')&(combined_df['track_number']=='06')&(combined_df['time_index']=='1:48')].index,'reps1'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='119')&(combined_df['track_number']=='06')&(combined_df['time_index']=='1:48')].index,'move2'] = 'MID-RANGE PULSE BICEP PLATE CURL'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='119')&(combined_df['track_number']=='06')&(combined_df['time_index']=='1:48')].index,'reps2'] = '2x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='119')&(combined_df['track_number']=='06')&(combined_df['time_index']=='2:26')].index,'move1'] = 'FAST ALT SINGLE-ARM BICEP PLATE CURL'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='119')&(combined_df['track_number']=='06')&(combined_df['time_index']=='2:26')].index,'reps1'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='119')&(combined_df['track_number']=='06')&(combined_df['time_index']=='2:26')].index,'move2'] = 'MID-RANGE PULSE BICEP PLATE CURL'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='119')&(combined_df['track_number']=='06')&(combined_df['time_index']=='2:26')].index,'reps2'] = '2x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='119')&(combined_df['track_number']=='06')&(combined_df['time_index']=='3:12')].index,'move1'] = 'FAST ALT SINGLE-ARM BICEP PLATE CURL'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='119')&(combined_df['track_number']=='06')&(combined_df['time_index']=='3:12')].index,'reps1'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='119')&(combined_df['track_number']=='06')&(combined_df['time_index']=='3:12')].index,'move2'] = 'MID-RANGE PULSE BICEP PLATE CURL'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='119')&(combined_df['track_number']=='06')&(combined_df['time_index']=='3:12')].index,'reps2'] = '2x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='119')&(combined_df['track_number']=='06')&(combined_df['time_index']=='2:35')].index,'reps'] = '4x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='119')&(combined_df['track_number']=='06')&(combined_df['time_index']=='2:35')].index,'move1'] = 'BICEP PLATE ROW'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='119')&(combined_df['track_number']=='06')&(combined_df['time_index']=='2:35')].index,'tempo1'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='119')&(combined_df['track_number']=='06')&(combined_df['time_index']=='2:35')].index,'reps1'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='119')&(combined_df['track_number']=='06')&(combined_df['time_index']=='2:35')].index,'move2'] = 'BICEP PLATE CURL'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='119')&(combined_df['track_number']=='06')&(combined_df['time_index']=='2:35')].index,'tempo2'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='119')&(combined_df['track_number']=='06')&(combined_df['time_index']=='2:35')].index,'reps2'] = '2x'

to_append_119_07 = combined_df[(combined_df['bodypump_release_number']=='119')&(combined_df['track_number']=='07')&(combined_df['block_clean']=='119-07-3')]
to_append_119_07.loc[:,'block']='5'
to_append_119_07.loc[:,'block_clean']='119-07-5'
combined_df = pd.concat([combined_df, to_append_119_07], ignore_index= True)
combined_df = combined_df.sort_values(by=['bodypump_release_number','track_number','block_clean','time_index'], ascending=[True,True,True,True]).reset_index(drop=True)
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='119')&(combined_df['track_number']=='07')&(combined_df['time_index']=='3:01')].index,'move1'] = 'PULSE SQUAT'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='119')&(combined_df['track_number']=='07')&(combined_df['time_index']=='1:02')].index,'reps1'] = '8x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='119')&(combined_df['track_number']=='07')&(combined_df['time_index']=='2:01')].index,'reps1'] = '8x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='119')&(combined_df['track_number']=='07')&(combined_df['time_index']=='15')].index,'move1'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='119')&(combined_df['track_number']=='07')&(combined_df['time_index']=='15')].index,'reps1'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='119')&(combined_df['track_number']=='07')&(combined_df['time_index']=='15')].index,'time_index'] = '3:15'

combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='119')&(combined_df['track_number']=='08')&(combined_df['time_index']=='4:40')].index,'reps1'] = '14x'

combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='119')&(combined_df['track_number']=='09')&(combined_df['time_index']=='0:21')].index,'reps'] = '4x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='119')&(combined_df['track_number']=='09')&(combined_df['time_index']=='0:21')].index,'move1'] = 'C-CRUNCH / SINGLE LEG EXTENSION'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='119')&(combined_df['track_number']=='09')&(combined_df['time_index']=='0:21')].index,'tempo1'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='119')&(combined_df['track_number']=='09')&(combined_df['time_index']=='0:21')].index,'reps1'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='119')&(combined_df['track_number']=='09')&(combined_df['time_index']=='0:21')].index,'move2'] = 'C-CRUNCH / SINGLE LEG EXTENSION'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='119')&(combined_df['track_number']=='09')&(combined_df['time_index']=='0:21')].index,'tempo2'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='119')&(combined_df['track_number']=='09')&(combined_df['time_index']=='0:21')].index,'reps2'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='119')&(combined_df['track_number']=='09')&(combined_df['time_index']=='1:22')].index,'reps'] = '4x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='119')&(combined_df['track_number']=='09')&(combined_df['time_index']=='1:22')].index,'move1'] = 'C-CRUNCH / SINGLE LEG EXTENSION'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='119')&(combined_df['track_number']=='09')&(combined_df['time_index']=='1:22')].index,'tempo1'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='119')&(combined_df['track_number']=='09')&(combined_df['time_index']=='1:22')].index,'reps1'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='119')&(combined_df['track_number']=='09')&(combined_df['time_index']=='1:22')].index,'move2'] = 'C-CRUNCH / SINGLE LEG EXTENSION'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='119')&(combined_df['track_number']=='09')&(combined_df['time_index']=='1:22')].index,'tempo2'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='119')&(combined_df['track_number']=='09')&(combined_df['time_index']=='1:22')].index,'reps2'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='119')&(combined_df['track_number']=='09')&(combined_df['time_index']=='2:55')].index,'move1'] = 'SPRINTER'

# 120
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='120')&(combined_df['track_number']=='01')&(combined_df['time_index']=='1:36')].index,'move1'] = 'CLEAN & PRESS'

combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='120')&(combined_df['track_number']=='02')&(combined_df['time_index']=='1:22')].index,'tempo1'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='120')&(combined_df['track_number']=='02')&(combined_df['time_index']=='1:22')].index,'tempo2'] = '1/3'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='120')&(combined_df['track_number']=='02')&(combined_df['time_index']=='1:44')].index,'reps1'] = '8x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='120')&(combined_df['track_number']=='02')&(combined_df['time_index']=='3:23')].index,'tempo1'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='120')&(combined_df['track_number']=='02')&(combined_df['time_index']=='3:23')].index,'tempo2'] = '1/3'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='120')&(combined_df['track_number']=='02')&(combined_df['time_index']=='3:45')].index,'reps1'] = '8x'

combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='120')&(combined_df['track_number']=='03')&(combined_df['time_index']=='0:32')].index,'reps1'] = '8x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='120')&(combined_df['track_number']=='03')&(combined_df['time_index']=='0:32')].index,'tempo1'] = '3/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='120')&(combined_df['track_number']=='03')&(combined_df['time_index']=='3:17')].index,'combo_clean'] = '120-03-COMBINATION 2'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='120')&(combined_df['track_number']=='03')&(combined_df['time_index']=='3:17')].index,'move1'] = 'CHEST PRESS'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='120')&(combined_df['track_number']=='03')&(combined_df['time_index']=='3:17')].index,'move2'] = 'CHEST PRESS'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='120')&(combined_df['track_number']=='03')&(combined_df['time_index']=='4:18')].index,'combo_clean'] = '120-03-COMBINATION 2'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='120')&(combined_df['track_number']=='03')&(combined_df['time_index']=='4:18')].index,'move1'] = 'CHEST PRESS'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='120')&(combined_df['track_number']=='03')&(combined_df['time_index']=='4:18')].index,'move2'] = 'CHEST PRESS'

combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='120')&(combined_df['track_number']=='05')&(combined_df['time_index']=='0:50')].index,'move2'] = 'TRIPLE PULSE TRICEP PRESS'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='120')&(combined_df['track_number']=='05')&(combined_df['time_index']=='1:50')].index,'move2'] = 'TRIPLE PULSE TRICEP PRESS'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='120')&(combined_df['track_number']=='05')&(combined_df['time_index']=='2:35')].index,'combo_clean'] = '120-05-COMBINATION 2'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='120')&(combined_df['track_number']=='05')&(combined_df['time_index']=='2:35')].index,'move1'] = 'TRICEP BENCH KICKBACK'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='120')&(combined_df['track_number']=='05')&(combined_df['time_index']=='2:35')].index,'move2'] = 'TRIPLE PULSE TRICEP BENCH KICKBACK'

combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='120')&(combined_df['track_number']=='06')&(combined_df['time_index']=='0:53')].index,'reps'] = '4x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='120')&(combined_df['track_number']=='06')&(combined_df['time_index']=='0:53')].index,'tempo1'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='120')&(combined_df['track_number']=='06')&(combined_df['time_index']=='0:53')].index,'reps1'] = '2x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='120')&(combined_df['track_number']=='06')&(combined_df['time_index']=='0:53')].index,'move2'] = 'PULSE MID-RANGE CURL'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='120')&(combined_df['track_number']=='06')&(combined_df['time_index']=='0:53')].index,'reps2'] = '2x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='120')&(combined_df['track_number']=='06')&(combined_df['time_index']=='1:46')].index,'reps'] = '4x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='120')&(combined_df['track_number']=='06')&(combined_df['time_index']=='1:46')].index,'tempo1'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='120')&(combined_df['track_number']=='06')&(combined_df['time_index']=='1:46')].index,'reps1'] = '2x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='120')&(combined_df['track_number']=='06')&(combined_df['time_index']=='1:46')].index,'move2'] = 'PULSE MID-RANGE CURL'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='120')&(combined_df['track_number']=='06')&(combined_df['time_index']=='1:46')].index,'reps2'] = '2x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='120')&(combined_df['track_number']=='06')&(combined_df['time_index']=='2:40')].index,'reps'] = '4x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='120')&(combined_df['track_number']=='06')&(combined_df['time_index']=='2:40')].index,'tempo1'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='120')&(combined_df['track_number']=='06')&(combined_df['time_index']=='2:40')].index,'reps1'] = '2x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='120')&(combined_df['track_number']=='06')&(combined_df['time_index']=='2:40')].index,'move2'] = 'PULSE MID-RANGE CURL'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='120')&(combined_df['track_number']=='06')&(combined_df['time_index']=='2:40')].index,'reps2'] = '2x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='120')&(combined_df['track_number']=='06')&(combined_df['time_index']=='3:37')].index,'reps'] = '4x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='120')&(combined_df['track_number']=='06')&(combined_df['time_index']=='3:37')].index,'tempo1'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='120')&(combined_df['track_number']=='06')&(combined_df['time_index']=='3:37')].index,'reps1'] = '2x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='120')&(combined_df['track_number']=='06')&(combined_df['time_index']=='3:37')].index,'move2'] = 'PULSE MID-RANGE CURL'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='120')&(combined_df['track_number']=='06')&(combined_df['time_index']=='3:37')].index,'reps2'] = '2x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='120')&(combined_df['track_number']=='06')&(combined_df['time_index']=='4:30')].index,'reps'] = '4x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='120')&(combined_df['track_number']=='06')&(combined_df['time_index']=='4:30')].index,'tempo1'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='120')&(combined_df['track_number']=='06')&(combined_df['time_index']=='4:30')].index,'reps1'] = '2x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='120')&(combined_df['track_number']=='06')&(combined_df['time_index']=='4:30')].index,'move2'] = 'PULSE MID-RANGE CURL'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='120')&(combined_df['track_number']=='06')&(combined_df['time_index']=='4:30')].index,'reps2'] = '2x'

combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='120')&(combined_df['track_number']=='07')&(combined_df['time_index']=='0:35')].index,'move2'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='120')&(combined_df['track_number']=='07')&(combined_df['time_index']=='1:05')].index,'combo_clean'] = '120-07-COMBINATION 2'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='120')&(combined_df['track_number']=='07')&(combined_df['time_index']=='1:05')].index,'move1'] = 'TRIPLE PULSE LUNGE'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='120')&(combined_df['track_number']=='07')&(combined_df['time_index']=='1:05')].index,'reps1'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='120')&(combined_df['track_number']=='07')&(combined_df['time_index']=='1:05')].index,'tempo1'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='120')&(combined_df['track_number']=='07')&(combined_df['time_index']=='1:05')].index,'move2'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='120')&(combined_df['track_number']=='07')&(combined_df['time_index']=='1:50')].index,'move2'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='120')&(combined_df['track_number']=='07')&(combined_df['time_index']=='2:20')].index,'combo_clean'] = '120-07-COMBINATION 2'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='120')&(combined_df['track_number']=='07')&(combined_df['time_index']=='2:20')].index,'move1'] = 'TRIPLE PULSE LUNGE'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='120')&(combined_df['track_number']=='07')&(combined_df['time_index']=='2:20')].index,'reps1'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='120')&(combined_df['track_number']=='07')&(combined_df['time_index']=='2:20')].index,'tempo1'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='120')&(combined_df['track_number']=='07')&(combined_df['time_index']=='2:20')].index,'move2'] = ''
to_append_120_07 = combined_df[(combined_df['bodypump_release_number']=='120')&(combined_df['track_number']=='07')&(combined_df['block_clean']=='120-07-1')]
to_append_120_07.loc[:,'block']='2'
to_append_120_07.loc[:,'block_clean']='120-07-2'
combined_df = pd.concat([combined_df, to_append_120_07], ignore_index= True)
combined_df = combined_df.sort_values(by=['bodypump_release_number','track_number','block_clean','time_index'], ascending=[True,True,True,True]).reset_index(drop=True)

combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='120')&(combined_df['track_number']=='08')&(combined_df['time_index']=='1:48')].index,'move1'] = 'TIPPING REAR DELTOID FLY WITH TWIST'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='120')&(combined_df['track_number']=='08')&(combined_df['time_index']=='1:48')].index,'tempo1'] = '2/2'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='120')&(combined_df['track_number']=='08')&(combined_df['time_index']=='1:48')].index,'reps1'] = '8x'

combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='120')&(combined_df['track_number']=='09')&(combined_df['time_index']=='3:41')].index,'move3'] = ''

# 123
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='123')&(combined_df['track_number']=='01')&(combined_df['time_index']=='2:12')].index,'move1'] = 'CLEAN & PRESS'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='123')&(combined_df['track_number']=='01')&(combined_df['time_index']=='3:30')].index,'reps'] = '3x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='123')&(combined_df['track_number']=='01')&(combined_df['time_index']=='3:30')].index,'reps1'] = '3x'

combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='123')&(combined_df['track_number']=='04')&(combined_df['time_index']=='1:38')].index,'move3'] = ''

combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='123')&(combined_df['track_number']=='05')&(combined_df['time_index']=='0:46')].index,'move1'] = 'PULSE STANDING PLATE O/H TRICEP EXTENSION'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='123')&(combined_df['track_number']=='05')&(combined_df['time_index']=='1:41')].index,'move1'] = 'TRIPLE KNEELING PLATE TRICEP KICKBACK ROW'

combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='123')&(combined_df['track_number']=='06')&(combined_df['time_index']=='0:45')].index,'reps'] = '4x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='123')&(combined_df['track_number']=='06')&(combined_df['time_index']=='0:45')].index,'reps1'] = '4x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='123')&(combined_df['track_number']=='06')&(combined_df['time_index']=='0:45')].index,'tempo1'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='123')&(combined_df['track_number']=='06')&(combined_df['time_index']=='1:32')].index,'tempo1'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='123')&(combined_df['track_number']=='06')&(combined_df['time_index']=='1:59')].index,'tempo1'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='123')&(combined_df['track_number']=='06')&(combined_df['time_index']=='2:39')].index,'reps'] = '4x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='123')&(combined_df['track_number']=='06')&(combined_df['time_index']=='2:39')].index,'move1'] = 'TRIPLE PREACHER CURL'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='123')&(combined_df['track_number']=='06')&(combined_df['time_index']=='2:39')].index,'tempo1'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='123')&(combined_df['track_number']=='06')&(combined_df['time_index']=='2:39')].index,'reps1'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='123')&(combined_df['track_number']=='06')&(combined_df['time_index']=='2:39')].index,'move1'] = 'BICEP CURL'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='123')&(combined_df['track_number']=='06')&(combined_df['time_index']=='2:39')].index,'tempo1'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='123')&(combined_df['track_number']=='06')&(combined_df['time_index']=='2:39')].index,'reps1'] = '4x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='123')&(combined_df['track_number']=='06')&(combined_df['time_index']=='4:01')].index,'reps'] = '4x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='123')&(combined_df['track_number']=='06')&(combined_df['time_index']=='4:01')].index,'move1'] = 'TRIPLE PREACHER CURL'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='123')&(combined_df['track_number']=='06')&(combined_df['time_index']=='4:01')].index,'tempo1'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='123')&(combined_df['track_number']=='06')&(combined_df['time_index']=='4:01')].index,'reps1'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='123')&(combined_df['track_number']=='06')&(combined_df['time_index']=='4:01')].index,'move1'] = 'BICEP CURL'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='123')&(combined_df['track_number']=='06')&(combined_df['time_index']=='4:01')].index,'tempo1'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='123')&(combined_df['track_number']=='06')&(combined_df['time_index']=='4:01')].index,'reps1'] = '4x'

combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='123')&(combined_df['track_number']=='07')&(combined_df['time_index']=='32')].index,'time_index'] = '3:32'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='123')&(combined_df['track_number']=='07')&(combined_df['time_index']=='3:32')].index,'reps'] = '1x'
to_append_123_07 = combined_df[(combined_df['bodypump_release_number']=='123')&(combined_df['track_number']=='07')&(combined_df['block_clean']=='123-07-2')]
to_append_123_07.loc[:,'block']='3'
to_append_123_07.loc[:,'block_clean']='123-07-2'
combined_df = pd.concat([combined_df, to_append_123_07], ignore_index= True)
combined_df = combined_df.sort_values(by=['bodypump_release_number','track_number','block_clean','time_index'], ascending=[True,True,True,True]).reset_index(drop=True)

combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='123')&(combined_df['track_number']=='08')&(combined_df['time_index']=='0 :53')].index,'time_index'] = '0:53'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='123')&(combined_df['track_number']=='08')&(combined_df['time_index']=='2:41')].index,'move1'] = 'STANDING O/H SHOULDER PRESS'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='123')&(combined_df['track_number']=='08')&(combined_df['time_index']=='2:41')].index,'move2'] = 'STANDING O/H SHOULDER PRESS'

# 124
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='124')&(combined_df['track_number']=='01')&(combined_df['time_index']=='3:55')].index,'reps'] = '3x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='124')&(combined_df['track_number']=='01')&(combined_df['time_index']=='3:55')].index,'reps1'] = '3x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='124')&(combined_df['track_number']=='01')&(combined_df['time_index']=='5:02')].index,'no_rep'] = 'RECOVERY'

combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='124')&(combined_df['track_number']=='02')&(combined_df['time_index']=='1:00')].index,'reps3'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='124')&(combined_df['track_number']=='02')&(combined_df['time_index']=='1:00')].index,'move3'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='124')&(combined_df['track_number']=='02')&(combined_df['time_index']=='2:28')].index,'reps3'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='124')&(combined_df['track_number']=='02')&(combined_df['time_index']=='2:28')].index,'move3'] = ''

combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='124')&(combined_df['track_number']=='04')&(combined_df['time_index']=='2:54')].index,'move1'] = 'TRIPLE WIDE PLATE ROW'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='124')&(combined_df['track_number']=='04')&(combined_df['time_index']=='2:54')].index,'reps1'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='124')&(combined_df['track_number']=='04')&(combined_df['time_index']=='2:54')].index,'tempo1'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='124')&(combined_df['track_number']=='04')&(combined_df['time_index']=='2:54')].index,'move1'] = 'TRIPLE NARROW PLATE ROW'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='124')&(combined_df['track_number']=='04')&(combined_df['time_index']=='2:54')].index,'reps1'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='124')&(combined_df['track_number']=='04')&(combined_df['time_index']=='2:54')].index,'tempo1'] = ''

combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='124')&(combined_df['track_number']=='05')&(combined_df['time_index']=='1:31')].index,'move1'] = 'KNEELING TRICEP KICKBACK'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='124')&(combined_df['track_number']=='05')&(combined_df['time_index']=='1:31')].index,'move2'] = 'KNEELING TRICEP KICKBACK'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='124')&(combined_df['track_number']=='05')&(combined_df['time_index']=='2:20')].index,'move1'] = 'TRICEP PUSHUP'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='124')&(combined_df['track_number']=='05')&(combined_df['time_index']=='2:20')].index,'move2'] = 'TRICEP PUSHUP'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='124')&(combined_df['track_number']=='05')&(combined_df['time_index']=='3:35')].index,'move1'] = 'KNEELING TRICEP KICKBACK'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='124')&(combined_df['track_number']=='05')&(combined_df['time_index']=='3:35')].index,'move2'] = 'KNEELING TRICEP KICKBACK'

combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='124')&(combined_df['track_number']=='06')&(combined_df['time_index']=='0:24')].index,'reps'] = '8x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='124')&(combined_df['track_number']=='06')&(combined_df['time_index']=='0:24')].index,'reps1'] = '8x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='124')&(combined_df['track_number']=='06')&(combined_df['time_index']=='1:21')].index,'reps'] = '8x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='124')&(combined_df['track_number']=='06')&(combined_df['time_index']=='1:21')].index,'reps1'] = '8x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='124')&(combined_df['track_number']=='06')&(combined_df['time_index']=='2:00')].index,'reps'] = '8x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='124')&(combined_df['track_number']=='06')&(combined_df['time_index']=='2:00')].index,'reps1'] = '8x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='124')&(combined_df['track_number']=='06')&(combined_df['time_index']=='2:57')].index,'reps'] = '8x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='124')&(combined_df['track_number']=='06')&(combined_df['time_index']=='2:57')].index,'reps1'] = '8x'

combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='124')&(combined_df['track_number']=='07')&(combined_df['time_index']=='1:22')].index,'move1'] = 'LUNGE'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='124')&(combined_df['track_number']=='07')&(combined_df['time_index']=='1:54')].index,'move1'] = 'LUNGE'
to_append_124_07 = combined_df[(combined_df['bodypump_release_number']=='124')&(combined_df['track_number']=='07')&(combined_df['block_clean']=='124-07-1')]
to_append_124_07.loc[:,'block']='3'
to_append_124_07.loc[:,'block_clean']='124-07-3'
combined_df = pd.concat([combined_df, to_append_124_07], ignore_index= True)
to_append_124_07_bis = combined_df[(combined_df['bodypump_release_number']=='124')&(combined_df['track_number']=='07')&(combined_df['block_clean']=='124-07-2')]
to_append_124_07_bis.loc[:,'block']='4'
to_append_124_07_bis.loc[:,'block_clean']='124-07-4'
combined_df = pd.concat([combined_df, to_append_124_07_bis], ignore_index= True)
combined_df = combined_df.sort_values(by=['bodypump_release_number','track_number','block_clean','time_index'], ascending=[True,True,True,True]).reset_index(drop=True)

combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='124')&(combined_df['track_number']=='08')&(combined_df['time_index']=='3:51')].index,'move1'] = 'UPRIGHT ROW'

combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='124')&(combined_df['track_number']=='09')&(combined_df['time_index']=='0:37')].index,'reps1'] = '4x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='124')&(combined_df['track_number']=='09')&(combined_df['time_index']=='0:37')].index,'tempo1'] = '1/1/2'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='124')&(combined_df['track_number']=='09')&(combined_df['time_index']=='0:53')].index,'move1'] = 'CROSS CRAWL'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='124')&(combined_df['track_number']=='09')&(combined_df['time_index']=='1:09')].index,'reps1'] = '4x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='124')&(combined_df['track_number']=='09')&(combined_df['time_index']=='1:09')].index,'tempo1'] = '2/2'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='124')&(combined_df['track_number']=='09')&(combined_df['time_index']=='1:41')].index,'reps1'] = '16x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='124')&(combined_df['track_number']=='09')&(combined_df['time_index']=='1:41')].index,'tempo1'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='124')&(combined_df['track_number']=='09')&(combined_df['time_index']=='2:13')].index,'no_rep'] = 'TRANSITION'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='124')&(combined_df['track_number']=='09')&(combined_df['time_index']=='2:37')].index,'reps1'] = '16x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='124')&(combined_df['track_number']=='09')&(combined_df['time_index']=='2:37')].index,'tempo1'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='124')&(combined_df['track_number']=='09')&(combined_df['time_index']=='3:09')].index,'no_rep'] = 'TRANSITION'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='124')&(combined_df['track_number']=='09')&(combined_df['time_index']=='3:41')].index,'move1'] = 'LEG LIFT'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='124')&(combined_df['track_number']=='09')&(combined_df['time_index']=='4:36')].index,'move1'] = 'ROTATING HOVER'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='124')&(combined_df['track_number']=='09')&(combined_df['time_index']=='4:36')].index,'tempo1'] = '2/2'

# 127
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='127')&(combined_df['track_number']=='01')&(combined_df['time_index']=='2:34')].index,'move1'] = 'CLEAN & PRESS'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='127')&(combined_df['track_number']=='01')&(combined_df['time_index']=='3:34')].index,'no_rep'] = 'TRANSITION'
combined_df.drop(index= combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='127')&(combined_df['track_number']=='01')&(combined_df['time_index']=='6x8')].index].index, axis='index', inplace = True)

combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='127')&(combined_df['track_number']=='02')&(combined_df['time_index']=='0:57')].index,'move1'] = 'SQUAT MID Stance'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='127')&(combined_df['track_number']=='02')&(combined_df['time_index']=='2:56')].index,'combo_clean'] = '127-02-COMBINATION 2'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='127')&(combined_df['track_number']=='02')&(combined_df['time_index']=='2:56')].index,'reps1'] = '4x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='127')&(combined_df['track_number']=='02')&(combined_df['time_index']=='2:56')].index,'move2'] = '7x BOTTOM HALF PULSE SQUAT'

combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='127')&(combined_df['track_number']=='03')&(combined_df['time_index']=='0:59')].index,'reps1'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='127')&(combined_df['track_number']=='03')&(combined_df['time_index']=='0:59')].index,'tempo1'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='127')&(combined_df['track_number']=='03')&(combined_df['time_index']=='0:59')].index,'tempo2'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='127')&(combined_df['track_number']=='03')&(combined_df['time_index']=='2:08')].index,'move1'] = 'A-PRESS'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='127')&(combined_df['track_number']=='03')&(combined_df['time_index']=='2:22')].index,'move1'] = 'A-PRESS'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='127')&(combined_df['track_number']=='03')&(combined_df['time_index']=='2:35')].index,'reps1'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='127')&(combined_df['track_number']=='03')&(combined_df['time_index']=='2:35')].index,'tempo1'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='127')&(combined_df['track_number']=='03')&(combined_df['time_index']=='2:35')].index,'tempo2'] = '1/1'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='127')&(combined_df['track_number']=='03')&(combined_df['time_index']=='4:13')].index,'move1'] = 'CHEST PRESS'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='127')&(combined_df['track_number']=='03')&(combined_df['time_index']=='4:27')].index,'move1'] = 'CHEST PRESS'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='127')&(combined_df['track_number']=='03')&(combined_df['time_index']=='4:41')].index,'reps1'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='127')&(combined_df['track_number']=='03')&(combined_df['time_index']=='4:41')].index,'tempo1'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='127')&(combined_df['track_number']=='03')&(combined_df['time_index']=='4:41')].index,'tempo2'] = '1/1'

combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='127')&(combined_df['track_number']=='04')&(combined_df['time_index']=='0:34')].index,'no_rep'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='127')&(combined_df['track_number']=='04')&(combined_df['time_index']=='0:34')].index,'move1'] = 'WIDE DEADLIFT'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='127')&(combined_df['track_number']=='04')&(combined_df['time_index']=='2:01')].index,'no_rep'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='127')&(combined_df['track_number']=='04')&(combined_df['time_index']=='2:01')].index,'move1'] = 'PLATE SUMO DEADLIFT'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='127')&(combined_df['track_number']=='04')&(combined_df['time_index']=='3:28')].index,'no_rep'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='127')&(combined_df['track_number']=='04')&(combined_df['time_index']=='3:28')].index,'move1'] = 'WIDE DEADLIFT'

combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='127')&(combined_df['track_number']=='05')&(combined_df['time_index']=='0:56')].index,'move1'] = 'TRICEP BARBELL PRESS'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='127')&(combined_df['track_number']=='05')&(combined_df['time_index']=='1:07')].index,'move1'] = 'TRICEP BARBELL PRESS'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='127')&(combined_df['track_number']=='05')&(combined_df['time_index']=='1:41')].index,'tempo1'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='127')&(combined_df['track_number']=='05')&(combined_df['time_index']=='1:41')].index,'reps1'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='127')&(combined_df['track_number']=='05')&(combined_df['time_index']=='1:41')].index,'tempo2'] = '2/2'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='127')&(combined_df['track_number']=='05')&(combined_df['time_index']=='3:11')].index,'no_rep'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='127')&(combined_df['track_number']=='05')&(combined_df['time_index']=='3:1')].index,'move1'] = 'TRICEP PUSHUP'
to_append_127_05 = combined_df[(combined_df['bodypump_release_number']=='127')&(combined_df['track_number']=='05')&(combined_df['block_clean']=='127-05-2')]
to_append_127_05.loc[:,'block']='4'
to_append_127_05.loc[:,'block_clean']='127-05-34'
combined_df = pd.concat([combined_df, to_append_127_05], ignore_index= True)
to_append_127_05_bis = combined_df[(combined_df['bodypump_release_number']=='127')&(combined_df['track_number']=='05')&(combined_df['block_clean']=='127-05-3')]
to_append_127_05_bis.loc[:,'block']='4'
to_append_127_05_bis.loc[:,'block_clean']='127-05-5'
combined_df = pd.concat([combined_df, to_append_127_05_bis], ignore_index= True)
combined_df = combined_df.sort_values(by=['bodypump_release_number','track_number','block_clean','time_index'], ascending=[True,True,True,True]).reset_index(drop=True)

combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='127')&(combined_df['track_number']=='06')&(combined_df['time_index']=='2:00')].index,'no_rep'] = 'TRANSITION'
to_append_127_06 = combined_df[(combined_df['bodypump_release_number']=='127')&(combined_df['track_number']=='06')&(combined_df['block_clean']=='127-06-5')]
to_append_127_06.loc[:,'block']='2'
to_append_127_06.loc[:,'block_clean']='127-06-2'
combined_df = pd.concat([combined_df, to_append_127_06], ignore_index= True)
combined_df = combined_df.sort_values(by=['bodypump_release_number','track_number','block_clean','time_index'], ascending=[True,True,True,True]).reset_index(drop=True)

combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='127')&(combined_df['track_number']=='07')&(combined_df['time_index']=='0:35')].index,'tempo1'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='127')&(combined_df['track_number']=='07')&(combined_df['time_index']=='0:35')].index,'reps1'] = ''
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='127')&(combined_df['track_number']=='07')&(combined_df['time_index']=='0:35')].index,'tempo2'] = '1/1'

combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='127')&(combined_df['track_number']=='08')&(combined_df['time_index']=='0:46')].index,'move1'] = 'UPRIGHT ROW'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='127')&(combined_df['track_number']=='08')&(combined_df['time_index']=='1:37')].index,'move2'] = 'T-RAISE'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='127')&(combined_df['track_number']=='08')&(combined_df['time_index']=='1:37')].index,'tempo2'] = '2/2'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='127')&(combined_df['track_number']=='08')&(combined_df['time_index']=='1:37')].index,'reps2'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='127')&(combined_df['track_number']=='08')&(combined_df['time_index']=='1:37')].index,'move3'] = 'A-RAISE'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='127')&(combined_df['track_number']=='08')&(combined_df['time_index']=='1:37')].index,'tempo3'] = '2/2'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='127')&(combined_df['track_number']=='08')&(combined_df['time_index']=='1:37')].index,'reps3'] = '1x'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='127')&(combined_df['track_number']=='08')&(combined_df['time_index']=='1:37')].index,'move4'] = 'T-RAISE'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='127')&(combined_df['track_number']=='08')&(combined_df['time_index']=='1:37')].index,'tempo4'] = '2/2'
combined_df.loc[combined_df[(combined_df['bodypump_release_number']=='127')&(combined_df['track_number']=='08')&(combined_df['time_index']=='1:37')].index,'reps4'] = '1x'


In [348]:
# 10b. Cleaning up the number of reps of the whole move. If the move is a combination we'll multiply moves 1,2,3,4 by that number, otherwise no (just by 1)
combined_df['reps_int']= combined_df['reps'].str.extract(r'(\d+)', expand=False).fillna(0).astype(int)
combined_df['combo_multiplier'] = combined_df.apply(lambda row: row['reps_int'] if 'COMBINATION' in row['combo_clean'] else 1, axis=1)

# For each move, extracting the number of reps (removing the 'x'), but also checking if the move contains a 'Nx' (like 2x PULSE) --> 'moveN_pulse' or a 'TRIPLE' --> 'repN_multiplier'
combined_df['reps1_int']= combined_df['reps1'].str.extract(r'(\d+)', expand=False).fillna(0).astype(int)
combined_df['reps1_multiplier'] = 1
combined_df.loc[combined_df['move1'].str.contains('TRIPLE'), 'reps1_multiplier'] = 3
combined_df.loc[combined_df['move1'].str.contains('DOUBLE'), 'reps1_multiplier'] = 2
combined_df['move1_pulse']= combined_df['move1'].str.extract(r'(\d+)', expand=False).fillna(1).astype(int)

combined_df['reps2_int']= combined_df['reps2'].str.extract(r'(\d+)', expand=False).fillna(0).astype(int)
combined_df['reps2_multiplier'] = 1
combined_df.loc[combined_df['move2'].str.contains('TRIPLE'), 'reps2_multiplier'] = 3
combined_df.loc[combined_df['move2'].str.contains('DOUBLE'), 'reps2_multiplier'] = 2
combined_df['move2_pulse']= combined_df['move2'].str.extract(r'(\d+)', expand=False).fillna(1).astype(int)

combined_df['reps3_int']= combined_df['reps3'].str.extract(r'(\d+)', expand=False).fillna(0).astype(int)
combined_df['reps3_multiplier'] = 1
combined_df.loc[combined_df['move3'].str.contains('TRIPLE'), 'reps3_multiplier'] = 3
combined_df.loc[combined_df['move3'].str.contains('DOUBLE'), 'reps3_multiplier'] = 2
combined_df['move3_pulse']= combined_df['move3'].str.extract(r'(\d+)', expand=False).fillna(1).astype(int)

combined_df['reps4_int']= combined_df['reps4'].str.extract(r'(\d+)', expand=False).fillna(0).astype(int)
combined_df['reps4_multiplier'] = 1
combined_df.loc[combined_df['move4'].str.contains('TRIPLE'), 'reps4_multiplier'] = 3
combined_df.loc[combined_df['move4'].str.contains('DOUBLE'), 'reps4_multiplier'] = 2
combined_df['move4_pulse']= combined_df['move4'].str.extract(r'(\d+)', expand=False).fillna(1).astype(int)

# Finally calculating the final number of each rep (not distinguishing full range and pulses) by multiplying everything together
combined_df['reps1_final'] = combined_df['combo_multiplier'] * combined_df['reps1_multiplier'] * combined_df['move1_pulse'] * combined_df['reps1_int']
combined_df['reps2_final'] = combined_df['combo_multiplier'] * combined_df['reps2_multiplier'] * combined_df['move2_pulse'] * combined_df['reps2_int']
combined_df['reps3_final'] = combined_df['combo_multiplier'] * combined_df['reps3_multiplier'] * combined_df['move3_pulse'] * combined_df['reps3_int']
combined_df['reps4_final'] = combined_df['combo_multiplier'] * combined_df['reps4_multiplier'] * combined_df['move3_pulse'] * combined_df['reps4_int']

# Also cleaning up the tempo of moves 1,2,3,4. We keep the tempo if indicated but if there are PULSES or BOTTOM/TOP HALF, we will keep the info 'PULSE' or 'HALF'
combined_df['tempo1_final'] = combined_df.apply(lambda row: 'PULSE' if 'PULSE' in row['move1'] else 'HALF' if 'HALF' in row['move1'] else 'TRIPLE' if 'TRIPLE' in row['move1'] else 'DOUBLE' if 'DOUBLE' in row['move1'] else row['tempo1'], axis=1)
combined_df['tempo2_final'] = combined_df.apply(lambda row: 'PULSE' if 'PULSE' in row['move2'] else 'HALF' if 'HALF' in row['move2'] else 'TRIPLE' if 'TRIPLE' in row['move2'] else 'DOUBLE' if 'DOUBLE' in row['move2'] else row['tempo2'], axis=1)
combined_df['tempo3_final'] = combined_df.apply(lambda row: 'PULSE' if 'PULSE' in row['move3'] else 'HALF' if 'HALF' in row['move3'] else 'TRIPLE' if 'TRIPLE' in row['move3'] else 'DOUBLE' if 'DOUBLE' in row['move3'] else row['tempo3'], axis=1)
combined_df['tempo4_final'] = combined_df.apply(lambda row: 'PULSE' if 'PULSE' in row['move4'] else 'HALF' if 'HALF' in row['move4'] else 'TRIPLE' if 'TRIPLE' in row['move4'] else 'DOUBLE' if 'DOUBLE' in row['move4'] else row['tempo4'], axis=1)

combined_df = combined_df.drop(['reps','reps_int','combo_multiplier','tempo1','reps1','reps1_int','reps1_multiplier','move1_pulse','tempo2','reps2','reps2_int','reps2_multiplier','move2_pulse',\
                                'tempo3','reps3','reps3_int','reps3_multiplier','move3_pulse','tempo4','reps4','reps4_int','reps4_multiplier','move4_pulse'],axis=1)

# Cleaning up the incorrect time indexes (INTRo, TRANSITION, RECOVERY...) the time index should be in the 'lyrics' column
combined_df.loc[combined_df['time_index'].isin(['INTRO','TRANSITION','RECOVERY']),'time_index'] = combined_df.loc[combined_df['time_index'].isin(['INTRO','TRANSITION','RECOVERY']),'lyrics']

combined_df = combined_df.sort_values(by=['bodypump_release_number','track_number','block_clean','time_index'], ascending=[True,True,True,True]).reset_index(drop=True)

# Calculating the intervals in second between each row, and converting also the dong duration in seconds.
def duration_to_seconds(duration_str):
    minutes, seconds = map(int, duration_str.split(':'))
    return minutes * 60 + seconds
                           
combined_df = combined_df.assign(time_index_sec=pd.to_datetime(combined_df['time_index'], format='%M:%S'))
combined_df['duration_sec'] = pd.to_datetime(combined_df['Duration'],format='%M:%S')
combined_df['interval_sec'] = combined_df.groupby(['bodypump_release_number', 'track_number'])['time_index_sec'].diff().dt.total_seconds().shift(-1).fillna(0).astype(int)
combined_df['interval_to_song_end'] = (combined_df['duration_sec']-combined_df['time_index_sec']).dt.total_seconds().fillna(0).astype(int)
combined_df.loc[combined_df['interval_sec']<=0,'interval_sec'] = combined_df.loc[combined_df['interval_sec']<=0,'interval_to_song_end']
combined_df['song_duration_sec'] = combined_df['Duration'].apply(duration_to_seconds)


In [349]:
squats_02_moves_rename_rules = {
    r'PULSE SQUAT with CALF RAISE|TRIPLE SQUAT with CALF RAISE|2x SQUAT with CALF RAISE':'SQUAT with CALF RAISE',
    r'2x BOTTOM HALF FRONT SQUAT|4x BOTTOM HALF FRONT SQUAT|BARBELL FRONT SQUAT|TRIPLE PULSE FRONT SQUAT|PULSE FRONT SQUAT|FRONT SQUAT': 'FRONT SQUAT',
    r'SQUAT CURL & PRESS|SQUAT CURL POWER PRESS|SQUAT CURL PRESS|SQUAT PRESS|3x CHEST PRESS SQUAT|7x CHEST PRESS SQUAT':'PLATE SQUAT PRESS',
    r'SQUAT Mid Stance|SQUAT Wide Stance|SQUAT Wider Stance|Squat MID Stance|SQUAT MID Stance|SQUAT MID STANCE|SQUAT WIDE STANCE|SQUAT WIDE Stance|SQUAT WIDER STANCE|SQUAT WIDER Stance|TOP HALF SQUAT|4x PULSE BOTTOM HALF SQUAT|TRIPLE PULSE SQUAT|3x BOTTOM HALF PULSE SQUAT|7x BOTTOM HALF PULSE SQUAT|2x BOTTOM-HALF SQUAT|2x BOTTOM HALF SQUAT|7x PULSE SQUAT|4x PULSE SQUAT|2x PULSE SQUAT|3x PULSE SQUAT|PULSE SQUAT|SQUAT BOTTOM HALF PULSE|STAGGERED SQUAT|BOTTOM HALF SQUAT|PAUSE SQUAT|PLATE SQUAT|Squat|SQUAT': 'SQUAT',
}
chest_03_moves_rename_rules = {
    r'BENCH CHEST PRESS|4x BOTTOM HALF PULSE CHEST PRESS|STAGGERED CHEST PRESS|TRIPLE PULSE CHEST PRESS|4x PULSE CHEST PRESS|4x BOTTOM HALF CHEST PRESS|4x Bottom Half Pulse CHEST PRESS|3x PULSE CHEST PRESS|PULSE CHEST PRESS|CHEST PRESS BOTTOM HALF PULSE|CHEST PRESS': 'CHEST PRESS',
    r'PULSE NARROW PLATE PRESS|NARROW PLATE PRESS':'NARROW PLATE PRESS',
    r'3x PULSE A-PRESS|4x PULSE A-PRESS|TRIPLE PULSE A-PRESS|PULSE A-PRESS|A-PRESS':'A-PRESS',
    r'3x PULSE CHEST FLY|4x BOTTOM HALF CHEST FLY|4x PULSE CHEST FLY|CHEST FLY': 'CHEST FLY',
    r'3x PULSE PUSHUP|4x PULSE PUSHUP|16x PULSE PUSHUP|4x PULSE BENCH PUSHUP|TRIPLE PULSE PUSHUP|PUSHUP PULSE|WIDE-GRIP PUSHUP|WIDE PUSHUP|MID-GRIP PUSHUP|TRIPLE WIDE-GRIP PUSHUP|STAGGERED BENCH PUSHUP|STAGGERED PUSHUP|PULSE BENCH PUSHUP|WALKING BENCH PUSHUP|BENCH PUSHUP|PULSE PUSHUP|PUSHUP|Pushup': 'PUSHUP',
    r'BENCH PUSHUP': 'PUSHUP',
    r'FAST PLANK FOOT TAP|SLOW PLANK FOOT TAP|PLANK FOOT TAP|ALT STRAIGHT LEG LIFT' : 'PLANK FOOT TAP'
}
back_04_moves_rename_rules = {
    r'WIDE DEADLIFT':'WIDE DEADLIFT',
    r'PLATE SUMO DEADLIFT': 'PLATE DEADLIFT',
    r'DEADLIFT': 'DEADLIFT',
    r'TRIPLE WIDE DEADROW|WIDE DEADROW|WIDE TRIPLE DEADROW':'WIDE DEADROW',
    r'DOUBLE DEADROW|Triple DEADROW|TRIPLE DEADROW|TRIPLE ROW|Deadrow|DEADROW|Triple Row': 'DEADROW',
    r'TRIPLE PLATE DEADROW|TRIPLE SPLIT PLATE ROW|TRIPLE NARROW PLATE ROW|TRIPLE WIDE PLATE ROW|TRIPLE SINGLE-ARM PLATE DEADROW|TRIPLE PULSE KNEELING SINGLE-ARM PLATE ROW|KNEELING SINGLE-ARM PLATE ROW|SINGLE- ARM PLATE ROW|WIDE PLATE ROW|NARROW PLATE ROW|PLATE DEADROW':'PLATE DEADROW',
    r'CLEAN AND PRESSS|PLATE CLEAN AND PRESSS|CLEAN & PRESS|CLEAN AND PRESS|PLATE CLEAN AND PRESS|PLATE CLEAN & PRESS': 'CLEAN AND PRESS',
    r'PLATE TRIPLE SQUAT PRESS|DROP SQUAT PLATE PRESS|PLATE CURL SQUAT PRESS|PLATE SQUAT CURL & PRESS|PLATE CURL SQUAT & PRESS|PLATE SQUAT CURL PRESS|SQUAT PLATE PRESS|PLATE SQUAT PRESS|3x CHEST PRESS SQUAT|7x CHEST PRESS SQUAT': 'PLATE SQUAT PRESS',
    r'PLATE SQUAT CURL POWER PRESS|POWER PRESS' : 'POWER PRESS',
    r'WIDE HIGH PULL|LOW PULL|HIGH PULL' : 'HIGH PULL'
}
triceps_05_moves_rename_rules = {
    r'HOLD TRICEP PUSHUP|TOP HALF PULSE BENCH TRICEP PUSHUP|TRIPLE PULSE BENCH TRICEP PUSHUP|3x PULSE TRICEP BENCH PUSHUP|3x PULSE TRICEPS PUSHUP|TRICEP PUSHUP BOTTOM HALF PULSE|BENCH TRICEP PUSHUP|TRICEP PUSHUP|TRICEP BENCH PUSHUP': 'TRICEPS PUSHUP',
    r'TRICEP BARBELL O/H EXTENSION|TRICEP BARBELL EXTENSION' : 'TRICEPS BARBELL EXTENSION',
    r'SEATED TRICEP OVERHEAD EXTENSION|STANDING TRICEP OVERHEAD EXTENSION|TRIPLE PULSE KNEELING O/H TRICEP EXTENSION|3x PULSE SEATED O/H TRICEP PLATE EXTENSION|16x PULSE STANDING O/H TRICEP EXTENSION|TRIPLE PULSE STANDING O/H TRICEP EXTENSION|TRIPLE PULSE SEATED O/H TRICEP PLATE EXTENSION|PULSE KNEELING O/H TRICEP EXTENSION|STANDING O/H TRICEP PLATE EXTENSION|STANDING O/H EXTENSION|KNEELING PLATE O/H TRICEP EXTENSION|KNEELING O/H TRICEP PLATE EXTENSION|KNEELING O/H TRICEP EXTENSION|PULSE STANDING O/H TRICEP EXTENSION|STANDING O/H TRICEP EXTENSION|PULSE STANDING PLATE O/H TRICEP EXTENSION|STANDING PLATE O/H TRICEP EXTENSION|STANDING OVERHEAD TRICEP EXTENSION BOTTOM HALF PULSE|STANDING OVERHEAD TRICEP EXTENSION|SEATED O/H TRICEP EXTENSION|SEATED O/H TRICEP PLATE EXTENSION|TRIPLE PULSE STANDING O/H TRICEP EXTENSION|TRIPLE PULSE TRICEPS OVERHEAD EXTENSION|PULSE TRICEP EXTENSION|TRICEP EXTENSION' : 'TRICEPS OVERHEAD EXTENSION',
    r'TRICEP PRESS PULLOVER|TRICEP PRESS PULL-OVER|TRICEP PRESS / PULLOVER' : 'TRICEPS PULLOVER',
    r'Kickback Row|Kneeling Tricep Kickback|TRIPLE PULSE TRICEP KICKBACK|TRIPLE KNEELING TRICEP KICKBACK|TRIPLE KNEELING PLATE TRICEP KICKBACK ROW|KNEELING TRICEP KICKBACK|TRICEP KICKBACK ROW|16x PULSE TRICEP BENCH KICKBACK|TRIPLE PULSE TRICEP BENCH KICKBACK|TRICEP BENCH KICKBACK|KICKBACK ROW|TRICEP KICKBACK' : 'TRICEPS KICKBACK',
    r'BOTTOM HALF PULSE TRICEP PRESS|8x TRICEP PRESS CHEST PRESS|8x TRICEP PRESS BOTTOM HALF PULSE|TRICEP PRESS BOTTOM HALF PULSE|TOP HALF PULSE TRICEP NARROW PRESS|BOTTOM HALF PULSE TRICEP NARROW PRESS|3x PULSE TRICEP PRESS|TRIPLE PULSE TRICEP PRESS|TRICEP NARROW PRESS|TRICEP BARBELL PRESS|PULSE TRICEP PRESS|TRICEP PRESS' : 'TRICEPS PRESS',
    r'PULSE TRICEP DIP|TRICEP DIP' : 'TRICEPS DIP'
}
biceps_06_moves_rename_rules = {
    r'TRIPLE MID RANGE PULSE PLATE CURL|MID-RANGE PULSE BICEP PLATE CURL|BICEP CURL MIDRANGE PULSE|BICEP CURL MID-RANGE PULSE|3x MID-RANGE PULSE|TRIPLE MID-RANGE PULSE|MID-RANGE ISOMETRIC HOLD|PULSE MID-RANGE CURL|MID-RANGE TRIPLE PULSE|MID-RANGE PULSE': 'BICEPS MID RANGE CURL',
    r'ALT SINGLE ARM BICEP PLATE CURL|ALT SINGLE-ARM BICEP PLATE CURL|FAST ALT SINGLE-ARM BICEP PLATE CURL|ALT PLATE CURL|BICEP PLATE CURL|PLATE CURL': 'BICEPS PLATE CURL',
    r'BARBELL BICEP CURL|BICEP CURL' : 'BICEPS BARBELL CURL',
    r'TRIPLE PREACHER CURL|PREACHER CURL' : 'BICEPS PREACHER CURL',
    r'BICEP PLATE ROW|PLATE ROW|BICEP ROW': 'BICEPS PLATE ROW'
}
lunges_07_moves_rename_rules = {
    r'BACK STEPPING TRIPLE PULSE LUNGE|3x PULSE ALT BENCH BACKWARD-STEPPING LUNGE|3x PULSE BENCH BACKWARD-STEPPING LUNGE|ALT BACK STEPPING LUNGE|ALT BACKWARD STEPPING LUNGE|ALT BACKWARD-STEPPING LUNGE|TRIPLE PULSE BACK STEPPING LUNGE|TRIPLE PULSE BACKWARD-STEPPING LUNGE|BACKWARD-STEPPING LUNGE|2x PULSE PLATE LUNGE WITH STEP|PULSE PLATE LUNGE WITH STEP|STEPPING LUNGE': 'BACK STEPPING LUNGE',
    r'7x BOTTOM HALF PULSE PLATE SQUAT|3x BOTTOM HALF PULSE PLATE SQUAT|PLATE FRONT SQUAT|TRIPLE PULSE PLATE FRONT SQUAT|PULSE PLATE FRONT SQUAT|TRIPLE PULSE PLATE WIDE FRONT SQUAT|TRIPLE PULSE PLATE WIDER FRONT SQUAT': 'PLATE FRONT SQUAT',
    r'7x BOTTOM HALF PULSE LUNGE|BOTTOM HALF PULSE LUNGE|TRIPLE PULSE LUNGE|PLATE LUNGE|16x PULSE LUNGE|7x PULSE LUNGE|3x PULSE LUNGE|PULSE LUNGE|LUNGE': 'LUNGE'
}
shoulders_08_moves_rename_rules = {
    r'WIDE UPRIGHT ROW|KNEELING UPRIGHT ROW|TRIPLE PULSE UPRIGHT ROW|TRIPLE WIDE ROW|STANDING UPRIGHT ROW|PULSE UPRIGHT ROW|UPRIGHT ROW': 'UPRIGHT ROW',
    r'PLATE UPRIGHT ROW': 'PLATE UPRIGHT ROW',
    r'STANDING PLATE SIDE RAISE EXTENSION': 'SIDE RAISE EXTENSION',
    r'STANDING PLATE SIDE RAISE|KNEELING SIDE RAISE|STANDING SIDE RAISE|SIDE RAISE': 'SIDE RAISE',
    r'SPLIT REAR RAISE|PULSE REAR DELTOID RAISE|TRIPLE PULSE KNEELING REAR RAISE|KNEELING REAR RAISE|REAR DELTOID RAISE|A-RAISE': 'REAR RAISE',
    r'I RAISE|Y RAISE|T RAISE|Y-RAISE|T-RAISE' : 'I-Y-T RAISE',
    r'STANDING SIDE ROTATOR RAISE|ROTATOR RAISE' : 'ROTATOR RAISE',
    r'STANDING MAC RAISE|ALT MAC RAISE': 'MAC RAISE',
    r'STANDING PLATE FORWARD RAISE|KNEELING FORWARD RAISE|STANDING FORWARD RAISE': 'FORWARD RAISE',
    r'STANDING SHOULDER PRESS|PULSE STANDING PLATE O/H PRESS|STANDING PLATE O/H PRESS|STANDING O/H SHOULDER PRESS|O/H SHOULDER PRESS|STANDING O/H SHOULDER PRESS|STANDING OVERHEAD PRESS|STANDING O/H PRESS|SHOULDER PRESS': 'SHOULDER STRICT PRESS',
    r'STANDING PUSH PRESS|PUSH PRESS': 'SHOULDER PUSH PRESS',
    r'STANDING ARNOLD PRESS': 'ARNOLD PRESS',
    r'STANDING REAR DELTOID FLY WITH STEP|STANDING REAR DELTOID FLY|STANDING TRIPLE REAR DELTOID FLY|TRIPLE PULSE KNEELING REAR DELTOID RAISE|KNEELING REAR DELTOID RAISE|KNEELING REAR DELTOID FLY|STANDING REAR DELTOID RAISE|TIPPING REAR DELTOID FLY WITH TWIST': 'DELTOID FLY',
    r'MID PUSHUP|MID-GRIP PUSHUP': 'PUSHUP',
    r'WALKING UP / DOWN BENCH PLANK': 'WALKING PLANK',
    r'SLOW MOUNTAIN CLIMBER': 'MOUNTAIN CLIMBER'
}
core_09_moves_rename_rules = {
    r'TRIPLE PULSE C-CRUNCH|PULSE CRUNCH|16x PULSE PLATE CRUNCH|15x PULSE PLATE CRUNCH|3x PULSE PLATE CRUNCH|16x PLATE PULSE CRUNCH|PLATE CRUNCH|PULSE C-CRUNCH|C CRUNCH|C-CRUNCH|BICYCLE CRUNCH|BENCH CRUNCH': 'CRUNCH',
    r'SINGLE-ARM PLATE LIFT|16x PULSE PLATE LIFT|PULSE PLATE LIFT|PLATE LIFT': 'CRUNCH PLATE LIFT',
    r'BENCH REVERSE CRUNCH': 'REVERSE CRUNCH',
    r'C CRUNCH HOLD with ALT LEG EXTENSION|C CRUNCH HOLD with DOUBLE LEG EXTENSION': 'CRUNCH with LEG EXTENSION',
    r'ALT CROSS CRAWL|ALT SINGLE-LEG EXTENSION|SINGLE LEG EXTENSION|CROSS CRAWL': 'CROSS CRAWL',
    r'DOUBLE-LEG EXTENSION|SHOOT OUT': 'SHOOT OUT',
    r'DIAGONAL LEG LIFT|LEG LIFT|HORSE STANCE': 'HOVER WITH LEG LIFT',
    r'HOVER FORWARD / BACK': 'HOVER WITH LEG LIFT',
    r'SIDE HOVER WITH ROTATION|ROTATING SIDE PLANK|ROTATING PLATE SIDE HOVER|ROTATING SIDE HOVER|ROTATING HOVER|SIDE PLANK ROTATION' : 'HOVER with ROTATON',
    r'MOUNTAIN CLIMBER|SPRINTER': 'MOUNTAIN CLIMBER',
    r'HIP BRIDGE with ALT KNEE LIFT|ALT SINGLE-LEG HIP BRIDGE WITH LEG EXTENSION|ALT SINGLE-LEG HIP BRIDGE WITH PIKE LEG EXTENSION|12x PULSE HIP BRIDGE|16x PULSE HIP BRIDGE|PULSE HIP BRIDGE|HIP BRIDGE' : 'HIP BRIDGE',
    r'ALT FAST SHOULDER TAP PLANK|SIDE PLANK|PULSING PLATE SIDE HOVER|SLOW HOVER TAP|FAST HOVER TAP|WALKING HOVER|SIDE HOVER PULSE|15x PULSE SIDE HOVER|HOVER': 'HOVER',
    r'PLANK': 'PLANK',
    r'ALT WALKING PLANK/HOVER': 'HOVER',
    r'C-CRUNCH / SINGLE LEG EXTENSION': 'CRUNCH with LEG EXTENSION',
    r'BOTTOM HALF PULSE': 'CHEST PRESS'
}
moves_rename_rules = squats_02_moves_rename_rules | chest_03_moves_rename_rules | back_04_moves_rename_rules |triceps_05_moves_rename_rules | biceps_06_moves_rename_rules | lunges_07_moves_rename_rules | shoulders_08_moves_rename_rules | core_09_moves_rename_rules

In [350]:
# 11. For our working dataframe, dropping some columns for calculations and keeping only the rows with reps (dropping the no_rep rows)
final_df = combined_df[combined_df['no_rep']=='']

# Renaming the moves to more generic names for easier comparison
final_df = final_df.assign(move1_final=final_df['move1'].replace(moves_rename_rules, regex=True))
final_df = final_df.assign(move2_final=final_df['move2'].replace(moves_rename_rules, regex=True))
final_df = final_df.assign(move3_final=final_df['move3'].replace(moves_rename_rules, regex=True))
final_df = final_df.assign(move4_final=final_df['move4'].replace(moves_rename_rules, regex=True))

# Dropping some columns
final_df = final_df[['bodypump_release_number', 'track_number', 'muscle_group', 'Title','Duration','song_duration_sec', 'move1_final', 'move2_final','move3_final', 'move4_final', 'reps1_final', 'reps2_final', 'reps3_final',\
       'reps4_final', 'tempo1_final', 'tempo2_final', 'tempo3_final','tempo4_final', 'interval_sec']]

# Forcing cells with rep number to 0 when there is no move
final_df['reps1_final'] = np.where(final_df['move1_final']=='',0, final_df['reps1_final'])
final_df['reps2_final'] = np.where(final_df['move2_final']=='',0, final_df['reps2_final'])
final_df['reps3_final'] = np.where(final_df['move3_final']=='',0, final_df['reps3_final'])
final_df['reps4_final'] = np.where(final_df['move4_final']=='',0, final_df['reps4_final'])

# Calculating the total nb of reps per row and the nb of reps per sec of each row
final_df['total_reps']= (final_df['reps1_final']+final_df['reps2_final']+final_df['reps3_final']+final_df['reps4_final'])
final_df['sec_per_rep']= final_df['interval_sec']/final_df['total_reps']

# Calculating the number of pulses on each row
final_df[final_df['tempo1_final']=='PULSE']
final_df['total_reps_pulses'] = np.where(final_df['tempo1_final']=='PULSE',final_df['reps1_final'],0) + np.where(final_df['tempo2_final']=='PULSE',final_df['reps2_final'],0) +\
      np.where(final_df['tempo3_final']=='PULSE',final_df['reps3_final'],0) + np.where(final_df['tempo4_final']=='PULSE',final_df['reps4_final'],0)

final_df.head()

,bodypump_release_number,track_number,muscle_group,Title,Duration,song_duration_sec,move1_final,move2_final,move3_final,move4_final,...,reps3_final,reps4_final,tempo1_final,tempo2_final,tempo3_final,tempo4_final,interval_sec,total_reps,sec_per_rep,total_reps_pulses
1,105,01,WARMUP,ALL STARS,5:21,321,DEADLIFT,,,,...,0,0,4/4,,,,15,2,7.50,0
2,105,01,WARMUP,ALL STARS,5:21,321,DEADLIFT,,,,...,0,0,2/2,,,,15,4,3.75,0
3,105,01,WARMUP,ALL STARS,5:21,321,DEADROW,,,,...,0,0,1/1,,,,15,4,3.75,0
4,105,01,WARMUP,ALL STARS,5:21,321,DEADROW,,,,...,0,0,TRIPLE,,,,15,6,2.50,0
5,105,01,WARMUP,ALL STARS,5:21,321,UPRIGHT ROW,,,,...,0,0,2/2,,,,15,4,3.75,0


In [351]:
final_df[(final_df['bodypump_release_number']=='95')&(final_df['track_number']=='08')]

,bodypump_release_number,track_number,muscle_group,Title,Duration,song_duration_sec,move1_final,move2_final,move3_final,move4_final,...,reps3_final,reps4_final,tempo1_final,tempo2_final,tempo3_final,tempo4_final,interval_sec,total_reps,sec_per_rep,total_reps_pulses
2622,95,08,SHOULDERS,I Am Machine,5:17,317,SQUAT,,,,...,0,0,,,,,3,0,inf,0
2623,95,08,SHOULDERS,I Am Machine,5:17,317,UPRIGHT ROW,,,,...,0,0,4/4,,,,13,2,6.500000,0
2624,95,08,SHOULDERS,I Am Machine,5:17,317,UPRIGHT ROW,UPRIGHT ROW,,,...,0,0,1/1,1/1/2,,,49,24,2.041667,0
2625,95,08,SHOULDERS,I Am Machine,5:17,317,UPRIGHT ROW,,,,...,0,0,3/1,,,,25,7,3.571429,0
2626,95,08,SHOULDERS,I Am Machine,5:17,317,SHOULDER STRICT PRESS,SHOULDER STRICT PRESS,,,...,0,0,1/1,1/1/2,,,26,12,2.166667,0
2627,95,08,SHOULDERS,I Am Machine,5:17,317,SHOULDER PUSH PRESS,,,,...,0,0,1/1/2,,,,10,4,2.500000,0
2629,95,08,SHOULDERS,I Am Machine,5:17,317,SIDE RAISE,,,,...,0,0,4/4,,,,26,4,6.500000,0
2630,95,08,SHOULDERS,I Am Machine,5:17,317,SIDE RAISE,ROTATOR RAISE,,,...,0,0,1/1,1/1,,,24,12,2.000000,0
2631,95,08,SHOULDERS,I Am Machine,5:17,317,SIDE RAISE,,,,...,0,0,,,,,4,0,inf,0
2632,95,08,SHOULDERS,I Am Machine,5:17,317,UPRIGHT ROW,,,,...,0,0,3/1,,,,20,7,2.857143,0


In [352]:
# final rename of some moves
final_df.loc[final_df[final_df['move1_final']=='TRIPLE PULSE TRICEPS OVERHEAD EXTENSION'].index,'move1_final'] = 'TRICEPS OVERHEAD EXTENSION'
final_df.loc[final_df[final_df['move2_final']=='TRIPLE PULSE TRICEPS OVERHEAD EXTENSION'].index,'move2_final'] = 'TRICEPS OVERHEAD EXTENSION'
final_df.loc[final_df[final_df['move1_final']=='KNEELING REAR RAISE'].index,'move1_final'] = 'REAR RAISE'
final_df.loc[final_df[final_df['move2_final']=='KNEELING REAR RAISE'].index,'move2_final'] = 'REAR RAISE'
final_df.loc[final_df[final_df['move1_final']=='STANDING REAR RAISE'].index,'move1_final'] = 'REAR RAISE'
final_df.loc[final_df[final_df['move2_final']=='STANDING REAR RAISE'].index,'move2_final'] = 'REAR RAISE'
final_df.loc[final_df[final_df['move1_final']=='TRIPLE PULSE KNEELING REAR RAISE'].index,'move1_final'] = 'REAR RAISE'
final_df.loc[final_df[final_df['move2_final']=='TRIPLE PULSE KNEELING REAR RAISE'].index,'move2_final'] = 'REAR RAISE'
final_df.loc[final_df[final_df['move1_final']=='16x CRUNCH'].index,'move1_final'] = 'CRUNCH'
final_df.loc[final_df[final_df['move2_final']=='16x CRUNCH'].index,'move2_final'] = 'CRUNCH'
final_df.loc[final_df[final_df['move1_final']=='3x PULSE TRICEPS PUSHUP'].index,'move1_final'] = 'TRICEPS PUSHUP'
final_df.loc[final_df[final_df['move2_final']=='3x PULSE TRICEPS PUSHUP'].index,'move2_final'] = 'TRICEPS PUSHUP'
final_df.loc[final_df[final_df['move1_final']=='TRIPLE SQUAT with CALF RAISE'].index,'move1_final'] = 'SQUAT with CALF RAISE'
final_df.loc[final_df[final_df['move2_final']=='TRIPLE SQUAT with CALF RAISE'].index,'move2_final'] = 'SQUAT with CALF RAISE'
final_df.loc[final_df[final_df['move1_final']=='2x SQUAT with CALF RAISE'].index,'move1_final'] = 'SQUAT with CALF RAISE'
final_df.loc[final_df[final_df['move2_final']=='2x SQUAT with CALF RAISE'].index,'move2_final'] = 'SQUAT with CALF RAISE'
final_df.loc[final_df[final_df['move1_final']=='BENCH PUSHUP'].index,'move1_final'] = 'PUSHUP'
final_df.loc[final_df[final_df['move2_final']=='BENCH PUSHUP'].index,'move2_final'] = 'PUSHUP'
final_df.loc[final_df[final_df['move1_final']=='CRUNCH / CROSS CRAWL'].index,'move1_final'] = 'PUSHUP'
final_df.loc[final_df[final_df['move2_final']=='CRUNCH / CROSS CRAWL'].index,'move2_final'] = 'PUSHUP'
final_df.loc[final_df[final_df['move1_final']=='Triple DEADROW'].index,'move1_final'] = 'DEADROW'
final_df.loc[final_df[final_df['move2_final']=='Triple DEADROW'].index,'move2_final'] = 'DEADROW'
final_df.loc[final_df[final_df['move3_final']=='Triple DEADROW'].index,'move3_final'] = 'DEADROW'
final_df.loc[final_df[final_df['move4_final']=='Triple DEADROW'].index,'move4_final'] = 'DEADROW'
final_df.loc[final_df[final_df['move1_final']=='3x CHEST PRESS SQUAT'].index,'move1_final'] = 'PLATE SQUAT PRESS'
final_df.loc[final_df[final_df['move1_final']=='7x CHEST PRESS SQUAT'].index,'move1_final'] = 'PLATE SQUAT PRESS'



In [353]:
# Result df 1: total of each move per track 
# Melting the dataframe by moves to get a long dataframe with one row per move 1/2/3/4.
final_df_melt = pd.melt(final_df, id_vars=['bodypump_release_number', 'track_number','muscle_group','Title','Duration', 'reps1_final', 'reps2_final','reps3_final', 'reps4_final'], 
            value_vars=['move1_final', 'move2_final','move3_final','move4_final'],
            var_name='move_variable', value_name='move')

# Now based on the move_variable obtained on each row, assigning the corresponding number of reps
final_df_melt['reps'] = np.where(final_df_melt['move_variable']=='move1_final', final_df_melt['reps1_final'],\
                                    np.where(final_df_melt['move_variable']=='move2_final', final_df_melt['reps2_final'],\
                                    np.where(final_df_melt['move_variable']=='move3_final', final_df_melt['reps3_final'],\
                                             final_df_melt['reps4_final'])))

# Cleaning up the rows and columns remove rows with no reps (final) and the columns of reps1/2/3/4 and move_variable
final_df_melt = final_df_melt[final_df_melt['reps']!=0]
final_df_melt = final_df_melt[['bodypump_release_number', 'track_number','muscle_group','Title','Duration','move','reps']]

# Re-grouping the rows of the same track together and resetting the index
final_df_melt = final_df_melt.sort_values(by=['bodypump_release_number','track_number'], ascending=[True,True]).reset_index(drop=True)

# Finally summing the number of reps per move and per track
reps_by_move_analysis = final_df_melt.groupby(['bodypump_release_number', 'track_number', 'muscle_group', 'Title','Duration','move'])['reps'].sum().reset_index()

# Replacing the titles by the titles from the full list of tracks I have
reps_by_move_analysis = reps_by_move_analysis.drop(columns=['Title'],axis=1)
reps_by_move_analysis = reps_by_move_analysis.merge(my_bodypump_tracks_df[['bodypump_release_number','track_type_id','track_title']], how='left', left_on = ['bodypump_release_number','track_number'], right_on = ['bodypump_release_number','track_type_id'])
reps_by_move_analysis = reps_by_move_analysis.drop(columns=['track_type_id'],axis=1)

# Rename WARM-UP into WARMUP
reps_by_move_analysis['muscle_group'] = reps_by_move_analysis['muscle_group'].replace('WARM-UP', 'WARMUP')

reps_by_move_analysis.head()


,bodypump_release_number,track_number,muscle_group,Duration,move,reps,track_title
0,105,01,WARMUP,5:21,BACK STEPPING LUNGE,8,ALL STARS
1,105,01,WARMUP,5:21,CLEAN AND PRESS,4,ALL STARS
2,105,01,WARMUP,5:21,DEADLIFT,12,ALL STARS
3,105,01,WARMUP,5:21,DEADROW,20,ALL STARS
4,105,01,WARMUP,5:21,PLATE FRONT SQUAT,16,ALL STARS


In [354]:
# figure out which moves are not in the consolidated list 
consolidated_move_list = list(moves_rename_rules.values())
final_move_list = list(reps_by_move_analysis['move'].unique())
moves_to_review = [x for x in final_move_list if x not in consolidated_move_list]
moves_to_review

[]

In [355]:
# Result df 2a: total_reps_analysis_df to analyse the number of reps and of pulses per track and the speed (number of seconds per rep or 1 rep every X sec)
total_reps_analysis_df = final_df.groupby(['bodypump_release_number', 'track_number', 'muscle_group', 'Title','Duration','song_duration_sec'])[['total_reps','total_reps_pulses']].sum().reset_index()
total_reps_analysis_df['sec_per_rep'] = total_reps_analysis_df['song_duration_sec']/total_reps_analysis_df['total_reps']

In [356]:
# Result df 2b: analysing the intensity of each move + tempo
# first we need to melt the dataframe again to get the moves under each other but this time, we want to append to tempo to each move first.
# we also replace the 1/1/1 and the 2/2/2 (which are truncated 1/1/1/1 nd 2/2/2/2) by 1/1 and 2/2 respectively for simplicity
final_df['tempo1_final'] = np.where(final_df['tempo1_final']=='1/1/1','1/1',final_df['tempo1_final'])
final_df['tempo1_final'] = np.where(final_df['tempo1_final']=='2/2/2','2/2',final_df['tempo1_final'])
final_df['tempo1_final'] = np.where(final_df['tempo1_final']=='4/4/4','4/4',final_df['tempo1_final'])
final_df['tempo2_final'] = np.where(final_df['tempo2_final']=='1/1/1','1/1',final_df['tempo2_final'])
final_df['tempo2_final'] = np.where(final_df['tempo2_final']=='2/2/2','2/2',final_df['tempo2_final'])
final_df['tempo2_final'] = np.where(final_df['tempo2_final']=='4/4/4','4/4',final_df['tempo2_final'])
final_df['tempo3_final'] = np.where(final_df['tempo3_final']=='1/1/1','1/1',final_df['tempo3_final'])
final_df['tempo3_final'] = np.where(final_df['tempo3_final']=='2/2/2','2/2',final_df['tempo3_final'])
final_df['tempo4_final'] = np.where(final_df['tempo4_final']=='1/1/1','1/1',final_df['tempo4_final'])
final_df['tempo4_final'] = np.where(final_df['tempo4_final']=='2/2/2','2/2',final_df['tempo4_final'])

# by default, if there is no tempo, we will assign 1/1
final_df['move1_tempo1_final'] = final_df['move1_final'] +' - ' + np.where(final_df['tempo1_final']=='','1/1',final_df['tempo1_final'])
final_df['move2_tempo2_final'] = final_df['move2_final'] +' - ' + np.where(final_df['tempo2_final']=='','1/1',final_df['tempo2_final'])
final_df['move3_tempo3_final'] = final_df['move3_final'] +' - ' + np.where(final_df['tempo3_final']=='','1/1',final_df['tempo3_final'])
final_df['move4_tempo4_final'] = final_df['move4_final'] +' - ' + np.where(final_df['tempo4_final']=='','1/1',final_df['tempo4_final'])


final_df_melt_with_tempo = pd.melt(final_df, id_vars=['bodypump_release_number', 'track_number','muscle_group','Title','Duration', 'reps1_final', 'reps2_final','reps3_final', 'reps4_final'], 
            value_vars=['move1_tempo1_final', 'move2_tempo2_final','move3_tempo3_final','move4_tempo4_final'],
            var_name='move_tempo_variable', value_name='move_tempo')

# then doing the same steps as before to add the correct number of reps to each move
final_df_melt_with_tempo['reps'] = np.where(final_df_melt_with_tempo['move_tempo_variable']=='move1_tempo1_final', final_df_melt_with_tempo['reps1_final'],\
                                    np.where(final_df_melt_with_tempo['move_tempo_variable']=='move2_tempo2_final', final_df_melt_with_tempo['reps2_final'],\
                                    np.where(final_df_melt_with_tempo['move_tempo_variable']=='move3_tempo3_final', final_df_melt_with_tempo['reps3_final'],\
                                             final_df_melt_with_tempo['reps4_final'])))

# Cleaning up the rows and columns remove rows with no reps (final) and the columns of reps1/2/3/4 and move_variable
final_df_melt_with_tempo = final_df_melt_with_tempo[final_df_melt_with_tempo['reps']!=0]
final_df_melt_with_tempo = final_df_melt_with_tempo[['bodypump_release_number', 'track_number','muscle_group','Title','Duration','move_tempo','reps']]

# Re-grouping the rows of the same track together and resetting the index
final_df_melt_with_tempo = final_df_melt_with_tempo.sort_values(by=['bodypump_release_number','track_number'], ascending=[True,True]).reset_index(drop=True)

# Finally summing the number of reps per move and per track
move_intensity_analysis = final_df_melt_with_tempo.groupby(['bodypump_release_number', 'track_number', 'muscle_group', 'Title','Duration','move_tempo'])['reps'].sum().reset_index()

# We will now have to assign an intensity to all the unique combinations of move and tempo
# Let's assign an intensity to each move tempo combination. The default is 50, the max is 100.
move_intensity_assignmnent_df = pd.DataFrame(move_intensity_analysis['move_tempo'].unique(), columns=['move_tempo'])
move_intensity_assignmnent_df['intensity']=50
move_intensity_assignmnent_df = move_intensity_assignmnent_df.sort_values(by='move_tempo').reset_index(drop=True)
move_intensity_assignmnent_df.head(10)

# we will export the dataframe to a csv to do the intensity assignment separately in Excel and then reimport
move_intensity_assignmnent_df.to_csv('move_intensity_assignment_v0.csv')

# re-importing
move_intensity_df = pd.read_csv('move_intensity_assignment_v1.csv', usecols=['move_tempo','intensity'])

# joining the intensities to the df for analysis
move_intensity_df = move_intensity_analysis.merge(move_intensity_df, left_on='move_tempo', right_on='move_tempo', how='left' )

# calculating total intensity by track
move_intensity_df['total_intensity'] = move_intensity_df['reps']*move_intensity_df['intensity']
move_intensity_analysis = move_intensity_df.groupby(['bodypump_release_number','track_number','muscle_group','Title','Duration'])['total_intensity'].sum().reset_index()

# adding it back to the total_reps_analysis_df
tracks_analysis_df = total_reps_analysis_df.merge(move_intensity_analysis[['bodypump_release_number','track_number','total_intensity']], left_on=['bodypump_release_number','track_number'], right_on=['bodypump_release_number','track_number'], how = 'left')

# removing COOLDOWN tracks
tracks_analysis_df = tracks_analysis_df[tracks_analysis_df['track_number']!='10'].reset_index()

# normalizing by dividing by the avg sec_per_rep per track type
avg_sec_per_rep_per_track_type = tracks_analysis_df.groupby(['track_number'])['sec_per_rep'].mean()
tracks_analysis_df['sec_per_rep_normalized'] = tracks_analysis_df['sec_per_rep']/ tracks_analysis_df['track_number'].map(avg_sec_per_rep_per_track_type)

# adding one more column to modulate the intensity by the 'speed' of the song, which is defined as the inverse of sec_per_rep
tracks_analysis_df['total_intensity_modulated'] = tracks_analysis_df['total_intensity'] / tracks_analysis_df['sec_per_rep_normalized']
tracks_analysis_df.drop(columns=['index'],axis=1, inplace=True) # removing the new 'index'
tracks_analysis_df['track_id'] = tracks_analysis_df['bodypump_release_number']+tracks_analysis_df['track_number'] # adding the trackID

# Replacing the titles by the titles from the full list of tracks I have
tracks_analysis_df = tracks_analysis_df.drop(columns=['Title'],axis=1)
tracks_analysis_df = tracks_analysis_df.merge(my_bodypump_tracks_df[['bodypump_release_number','track_type_id','track_title']], how='left', left_on = ['bodypump_release_number','track_number'], right_on = ['bodypump_release_number','track_type_id'])
tracks_analysis_df = tracks_analysis_df.drop(columns=['track_type_id'],axis=1)

tracks_analysis_df.head()

,bodypump_release_number,track_number,muscle_group,Duration,song_duration_sec,total_reps,total_reps_pulses,sec_per_rep,total_intensity,sec_per_rep_normalized,total_intensity_modulated,track_id,track_title
0,105,01,WARMUP,5:21,321,78,0,4.115385,3470.0,1.153536,3008.141045,10501,ALL STARS
1,105,02,SQUATS,5:40,340,94,0,3.617021,5140.0,1.228367,4184.415707,10502,DO OR DIE
2,105,03,CHEST,5:25,325,104,48,3.125000,5810.0,1.146274,5068.594560,10503,THE MAN
3,105,04,BACK,6:12,372,88,0,4.227273,4040.0,1.153241,3503.170885,10504,STAY FOR IT
4,105,05,TRICEPS,5:34,334,79,15,4.227848,4060.0,1.557846,2606.161958,10505,WHAT ABOUT US


In [357]:
# Result 2c: Creating one more dataframe to add the track level info to my personal list of tracks with enjoyment and mixes
my_bodypump_tracks_df = my_bodypump_tracks_df.merge(tracks_analysis_df.drop(columns=['bodypump_release_number','track_number','muscle_group','track_title'],axis=1), left_on='track_id', right_on='track_id', how='left')
my_bodypump_tracks_df.head()

,track_id,bodypump_release_number,track_title,track_type_id,muscle_group,difficulty,enjoyment,Duration,song_duration_sec,total_reps,total_reps_pulses,sec_per_rep,total_intensity,sec_per_rep_normalized,total_intensity_modulated
0,10607,106,COCO'S MIRACLE (CLUB MIX),07,LUNGES,10,2,6:05,365.0,116.0,76.0,3.146552,5770.0,1.089407,5296.461418
1,9608,96,ADDICTED TO A MEMORY,08,SHOULDERS,9,5,6:06,366.0,116.0,0.0,3.155172,7580.0,0.972172,7796.975670
2,10408,104,INTO DUST,08,SHOULDERS,9,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,11104,111,RESERVE,04,BACK,9,5,5:43,343.0,102.0,0.0,3.362745,5450.0,0.917389,5940.770555
4,10507,105,HOW DO YOU FEEL RIGHT NOW,07,LUNGES,9,3,5:47,347.0,104.0,56.0,3.336538,6810.0,1.155184,5895.163459


In [358]:
# Result 2d: Creating one last dataframe to add tit to the mixes
my_bodypump_mixes_df = my_bodypump_mixes_df.merge(tracks_analysis_df.drop(columns=['bodypump_release_number','track_number','muscle_group','track_title'],axis=1),left_on='track_id', right_on='track_id', how='left')
my_bodypump_mixes_df = my_bodypump_mixes_df.merge(my_bodypump_tracks_df[['track_id','difficulty','enjoyment']],left_on='track_id', right_on='track_id', how='left')

# Removing the cooldown tracks
my_bodypump_mixes_df = my_bodypump_mixes_df[my_bodypump_mixes_df['track_type_id']!='10']

# Sorting in order of the mixes
my_bodypump_mixes_df = my_bodypump_mixes_df.sort_values(by=['mix_id','track_type_id'])

# Adding a count of tracks per date
my_bodypump_mixes_df['track_date_count'] = my_bodypump_mixes_df.groupby('date').cumcount()

# Adding a count of sessions per date (if more than 9 tracks on a day, that means 2 sessions)
def assign_day_session_id(x):
    if x< 10:
        return '1' 
    else:
        return '2'

my_bodypump_mixes_df['sessions_date_count'] = my_bodypump_mixes_df['track_date_count'].apply(assign_day_session_id)

# Adding a session date id (to be able to merge with polar fitness data)
my_bodypump_mixes_df['workout_date_id'] = my_bodypump_mixes_df['date'].astype(str) + '_' + my_bodypump_mixes_df['sessions_date_count'].astype(str)

# dropping unnecessary temporary columns
my_bodypump_mixes_df = my_bodypump_mixes_df.drop(columns = ['track_date_count','sessions_date_count'], axis=1)

my_bodypump_mixes_df.head()

,date,track_id,mix_id,mix_name,track_type_id,track_title,Duration,song_duration_sec,total_reps,total_reps_pulses,sec_per_rep,total_intensity,sec_per_rep_normalized,total_intensity_modulated,difficulty,enjoyment,workout_date_id
20,2024-02-24,9601,001,Mix 1 Medium+,01,FIVE MORE HOURS,5:29,329.0,80.0,0.0,4.112500,4120.0,1.152728,3574.130911,3,3,2024-02-24_1
21,2024-02-24,11702,001,Mix 1 Medium+,02,NOTHING FOR FREE,5:22,322.0,162.0,72.0,1.987654,9020.0,0.675022,13362.522470,8,5,2024-02-24_1
22,2024-02-24,10903,001,Mix 1 Medium+,03,SATISFIED,5:22,322.0,104.0,24.0,3.096154,5380.0,1.135693,4737.194165,6,5,2024-02-24_1
23,2024-02-24,11104,001,Mix 1 Medium+,04,RESERVE,5:43,343.0,102.0,0.0,3.362745,5450.0,0.917389,5940.770555,9,5,2024-02-24_1
24,2024-02-24,11005,001,Mix 1 Medium+,05,WOMAN LIKE ME,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8,5,2024-02-24_1


In [359]:
# Specify the directory where the JSON files are located
directory = 'polar_flow_data_2024'

# Create an empty list to store the extracted data
data_list = []

for filename in os.listdir(directory):
    if filename.endswith('.json'):
        # Construct the full file path
        file_path = os.path.join(directory, filename)
        
        # Load the JSON data from the file
        with open(file_path, 'r') as file:
            data = json.load(file)
        
        # Extract the desired fields
        training_datetime = pd.to_datetime(data['startTime'])
        workout_name = data['exercises'][0]['sport']
        cardio_load = data['loadInformation']['cardioLoad']
        maximum_heart_rate = data['maximumHeartRate']
        
        # Append the extracted data to the list
        data_list.append({
            'training_datetime': training_datetime,
            'training_date': training_datetime.date(),
            'workout_name': workout_name,
            'cardio_load': cardio_load,
            'maximum_heart_rate': maximum_heart_rate
        })
        
# Create a DataFrame from the list of extracted data
polar_data_df = pd.DataFrame(data_list)

# Keeping only the BODYPUMP data
polar_data_bodypump_df = polar_data_df[polar_data_df['workout_name'] == 'LES_MILLS_BODYPUMP']
polar_data_bodypump_df = polar_data_bodypump_df.drop(columns=['workout_name'],axis=1)

# Covnverting to date format
polar_data_bodypump_df['training_date'] = pd.to_datetime(polar_data_bodypump_df['training_date'])

# Sorting dataframe in ascending order of datetime
polar_data_bodypump_df = polar_data_bodypump_df.sort_values(by='training_datetime')

# Finding days with two workout sessions in fitness data and assigning a unique date_id to each session
sessions_count_df = pd.DataFrame(polar_data_bodypump_df.groupby('training_date')['training_date'].cumcount()+1, columns=['sessions_count'])
polar_data_bodypump_df = polar_data_bodypump_df.merge(sessions_count_df, left_on=polar_data_bodypump_df.index, right_on=sessions_count_df.index)
polar_data_bodypump_df = polar_data_bodypump_df.drop(columns=['key_0'], axis=1)
polar_data_bodypump_df['training_date_id'] = polar_data_bodypump_df['training_date'].astype(str)+'_'+polar_data_bodypump_df['sessions_count'].astype(str)

polar_data_bodypump_df.head()

,training_datetime,training_date,cardio_load,maximum_heart_rate,sessions_count,training_date_id
0,2024-01-23 12:17:35.194,2024-01-23,45.6969,173,1,2024-01-23_1
1,2024-01-29 18:55:27.527,2024-01-29,61.1646,167,1,2024-01-29_1
2,2024-02-02 18:07:09.902,2024-02-02,58.6005,169,1,2024-02-02_1
3,2024-02-06 19:52:52.587,2024-02-06,31.2732,147,1,2024-02-06_1
4,2024-02-15 19:03:23.087,2024-02-15,57.5895,158,1,2024-02-15_1


In [360]:
# Before exporting mixes file, merging and adding polar aggregated data
my_bodypump_mixes_df = my_bodypump_mixes_df.merge(polar_data_bodypump_df.drop(columns=['training_date','sessions_count'],axis=1), left_on='workout_date_id', right_on='training_date_id')
my_bodypump_mixes_df = my_bodypump_mixes_df.drop(columns =['training_date_id'],axis=1)
my_bodypump_mixes_df.head()

,date,track_id,mix_id,mix_name,track_type_id,track_title,Duration,song_duration_sec,total_reps,total_reps_pulses,sec_per_rep,total_intensity,sec_per_rep_normalized,total_intensity_modulated,difficulty,enjoyment,workout_date_id,training_datetime,cardio_load,maximum_heart_rate
0,2024-02-24,9601,001,Mix 1 Medium+,01,FIVE MORE HOURS,5:29,329.0,80.0,0.0,4.112500,4120.0,1.152728,3574.130911,3,3,2024-02-24_1,2024-02-24 18:37:04.648,19.8103,161
1,2024-02-24,11702,001,Mix 1 Medium+,02,NOTHING FOR FREE,5:22,322.0,162.0,72.0,1.987654,9020.0,0.675022,13362.522470,8,5,2024-02-24_1,2024-02-24 18:37:04.648,19.8103,161
2,2024-02-24,10903,001,Mix 1 Medium+,03,SATISFIED,5:22,322.0,104.0,24.0,3.096154,5380.0,1.135693,4737.194165,6,5,2024-02-24_1,2024-02-24 18:37:04.648,19.8103,161
3,2024-02-24,11104,001,Mix 1 Medium+,04,RESERVE,5:43,343.0,102.0,0.0,3.362745,5450.0,0.917389,5940.770555,9,5,2024-02-24_1,2024-02-24 18:37:04.648,19.8103,161
4,2024-02-24,11005,001,Mix 1 Medium+,05,WOMAN LIKE ME,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8,5,2024-02-24_1,2024-02-24 18:37:04.648,19.8103,161


In [361]:
# Exporting the tables we need for visualizations
reps_by_move_analysis.to_csv('reps_by_move_analysis_v0.csv')
my_bodypump_tracks_df.to_csv('bodypump_tracks_v0.csv')
my_bodypump_mixes_df.to_csv('my_bodypump_mixes_v0.csv')